# Library

In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

## [로드] 말소 데이터

In [3]:
ex_col_df = pd.read_excel('data/220819_22.06월기준_말소_컬럼.xlsx')
ex_col_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   컬럼명     42 non-null     object
dtypes: object(1)
memory usage: 464.0+ bytes


In [4]:
ex_col_df.head()

,컬럼명
0,차량관리번호
1,차량등록번호
2,본거지법정동코드
3,소유자구분
4,차종_등록정보


In [5]:
ex_col = ex_col_df['컬럼명']
len(ex_col)

42

In [6]:
ex_col

0          차량관리번호
1          차량등록번호
2        본거지법정동코드
3           소유자구분
4         차종_등록정보
5              용도
6            차대번호
7          최초등록일자
8            차량연식
9          차량제작일자
10          검사유효일
11         제원관리번호
12       배출가스인증번호
13         배출가스등급
14        저공해조치종류
15    저공해조치구조변경YN
16         차량말소YN
17           취득일자
18           취득금액
19             차명
20        차종_제원정보
21           차종분류
22           차종유형
23          자동차형식
24           제작사명
25             연료
26           엔진형식
27            총중량
28           적재중량
29           엔진출력
30            배기량
31           승차인원
32           차량길이
33           차량너비
34           차량높이
35           원산지명
36           삭제YN
37         제원승인일자
38           구동형식
39          변속기종류
40           말소일자
41           말소코드
Name: 컬럼명, dtype: object

In [7]:
# 20.8s
ex = pd.read_table('data/220819_22.06월기준_말소.txt', header=42, names=ex_col, usecols=['차대번호', '차량말소YN'], encoding='cp949')
ex.columns

Index(['차대번호', '차량말소YN'], dtype='object')

In [8]:
ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3653054 entries, 0 to 3653053
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   차대번호    object
 1   차량말소YN  object
dtypes: object(2)
memory usage: 55.7+ MB


In [9]:
ex['차량말소YN'].unique()

array(['Y'], dtype=object)

## [로드] 컬럼명 데이터

In [10]:
col_df = pd.read_table('data/[제원정보]컬럼명.txt', sep=',')
col= col_df.columns

In [11]:
col

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류'],
      dtype='object')

## [로드] 파일

In [278]:
# 22.3s
df = pd.read_table('data/220819_22.06월기준_4등급.txt', encoding='cp949', header=40, names=col)
df.info()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_19428\3437007485.py:2: DtypeWarning: Columns (14,17,29,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table('data/220819_22.06월기준_4등급.txt', encoding='cp949', header=40, names=col)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528586 entries, 0 to 1528585
Data columns (total 40 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   차량관리번호       1528586 non-null  object 
 1   차량등록번호       1528586 non-null  object 
 2   본거지법정동코드     1528586 non-null  int64  
 3   소유자구분        1528586 non-null  object 
 4   차종_등록정보      1528584 non-null  object 
 5   용도           1528586 non-null  object 
 6   차대번호         1528586 non-null  object 
 7   최초등록일자       1528586 non-null  int64  
 8   차량연식         1528586 non-null  int64  
 9   차량제작일자       1358925 non-null  float64
 10  검사유효일        1528574 non-null  float64
 11  제원관리번호       1528586 non-null  object 
 12  배출가스인증번호     1405585 non-null  object 
 13  배출가스등급       1528586 non-null  int64  
 14  저공해조치종류      195 non-null      object 
 15  저공해조치구조변경YN  1528586 non-null  object 
 16  차량말소YN       1528586 non-null  object 
 17  취득일자         1435514 non-null  object 
 18  취득

In [279]:
df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류'],
      dtype='object')

In [280]:
df['차종_등록정보'].unique()

array(['화물', '승용', '승합', '특수', nan], dtype=object)

In [281]:
df['용도'].unique()

array(['자가용', '영업용', '관용'], dtype=object)

In [282]:
df['저공해조치종류'].unique()

array([nan, '2종 저감장치 부착', '1종 저감장치 부착', '엔진개조', '3종 저감장치 부착', '동시저감장치 부착'],
      dtype=object)

In [283]:
df['저공해조치구조변경YN'].unique()

array(['N', 'Y'], dtype=object)

In [284]:
df['차종_제원정보'].unique()

array(['화물', '승용', '승합', '특수'], dtype=object)

In [285]:
df['차종분류'].unique()

array(['일반형-카고', '다목적', '일반형-픽업', '일반', '특수용도형-특수용도형', '밴', '기타', '특수',
       '구난', '견인', '덤프', '승용겸화물', '특수용도형-탱크로리', '특수작업형', '특수용도형-피견인'],
      dtype=object)

In [286]:
df['차종유형'].unique()

array(['소형', '중형', '대형', '경형'], dtype=object)

In [287]:
df['자동차형식'].unique()

array(['HREJB-HI-D', 'JM-3VAF7-1', 'Q5D20A2-009', ..., 'SR-3ASW-7',
       'N1122/3C', 'HD050-8MT2-LGc-A'], dtype=object)

# 분석시 필요한 컬럼

In [288]:
# es_col = ['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호', '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급', '차량말소YN', '차명', '차종_제원정보', '차종분류', '차종유형', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '구동형식', '변속기종류']

# [로드] 4등급 제원정보(필수)

In [289]:
# # 14.3s
# df = pd.read_table('data/220819_22.06월기준_4등급.txt', encoding='cp949', header=40, names=col)
# df.info()

# 전처리

## 차량말소 정보 추가

In [290]:
ex.columns

Index(['차대번호', '차량말소YN'], dtype='object')

In [291]:
ex['차량말소YN'].unique()

array(['Y'], dtype=object)

In [292]:
# # 9m 53.6s
# # 1,420,563 개 있음
# # 108,023 개 없음
# len(set(df['차대번호']) - set(ex['차대번호']))

In [293]:
df['차량말소YN'].unique()

array(['N'], dtype=object)

In [294]:
df.shape

(1528586, 40)

In [295]:
df2 = df.merge(ex, on='차대번호', how='left')
df2['차량말소YN_y'].unique()

array([nan, 'Y'], dtype=object)

In [296]:
df2.shape

(1528586, 41)

In [297]:
df2['차량말소YN_x'].unique()

array(['N'], dtype=object)

In [298]:
df2['차량말소YN_y'].unique()

array([nan, 'Y'], dtype=object)

In [299]:
df3 = df2[df2['차량말소YN_y'].isnull() == True].copy()
df3.shape

(1528579, 41)

In [300]:
df3.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN_x', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '차량말소YN_y'],
      dtype='object')

In [301]:
df3 = df3.drop(['차량말소YN_x', '차량말소YN_y'], axis=1)
df3.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류'],
      dtype='object')

## 차대번호 중복

In [302]:
len(df3['차대번호'].unique())

1528557

In [303]:
df3.shape

(1528579, 39)

In [304]:
df3.loc[df3.duplicated('차대번호', keep=False) == True].sort_values('차대번호').shape

(41, 39)

In [305]:
df3.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류'],
      dtype='object')

In [306]:
df3.loc[df3.duplicated('차대번호', keep=False) == True].sort_values('차대번호').head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,배기량,승차인원,차량길이,차량너비,차량높이,원산지명,삭제YN,제원승인일자,구동형식,변속기종류
747837,1GBEL19W3XB192382-02,39고6855,1141011800,주민,승용,자가용,1GBEL19W3XB192382,20091013,1999,19991231.0,...,4300.0,6,4805.0,1980.0,1915.0,독일,N,20090805,NaN,자동
1424917,1GBEL19W3XB192382-01,36거2927,1111017500,주민,승용,자가용,1GBEL19W3XB192382,20090917,1999,20081124.0,...,4300.0,6,4805.0,1980.0,1915.0,독일,N,20090805,NaN,자동
173765,2799대구2고2891,대구2고2891,2717010200,주민,승용,자가용,KLAEM19W1NB116889,19920127,1992,NaN,...,1998.0,5,4800.0,1720.0,1417.0,대한민국,N,19910531,NaN,수동
1522531,1199서울2수2138,서울2수2138,2818510600,주민,승용,자가용,KLAEM19W1NB116889,19920127,1992,NaN,...,1998.0,5,4800.0,1720.0,1417.0,대한민국,N,19910531.0,NaN,수동
639215,KLASM69F1JB054053-02,서울4모4528,1153010800,법인,승용,자가용,KLASM69F1JB054053,19880305,1988,NaN,...,1979.0,5,4934.0,1720.0,1420.0,대한민국,N,19871205.0,NaN,수동


In [307]:
# 차대번호 중복 예시
df3.loc[df3.duplicated('차대번호', keep=False) == True].sort_values('차대번호').to_excel('analysis/[G4][차대번호]중복_예시.xlsx')

In [308]:
df3.sort_values(['최초등록일자', '검사유효일', '차대번호'], ascending=False).head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,배기량,승차인원,차량길이,차량너비,차량높이,원산지명,삭제YN,제원승인일자,구동형식,변속기종류
1528583,KMCLK19KP8C035195-01,96부7399,4824034000,주민,화물,자가용,KMCLK18KP8C035195,20220614,2008,20080924.0,...,5899.0,3,7710.0,2420.0,2960.0,대한민국,N,20080529.0,NaN,자동
1528561,KMFLA18KP8C031274-01,92도0997,4482533000,법인,화물,자가용,KMFLA18KP8C031274,20220419,2008,20080605.0,...,5899.0,2,6675.0,2255.0,2950.0,대한민국,N,20080518.0,NaN,수동
1528540,KLACA26RD8B226753-01,153구6446,2826011000,주민,승용,자가용,KLACA26RD8B226753,20210909,2008,20080221.0,...,1991.0,7,4635.0,1850.0,1720.0,대한민국,N,20070703.0,NaN,자동
1528538,KMCGK17HP8C086937-01,88조4842,4476032000,법인,화물,자가용,KMCGK17HP8C086937,20210830,2008,20080315.0,...,3933.0,3,6180.0,2030.0,2920.0,대한민국,N,20070913.0,NaN,수동
1528539,KMCYSS7JP7U084697-01,99도4857,4423038028,주민,특수,자가용,KMCYSS7JP7U084697,20210826,2007,20070227.0,...,2497.0,3,5435.0,1820.0,2060.0,대한민국,N,20060714.0,NaN,수동


In [309]:
df3.shape

(1528579, 39)

### 차대번호 중복 처리
- 최초등록일자, 검사유효일 최신

In [310]:
df3_1 = df3.sort_values(['최초등록일자', '검사유효일', '차대번호'], ascending=False).reset_index(drop=True)
df3_2 = df3_1.drop_duplicates('차대번호', keep='first').reset_index(drop=True)

In [311]:
df3_2['차대번호'].value_counts()

KMCLK18KP8C035195    1
KMHJN81VP8U766322    1
KNAJE55537K463438    1
KNAJE55537K468980    1
KNAJE55537K470819    1
                    ..
KMHNV81WBAU108544    1
KMHNV81WDAU113985    1
KMHSH81UBAU530489    1
KMHSH81UBAU541710    1
KMHJF31JPPU555457    1
Name: 차대번호, Length: 1528557, dtype: int64

In [312]:
df3_2[df3_2['차대번호'] == 'KNADA2322JS052345']['차명']

1523727    프라이드
Name: 차명, dtype: object

In [313]:
df4 = df3_2.copy()

## 연료 정보 수정
- 엔진형식, 차명으로 확인

In [314]:
f = df4.groupby('연료', as_index=False)['차대번호'].count().sort_values('차대번호', ascending=False)
f

,연료,차대번호
2,경유,1167605
5,휘발유,334494
1,LPG(액화석유가스),26418
3,기타연료,38
0,CNG(압축천연가스),1
4,알코올,1


In [315]:
g = df4.groupby('연료', as_index=False)['차대번호'].groups
g['기타연료']

Int64Index([   3059, 1486836, 1498416, 1499706, 1501418, 1502065, 1502241,
            1502591, 1503075, 1504853, 1506843, 1506983, 1507069, 1507214,
            1507417, 1507583, 1507880, 1508213, 1508592, 1508820, 1509438,
            1510445, 1511196, 1511799, 1513346, 1514189, 1516533, 1518320,
            1519203, 1519358, 1519683, 1520041, 1521843, 1522798, 1522963,
            1523260, 1524481, 1528151],
           dtype='int64')

In [316]:
# B5 : 휘발유
# G16SF : 휘발유
# H20MR : 휘발유
# MSV3 : 휘발유
# 01821300581 : 
# G15CF : 휘발유
# G4AJ : 휘발유
df4.loc[df4['연료'] == '기타연료', '엔진형식'].unique()

array(['-', 'MSV3', 'B5', 'H20MR', 'G16SF', 'G4AJ', nan, 'G15CF',
       '01821300581'], dtype=object)

In [317]:
df4.loc[df4['엔진형식'] == 'B5', '연료'] = '휘발유'
df4.loc[df4['엔진형식'] == 'G16SF', '연료'] = '휘발유'
df4.loc[df4['엔진형식'] == 'H20MR', '연료'] = '휘발유'
df4.loc[df4['엔진형식'] == 'MSV3', '연료'] = '휘발유'
df4.loc[df4['엔진형식'] == 'G15CF', '연료'] = '휘발유'
df4.loc[df4['엔진형식'] == 'G4AJ', '연료'] = '휘발유'
df4[df4['연료'] == '기타연료'][['차명', '엔진형식']]

,차명,엔진형식
3059,에어스트림탐지견수송트레일러,-
1516533,스텔라오토매틱,NaN
1518320,벤즈 230E,NaN
1524481,벤즈230E,01821300581
1528151,도요다크라운로얄싸롱,NaN


In [318]:
df4.loc[df4['차명'] == '도요다크라운로얄싸롱', '연료'] = '휘발유'
df4.loc[df4['차명'] == '스텔라오토매틱', '연료'] = '휘발유'
df4.loc[df4['차명'] == '벤즈 230E', '연료'] = '휘발유'
df4.loc[df4['차명'] == '벤즈230E', '연료'] = '휘발유'
df4[df4['연료'] == '기타연료'][['차명', '엔진형식']]

,차명,엔진형식
3059,에어스트림탐지견수송트레일러,-


In [319]:
f = df4.groupby('연료', as_index=False)['차대번호'].count().sort_values('차대번호', ascending=False)
f

,연료,차대번호
2,경유,1167605
5,휘발유,334531
1,LPG(액화석유가스),26418
0,CNG(압축천연가스),1
3,기타연료,1
4,알코올,1


In [320]:
f.columns = ['연료', '4등급']
f

,연료,4등급
2,경유,1167605
5,휘발유,334531
1,LPG(액화석유가스),26418
0,CNG(압축천연가스),1
3,기타연료,1
4,알코올,1


## [excel] 연료별 통계

In [321]:
f.to_excel('analysis/[G4]연료별_차대번호_통계.xlsx', index=False, encoding='utf-8')

## 지역별 수량 통계

## [로드] 법정동코드

In [322]:
code = pd.read_excel('data/법정동코드_2022.10.14.xlsx')
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46340 entries, 0 to 46339
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     46340 non-null  int64  
 1   시도        46340 non-null  object 
 2   시구군       46317 non-null  object 
 3   읍면동       45847 non-null  object 
 4   리         33682 non-null  object 
 5   지역3       0 non-null      float64
 6   폐지여부      46340 non-null  object 
 7   대기관리권역    46340 non-null  object 
 8   대기관리권역YN  46334 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.2+ MB


In [323]:
code.head()

,법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN
0,1100000000,서울,NaN,NaN,NaN,NaN,존재,수도권,Y
1,1111000000,서울,종로구,NaN,NaN,NaN,존재,수도권,Y
2,1111010100,서울,종로구,청운동,NaN,NaN,존재,수도권,Y
3,1111010200,서울,종로구,신교동,NaN,NaN,존재,수도권,Y
4,1111010300,서울,종로구,궁정동,NaN,NaN,존재,수도권,Y


In [324]:
code['code1'] = code['법정동코드'].astype('str').str[:5]

In [325]:
code.head()

,법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN,code1
0,1100000000,서울,NaN,NaN,NaN,NaN,존재,수도권,Y,11000
1,1111000000,서울,종로구,NaN,NaN,NaN,존재,수도권,Y,11110
2,1111010100,서울,종로구,청운동,NaN,NaN,존재,수도권,Y,11110
3,1111010200,서울,종로구,신교동,NaN,NaN,존재,수도권,Y,11110
4,1111010300,서울,종로구,궁정동,NaN,NaN,존재,수도권,Y,11110


In [326]:
code.shape

(46340, 10)

In [327]:
code2 = code.drop_duplicates(['시도', 'code1'], keep='first').reset_index(drop=True)
code2.shape

(461, 10)

In [328]:
code2.head()

,법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN,code1
0,1100000000,서울,NaN,NaN,NaN,NaN,존재,수도권,Y,11000
1,1111000000,서울,종로구,NaN,NaN,NaN,존재,수도권,Y,11110
2,1114000000,서울,중구,NaN,NaN,NaN,존재,수도권,Y,11140
3,1117000000,서울,용산구,NaN,NaN,NaN,존재,수도권,Y,11170
4,1120000000,서울,성동구,NaN,NaN,NaN,존재,수도권,Y,11200


In [329]:
code2[code2['시도'] == '세종']

,법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN,code1
122,3611000000,세종,NaN,NaN,NaN,NaN,존재,중부권,Y,36110


In [330]:
code2['code1'].dtype

dtype('O')

In [331]:
code2 = code2[['시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', 'code1']]

In [332]:
code2.head()

,시도,시구군,폐지여부,대기관리권역,대기관리권역YN,code1
0,서울,NaN,존재,수도권,Y,11000
1,서울,종로구,존재,수도권,Y,11110
2,서울,중구,존재,수도권,Y,11140
3,서울,용산구,존재,수도권,Y,11170
4,서울,성동구,존재,수도권,Y,11200


### 법정동 코드 추가

In [333]:
df4['본거지법정동코드'].head()

0    4824034000
1    4482533000
2    2826011000
3    4476032000
4    4423038028
Name: 본거지법정동코드, dtype: int64

In [334]:
df4['본거지법정동코드'].isnull().sum()

0

In [335]:
df4['code1'] = df4['본거지법정동코드'].astype('str').str[:5]
df4['code1'].head()

0    48240
1    44825
2    28260
3    44760
4    44230
Name: code1, dtype: object

In [336]:
df5 = df4.merge(code2, on='code1', how='left')
df5[['code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN']].head()

,code1,시도,시구군,폐지여부,대기관리권역,대기관리권역YN
0,48240,경남,사천시,존재,동남권,N
1,44825,충남,태안군,존재,중부권,Y
2,28260,인천,서구,존재,수도권,Y
3,44760,충남,부여군,존재,중부권,Y
4,44230,충남,논산시,존재,중부권,Y


In [337]:
df5.shape

(1528557, 45)

In [338]:
df5['시도'].isnull().sum()

0

In [339]:
df5['시도'].unique()

array(['경남', '충남', '인천', '전북', '울산', '경북', '대전', '부산', '광주', '강원', '경기',
       '전남', '충북', '대구', '서울', '제주', '세종'], dtype=object)

In [340]:
df5['시구군'].isnull().sum()

8193

In [341]:
df5['시구군'].unique()

array(['사천시', '태안군', '서구', '부여군', '논산시', '김제시', '완주군', '울주군', '구미시',
       '고성군', '거제시', '울진군', '대덕구', '중구', '원주시', '성남시 중원구', '목포시', '완도군',
       '남양주시', '청주시 흥덕구', '춘천시', '단양군', '동구', '김천시', '전주시 덕진구', '함안군',
       '영주시', '안성시', '남구', '칠곡군', '사하구', '김포시', '서산시', '달서구', '군산시',
       '종로구', '성남시 수정구', '청주시 상당구', '인제군', '장성군', '광산구', '청도군', '경산시',
       '여수시', '천안시 서북구', '고양시 일산동구', '아산시', '동대문구', '창원시 의창구', '부천시',
       '홍성군', '구리시', '포천시', '철원군', '음성군', '창원시 진해구', '부평구', '양산시', '북구',
       '보은군', '여주시', '용인시 기흥구', '평택시', '광양시', '서천군', '서초구', '당진시', '은평구',
       '천안시 동남구', '고흥군', '달성군', '충주시', '양주시', '수원시 장안구', '파주시', '예산군',
       '강릉시', '강화군', '성동구', '노원구', '안산시 상록구', '성북구', '영등포구', '구로구', '고령군',
       '영동군', '제천시', '강남구', '강동구', '미추홀구', '광진구', '시흥시', '영암군', '연제구',
       '창녕군', '해운대구', '광주시', '영천시', '진주시', '제주시', '공주시', '마포구', '사상구',
       '용인시 처인구', '서대문구', '경주시', '화순군', '횡성군', '포항시 남구', '유성구', '나주시',
       '금정구', '청주시 청원구', '연수구', '수성구', '산청군', '광명시', '용산구', '밀양시', '양양군

In [342]:
df5.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [349]:
l1 = df5.groupby(['대기관리권역', '대기관리권역YN', '연료'])['차대번호'].count().unstack()
l1

연료               CNG(압축천연가스)  LPG(액화석유가스)        경유  기타연료  알코올       휘발유
대기관리권역 대기관리권역YN                                                         
그 외    N                 NaN       1185.0   69612.0   NaN  NaN   13047.0
남부권    N                 1.0        749.0   37746.0   NaN  NaN    5880.0
       Y                 NaN       1253.0   65696.0   NaN  NaN   15180.0
동남권    N                 NaN        776.0   81976.0   1.0  NaN   12579.0
       Y                 NaN       5204.0  248857.0   NaN  NaN   66572.0
수도권    N                 NaN        133.0    9465.0   NaN  NaN    2236.0
       Y                 NaN      12596.0  436118.0   NaN  1.0  169583.0
중부권    N                 NaN        590.0   40787.0   NaN  NaN    6323.0
       Y                 NaN       3932.0  177348.0   NaN  NaN   43131.0

In [350]:
l2 = df5.groupby(['시도', '대기관리권역YN', '연료'])['차대번호'].count().unstack()
l2

연료           CNG(압축천연가스)  LPG(액화석유가스)        경유  기타연료  알코올      휘발유
시도 대기관리권역YN                                                        
강원 N                 NaN        806.0   47557.0   NaN  NaN   8645.0
경기 N                 NaN        119.0    8029.0   NaN  NaN   2018.0
   Y                 NaN       6852.0  263413.0   NaN  NaN  87318.0
경남 N                 NaN        319.0   35671.0   1.0  NaN   5813.0
   Y                 NaN        725.0   62151.0   NaN  NaN  13147.0
경북 N                 NaN        457.0   46305.0   NaN  NaN   6766.0
   Y                 NaN       1014.0   50059.0   NaN  NaN  13076.0
광주 Y                 NaN        721.0   30708.0   NaN  NaN   8419.0
대구 Y                 NaN       1675.0   51173.0   NaN  NaN  16943.0
대전 Y                 NaN       1248.0   30365.0   NaN  NaN  10577.0
부산 Y                 NaN       1517.0   58875.0   NaN  NaN  17442.0
서울 Y                 NaN       4439.0  112475.0   NaN  NaN  63609.0
세종 Y                 NaN         66.0    6790.0   NaN  NaN   1337.0
울산 Y                 NaN        273.0   26599.0   NaN  NaN   5964.0
인천 N                 NaN         14.0    1436.0   NaN  NaN    218.0
   Y                 NaN       1305.0   60230.0   NaN  1.0  18656.0
전남 N                 1.0        749.0   37746.0   NaN  NaN   5880.0
   Y                 NaN        532.0   34988.0   NaN  NaN   6761.0
전북 N                 NaN        399.0   29035.0   NaN  NaN   4261.0
   Y                 NaN        590.0   32030.0   NaN  NaN   7142.0
제주 N                 NaN        379.0   22055.0   NaN  NaN   4402.0
충남 N                 NaN         33.0    2372.0   NaN  NaN    459.0
   Y                 NaN        957.0   68208.0   NaN  NaN  14436.0
충북 N                 NaN        158.0    9380.0   NaN  NaN   1603.0
   Y                 NaN       1071.0   39955.0   NaN  NaN   9639.0

### [excel] 지역별 수량 통계

In [353]:
local = pd.ExcelWriter('analysis/[G4][차대번호]지역별_통계.xlsx', engine='openpyxl')
l1.to_excel(local, sheet_name='권역')
l2.to_excel(local, sheet_name='지역')
local.save()

# [출력] 4등급 제원정보(csv)
- 차량말소 적용, 차대번호 중복제거, 연료정보 수정, 지역명 추가

In [354]:
# 4등급 제원정보 수정(차량말소 적용, 차대번호 중복제거, 연료정보 수정, 지역명 추가)
# 30.2s
df5.to_csv('data/220819_22.06월기준_4등급2.csv', encoding='cp949')

# [로드] 4등급 제원정보(csv)

In [274]:
# 약 24s
df5 = pd.read_csv('data/220819_22.06월기준_4등급2.csv', encoding='cp949', index_col=0)
df5.info()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_456\441361917.py:2: DtypeWarning: Columns (12,15,17,29,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df5 = pd.read_csv('data/220819_22.06월기준_4등급2.csv', encoding='cp949', index_col=0)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1528557 entries, 0 to 1528556
Data columns (total 45 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   차량관리번호       1528557 non-null  object 
 1   차량등록번호       1528557 non-null  object 
 2   본거지법정동코드     1528557 non-null  int64  
 3   소유자구분        1528557 non-null  object 
 4   차종_등록정보      1528555 non-null  object 
 5   용도           1528557 non-null  object 
 6   차대번호         1528557 non-null  object 
 7   최초등록일자       1528557 non-null  int64  
 8   차량연식         1528557 non-null  int64  
 9   차량제작일자       1358915 non-null  float64
 10  검사유효일        1528545 non-null  float64
 11  제원관리번호       1528557 non-null  object 
 12  배출가스인증번호     1405566 non-null  object 
 13  배출가스등급       1528557 non-null  int64  
 14  저공해조치종류      195 non-null      object 
 15  저공해조치구조변경YN  1528557 non-null  object 
 16  취득일자         1435498 non-null  object 
 17  취득금액         1453590 non-null  float64
 18  차명

## 지역별 연료별 통계

In [275]:
df5.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [276]:
statics_by_local_fuel = df5.groupby(['시도', '연료'])['차대번호'].count()
statics_by_local_fuel

시도  연료         
강원  LPG(액화석유가스)       806
    경유              47557
    휘발유              8645
경기  LPG(액화석유가스)      6971
    경유             271442
    휘발유             89336
경남  LPG(액화석유가스)      1044
    경유              97822
    기타연료                1
    휘발유             18960
경북  LPG(액화석유가스)      1471
    경유              96364
    휘발유             19842
광주  LPG(액화석유가스)       721
    경유              30708
    휘발유              8419
대구  LPG(액화석유가스)      1675
    경유              51173
    휘발유             16943
대전  LPG(액화석유가스)      1248
    경유              30365
    휘발유             10577
부산  LPG(액화석유가스)      1517
    경유              58875
    휘발유             17442
서울  LPG(액화석유가스)      4439
    경유             112475
    휘발유             63609
세종  LPG(액화석유가스)        66
    경유               6790
    휘발유              1337
울산  LPG(액화석유가스)       273
    경유              26599
    휘발유              5964
인천  LPG(액화석유가스)      1319
    경유              61666
    알코올                 1
    휘발유             18

In [357]:
statics_by_local_fuel2 = statics_by_local_fuel.unstack(['연료'])
statics_by_local_fuel2

연료,CNG(압축천연가스),LPG(액화석유가스),경유,기타연료,알코올,휘발유
시도,,,,,,
강원,NaN,806.0,47557.0,NaN,NaN,8645.0
경기,NaN,6971.0,271442.0,NaN,NaN,89336.0
경남,NaN,1044.0,97822.0,1.0,NaN,18960.0
경북,NaN,1471.0,96364.0,NaN,NaN,19842.0
광주,NaN,721.0,30708.0,NaN,NaN,8419.0
대구,NaN,1675.0,51173.0,NaN,NaN,16943.0
대전,NaN,1248.0,30365.0,NaN,NaN,10577.0
부산,NaN,1517.0,58875.0,NaN,NaN,17442.0
서울,NaN,4439.0,112475.0,NaN,NaN,63609.0


In [358]:
statics_by_local_fuel2.to_excel('analysis/[G4][법정동코드]지역_연료별_통계.xlsx', encoding='utf-8')

# 분석

## 배출가스인증번호

### 배출가스인증번호 내역 확인

In [359]:
df5['배출가스인증번호'].info()

<class 'pandas.core.series.Series'>
Int64Index: 1528557 entries, 0 to 1528556
Series name: 배출가스인증번호
Non-Null Count    Dtype 
--------------    ----- 
1405566 non-null  object
dtypes: object(1)
memory usage: 23.3+ MB


In [360]:
df5['배출가스인증번호'].isnull().sum()

122991

In [361]:
df5['배출가스인증번호'].value_counts()

7MY-HD-14-68     268972
8MY-KM-13-19      93969
6MY-KM-14-61      76370
9MY-KM-14-41      67493
7MY-HD-14-46      60364
                  ...  
5MY-DC-23-06          1
8MY-DH-0M-24          1
 7MY-DC-24-72         1
8MY-WI-0M-24          1
7MY-MM-0M-24          1
Name: 배출가스인증번호, Length: 304, dtype: int64

In [362]:
temp = df5['배출가스인증번호'].value_counts().reset_index()
temp

,index,배출가스인증번호
0,7MY-HD-14-68,268972
1,8MY-KM-13-19,93969
2,6MY-KM-14-61,76370
3,9MY-KM-14-41,67493
4,7MY-HD-14-46,60364
...,...,...
299,5MY-DC-23-06,1
300,8MY-DH-0M-24,1
301,7MY-DC-24-72,1
302,8MY-WI-0M-24,1


In [363]:
# 제원정보 배출가스인증번호 현황
temp.to_csv('analysis/[G4][배출가스인증번호]유니크값_통계.csv', encoding='cp949')

In [364]:
df5.shape

(1528557, 45)

#### 12\. [e]배출가스인증번호

In [365]:
# - 정보
#     - 총 : 1528557
#     - null : 123003
#     - 정상
#         - 2005.01.04 이후 표기 방식([숫자문자]{3}-[문자]{2}-[숫자]{2}-[숫자]{2}, 12자리) : 1152678
#         - 2005.01.04 이전 표기 방식([숫자문자]{3}-[문자]{2}-[숫자]{2}, 9자리) : 230866
#     - 오류(null 값 제외) : 22024
#       - 예시
#           - 163      TY87-배K-02
#           - 173    7MY-KJ-0M-24
#           - 216      TY87-배K-03
#           - 236      TY87-배K-10
#           - 293      TY87-배K-03

In [366]:
# 2005년 이후 표기 방식 중 길이가 12자리가 아닌 값 : 3018
#     - [max] 차량연식 : 2011
# 2005년 이전 표기 방식 중 길이가 9자리가 아닌 값 : 1

##### 빈칸 처리

In [277]:
gas_df = df5.copy()
gas_df.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,삭제YN,제원승인일자,구동형식,변속기종류,code1,시도,시구군,폐지여부,대기관리권역,대기관리권역YN
0,KMCLK19KP8C035195-01,96부7399,4824034000,주민,화물,자가용,KMCLK18KP8C035195,20220614,2008,20080924.0,...,N,20080529.0,NaN,자동,48240,경남,사천시,존재,동남권,N
1,KMFLA18KP8C031274-01,92도0997,4482533000,법인,화물,자가용,KMFLA18KP8C031274,20220419,2008,20080605.0,...,N,20080518.0,NaN,수동,44825,충남,태안군,존재,중부권,Y
2,KLACA26RD8B226753-01,153구6446,2826011000,주민,승용,자가용,KLACA26RD8B226753,20210909,2008,20080221.0,...,N,20070703.0,NaN,자동,28260,인천,서구,존재,수도권,Y
3,KMCGK17HP8C086937-01,88조4842,4476032000,법인,화물,자가용,KMCGK17HP8C086937,20210830,2008,20080315.0,...,N,20070913.0,NaN,수동,44760,충남,부여군,존재,중부권,Y
4,KMCYSS7JP7U084697-01,99도4857,4423038028,주민,특수,자가용,KMCYSS7JP7U084697,20210826,2007,20070227.0,...,N,20060714.0,NaN,수동,44230,충남,논산시,존재,중부권,Y


In [278]:
gas_df[gas_df['배출가스인증번호'] == ' ']['배출가스인증번호']

8808        
12687       
1194061     
1236423     
1336420     
1399623     
1477301     
1492213     
1507852     
1514578     
1518320     
1528151     
Name: 배출가스인증번호, dtype: object

In [279]:
gas_df[gas_df['배출가스인증번호'] == ' ']['배출가스인증번호'].shape

(12,)

In [280]:
blank_index = gas_df[gas_df['배출가스인증번호'] == ' ']['배출가스인증번호'].index
blank_index

Int64Index([   8808,   12687, 1194061, 1236423, 1336420, 1399623, 1477301,
            1492213, 1507852, 1514578, 1518320, 1528151],
           dtype='int64')

In [281]:
gas_df.groupby(['배출가스인증번호'])['차대번호'].count()

배출가스인증번호
                   12
 7MY-DC-24-72       1
 7MY-HD-24-30       8
 8MY-DC-24-12       1
3MY-FK-08        1067
                 ... 
XMY-HD-47         924
XMY-HD-48         364
XMY-HD-55        2532
XMY-KM-47        3217
YKM-KM-12          70
Name: 차대번호, Length: 304, dtype: int64

In [282]:
gas_df[gas_df['배출가스인증번호'] == ' 7MY-DC-24-72'].index

Int64Index([810574], dtype='int64')

In [283]:
# 빈칸을 포함한 예시
gas_df.loc[810574, '배출가스인증번호']

' 7MY-DC-24-72'

In [284]:
gas_df.loc[blank_index, '배출가스인증번호'] = np.nan


In [285]:
gas_df['배출가스인증번호'] = gas_df['배출가스인증번호'].str.replace(' ', '')
gas_df.loc[810574, '배출가스인증번호']

'7MY-DC-24-72'

In [286]:
gas_df['배출가스인증번호'].isnull().sum()

123003

In [287]:
df5.loc[:, '배출가스인증번호'].value_counts()

7MY-HD-14-68     268972
8MY-KM-13-19      93969
6MY-KM-14-61      76370
9MY-KM-14-41      67493
7MY-HD-14-46      60364
                  ...  
5MY-DC-23-06          1
8MY-DH-0M-24          1
 7MY-DC-24-72         1
8MY-WI-0M-24          1
7MY-MM-0M-24          1
Name: 배출가스인증번호, Length: 304, dtype: int64

In [288]:
df5.loc[:, '배출가스인증번호'].isnull().sum()

122991

In [289]:
# 2005.01.04 이후 표기 방식 : 1152678
count = 0
p121 = re.compile('[a-zA-Z0-9]{3}[-][a-zA-Z]{2}[-][0-9]{2}[-][0-9]{2}')
idx121 = []
for i, one in tqdm(enumerate(gas_df.loc[:, '배출가스인증번호'].to_numpy())):
    if p121.search(str(one)):
        idx121.append(i)
        count += 1
count

1528557it [00:01, 807920.99it/s]


1152678

In [290]:
# 2005.01.04 이후 표기 방식이 아닌 값 : 375879
count = 0
idx122 = []
for i, one in tqdm(enumerate(gas_df.loc[:, '배출가스인증번호'].to_numpy())):
    if not p121.search(str(one)):
        idx122.append(i)
        count += 1
count

1528557it [00:01, 946646.11it/s]


375879

In [291]:
gas_df.loc[:, '배출가스인증번호'].reindex(idx122)[:5]

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: 배출가스인증번호, dtype: object

In [292]:
# 2005.01.04 이전 배출가스 표기 방식 : 230852
count = 0
p123 = re.compile('[a-zA-Z0-9]{3}[-][a-zA-Z]{2}[-][0-9]{2}')
idx123 = []
for i, one in tqdm(enumerate(gas_df.loc[idx122, '배출가스인증번호'].to_numpy())):
    if p123.search(str(one)):
        idx123.append(idx122[i])
        count += 1
count

375879it [00:00, 795655.10it/s]


230852

In [293]:
gas_df.loc[idx123, ['최초등록일자', '차량연식', '차량제작일자', '차종유형', '배출가스인증번호']][:5]

,최초등록일자,차량연식,차량제작일자,차종유형,배출가스인증번호
936,20171018,1997,19971231.0,소형,RMY-HD-20
970,20170918,1998,19981231.0,경형,VMY-HD-30
1061,20170721,1999,19991231.0,중형,WMY-HD-32
1066,20170720,2002,20021231.0,중형,WMY-HD-32
2307,20150825,1999,19991231.0,경형,VMY-HD-30


In [294]:
# null 값 : 123003
idx124 = gas_df[gas_df['배출가스인증번호'].isnull() == True]['배출가스인증번호'].index
len(idx124)

123003

In [295]:
gas_df.loc[:, '배출가스인증번호'].reindex(idx124)[:5]

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: 배출가스인증번호, dtype: object

In [296]:
# 2005.01.04 이후 표기 방식이 아닌 값 중 null 값 제외한 오류 값 : 22024
idx125 = sorted(set(idx122) - set(idx124) - set(idx123))
len(idx125)

22024

In [297]:
gas_df.loc[:, '배출가스인증번호'].reindex(idx125)[:5]

30     6MY-SS-0M-24
104    FMY-VTK-14-1
131    FMY-VTK-14-1
133    6MY-SS-0M-24
134    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [298]:
# 2005년 이상 배출가스인증번호 데이터
gas_temp = gas_df.loc[idx123, ['최초등록일자', '차량연식', '차량제작일자', '차종유형', '배출가스인증번호']]
gas_temp[gas_temp['차량연식'] >= 2005].head()

,최초등록일자,차량연식,차량제작일자,차종유형,배출가스인증번호
8096,20120409,2011,20111219.0,대형,8MY-PA-14-3
14866,20120111,2011,20111219.0,대형,8MY-PA-14-3
15195,20120109,2011,20110517.0,대형,8MY-PA-14-3
15745,20120106,2011,20110216.0,대형,8MY-PA-14-3
19226,20120102,2011,20110516.0,대형,8MY-PA-14-3


In [299]:
# 2005년 이상년도에서 맞지않는 배출가스인증번호 샘플 : 3018
gas_temp.loc[(gas_temp['차량연식'] >= 2005) & (gas_temp['배출가스인증번호'].str.len() != 12)]

,최초등록일자,차량연식,차량제작일자,차종유형,배출가스인증번호
8096,20120409,2011,20111219.0,대형,8MY-PA-14-3
14866,20120111,2011,20111219.0,대형,8MY-PA-14-3
15195,20120109,2011,20110517.0,대형,8MY-PA-14-3
15745,20120106,2011,20110216.0,대형,8MY-PA-14-3
19226,20120102,2011,20110516.0,대형,8MY-PA-14-3
...,...,...,...,...,...
1168976,20051104,2005,20050905.0,중형,4MY-PA-06
1168984,20051103,2005,20051024.0,중형,4MY-PA-06
1169185,20051014,2005,20050903.0,중형,4MY-PA-06
1169636,20050823,2005,20050421.0,대형,4MY-AD-20


In [300]:
gas_temp.loc[(gas_temp['차량연식'] >= 2005) & (gas_temp['배출가스인증번호'].str.len() != 12 )].shape

(3018, 5)

In [301]:
gas_temp.loc[(gas_temp['차량연식'] >= 2005) & (gas_temp['배출가스인증번호'].str.len() != 12 ), '차량연식'].max()

2011

In [302]:
# 2005년 미만 년도에서 맞지않는 배출가스인증번호 샘플 : 1
gas_temp.loc[(gas_temp['차량연식'] < 2005) & (gas_temp['배출가스인증번호'].str.len() != 9 )]

,최초등록일자,차량연식,차량제작일자,차종유형,배출가스인증번호
534831,20091013,1999,19991231.0,대형,9MY-PP-14-W38


In [303]:
num_by_gas_num = gas_df.groupby('배출가스인증번호')['차대번호'].count().reset_index()
num_by_gas_num

,배출가스인증번호,차대번호
0,3MY-FK-08,1067
1,4MY-AD-20,38
2,4MY-DC-33,7
3,4MY-PA-06,13
4,5MY-AD-13-06,392
...,...,...
295,XMY-HD-47,924
296,XMY-HD-48,364
297,XMY-HD-55,2532
298,XMY-KM-47,3217


In [304]:
num_by_gas_num.to_excel('analysis/[G4][배출가스인증번호]유니크값_통계.xlsx', encoding='utf-8')

In [305]:
gas_df['배출가스인증번호'].isnull().sum()

123003

##### 오류 배출가스인증번호 유형 통계

In [306]:
# - 총 오류 샘플 수 : 22024
#     - 1번째 부분 3자리가 아닌 경우 : 17840
#         - 빈 값(' ')인 경우 : 0
#         - 바('-') 기호가 없는 경우 : 6
#         - 1번째 부분 1자리인 경우(n-XXX-XX-XX) : 0
#         - 1번째 부분 2자리인 경우(nn-XXX-XX-XX) : 0
#         - 1번째 부분 4자리인 경우(nnnn-XXX-XX-XX) : 17834(17840)
#
#     - 2번째 부분 2자리가 아닌 경우 : 2827
#         - 2번째 부분 1자리인 경우(XXX-n-XX-XX) : 0
#         - 2번째 부분 3자리인 경우(XXX-nnn-XX-XX) : 2827
#
#     - 2번째 부분 2자리이면서 숫자만 있는 경우 : 0
#
#     - 3번째 부분 2자리가 아닌 경우 : 1
#         - 3번째 부분 1자리인 경우(XXX-XX-n-XX) : 1
#
#     - 3번째 부분 2자리이면서 문자가 있는 경우 : 1356(19191)
#
#     - 4번째 부분 2자리가 아닌 경우 : 0(1999)
#         - 중복 오류 : 1999
#     - 4번째 부분 2자리이면서 문자가 있는 경우 : 0(19191)
#         - 중복 오류 : 19191

In [307]:
gas_df.loc[idx125[:5], '배출가스인증번호']

30     6MY-SS-0M-24
104    FMY-VTK-14-1
131    FMY-VTK-14-1
133    6MY-SS-0M-24
134    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [308]:
gas_temp2 = gas_df.loc[idx125, '배출가스인증번호']
gas_temp2.head()

30     6MY-SS-0M-24
104    FMY-VTK-14-1
131    FMY-VTK-14-1
133    6MY-SS-0M-24
134    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

###### 1번째 자리 오류 유형

In [309]:
# 1번째 부분 3자리가 아닌 경우 : 17840
#     - 빈 값(' ')인 경우 : 0
#     - 바('-') 기호가 없는 경우 : 6
#     - 1자리인 경우 : 0
#     - 2자리인 경우 : 0
#     - 4자리인 경우 : 17836

In [310]:
# 1번째 부분 3자리가 아닌 경우 : 17840
idx126 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) != 3:
            idx126.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 468513.25it/s]


17840

In [311]:
gas_df.loc[idx126, '배출가스인증번호']

877898           C135
891638           C135
898807           C135
915591           C135
960831           C135
              ...    
1528547    TY87-배K-03
1528548    TY87-배K-03
1528549    TY87-배K-03
1528551    TY87-배K-03
1528552    TY87-배K-03
Name: 배출가스인증번호, Length: 17840, dtype: object

In [312]:
gas_df.loc[idx126[0], '배출가스인증번호']

'C135'

In [313]:
# 빈 값(' ')인 경우 : 0
count = 0
idx126_1 = []
for i, one in enumerate(gas_df.loc[idx126, '배출가스인증번호']):
    if one == ' ':
        idx126_1.append(gas_df.loc[idx126, '배출가스인증번호'].index[i])
        count += 1
count

0

In [314]:
gas_df.loc[idx126_1, '배출가스인증번호']

Series([], Name: 배출가스인증번호, dtype: object)

In [315]:
# 바('-') 기호가 없는 경우 : 6
count = 0
idx126_2 = []
for i, one in enumerate(gas_df.loc[idx126, '배출가스인증번호']):
    if '-' not in one:
        idx126_2.append(gas_df.loc[idx126, '배출가스인증번호'].index[i])
        count += 1
count

6

In [316]:
gas_df.loc[list(set(idx126_2) - set(idx126_1)), '배출가스인증번호']

973286    C135
915591    C135
891638    C135
877898    C135
898807    C135
960831    C135
Name: 배출가스인증번호, dtype: object

In [317]:
len(list(set(idx126_2) - set(idx126_1)))

6

In [318]:
# 1번째 부분 1자리인 경우(n-XXX-XX-XX) : 0
idx126_3 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) == 1:
            idx126_3.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 957376.58it/s]


0

In [319]:
gas_df.loc[idx126_3[:5], '배출가스인증번호']

Series([], Name: 배출가스인증번호, dtype: object)

In [320]:
set(idx126_1) == set(idx126_3)

True

In [321]:
# 1번째 부분 2자리인 경우(nn-XXX-XX-XX) : 0
idx126_4 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) == 2:
            idx126_4.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 917104.51it/s]


0

In [322]:
# 1번째 부분 4자리인 경우(nnnn-XXX-XX-XX) : 17834(17840)
idx126_5 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) == 4:
            idx126_5.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 478653.56it/s]


17840

In [323]:
# 4자리인 경우(앞에서 중복된 샘플 제거) : 17834
len(set(idx126_5) - set(idx126_1) - set(idx126_2))

17834

###### 2번째 자리 오류 유형

In [324]:
# 2번째 부분 2자리가 아닌 경우 : 2827
#   - 2번째 부분 1자리인 경우(XXX-n-XX-XX) : 0
#   - 2번째 부분 3자리인 경우(XXX-nnn-XX-XX) : 2827
# 2번째 부분 2자리이면서 숫자만 있는 경우 : 0

In [325]:
# 2번째 부분 2자리가 아닌 경우 : 2827
idx127 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) != 2:
            idx127.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 759278.59it/s]


2827

In [326]:
len(set(idx127) - set(idx126))

2827

In [327]:
gas_df.loc[idx127[:5], '배출가스인증번호']

104    FMY-VTK-14-1
131    FMY-VTK-14-1
134    FMY-VTK-14-1
135    FMY-VTK-14-1
137    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [328]:
# 2번째 부분 1자리인 경우(XXX-n-XX-XX) : 0
idx127_1 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) == 1:
            idx127_1.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 733696.72it/s]


0

In [329]:
gas_df.iloc[idx127_1[:5], 12]

Series([], Name: 배출가스인증번호, dtype: object)

In [330]:
# 2번째 부분 3자리인 경우(XXX-nnn-XX-XX) : 2827
idx127_3 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) == 3:
            idx127_3.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 786079.54it/s]


2827

In [331]:
gas_df.loc[idx127_3[:5], '배출가스인증번호']

104    FMY-VTK-14-1
131    FMY-VTK-14-1
134    FMY-VTK-14-1
135    FMY-VTK-14-1
137    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [332]:
# 2번째 부분 4자리인 경우(XXX-nnnn-XX-XX) : 0
idx127_4 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) == 4:
            idx127_4.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 880763.45it/s]


0

In [333]:
# 2번째 부분 2자리이면서 숫자만 있는 경우 : 0
idx127_5 = []
p127_5 = re.compile('[0-9]+')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if (len(str(one).split('-')[1]) == 2) and p127_5.search(str(one).split('-')[1]):
            idx127_5.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 564544.89it/s]


0

###### 3번째 자리 오류 유형

In [334]:
# 3번째 부분 2자리가 아닌 경우 : 1
#   - 3번째 부분 1자리인 경우(XXX-XX-n-XX) : 1
# 3번째 부분 2자리이면서 문자가 있는 경우 : 1356

In [335]:
# 3번째 부분 2자리가 아닌 경우 : 1
idx128 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[2]) != 2:
            idx128.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 815186.92it/s]


1

In [336]:
gas_df.loc[idx128, '배출가스인증번호']

1187545    NMY-JK-S-107
Name: 배출가스인증번호, dtype: object

In [337]:
len(set(idx128) - set(idx126) - set(idx127))

1

In [338]:
# 3번째 부분 2자리이면서 문자가 있는 경우 : 1356(19191)
idx128_1 = []
p128_1 = re.compile('[a-zA-Z가-힣]+')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if (len(str(one).split('-')[2]) == 2) and p128_1.search(str(one).split('-')[1]):
            idx128_1.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 289732.31it/s]


22017

In [339]:
# 3번째 부분 2자리이면서 문자가 있는 경우(앞에서 중복된 샘플 제거) : 1356
len(set(idx128_1) - set(idx126) - set(idx127) - set(idx128))

1356

In [340]:
gas_df.loc[list(set(idx128_1) - set(idx126) - set(idx127) - set(idx128))[:5], '배출가스인증번호']

958464    7MY-HC-0M-24
933891    8MY-HS-0M-24
901127    7MY-CH-0M-24
958472    6MY-SS-0M-24
958477    8MY-HT-0M-24
Name: 배출가스인증번호, dtype: object

###### 4번째 자리 오류 유형

In [341]:
# 4번째 부분 2자리가 아닌 경우 : 0(1999)
#   - 중복 오류 : 1999
# 4번째 부분 2자리이면서 문자가 있는 경우 : 0(19191)

In [342]:
# 4번째 부분 2자리가 아닌 경우 : 0(1999)
idx129 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[3]) != 2:
            idx129.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 786420.84it/s]


1999

In [343]:
gas_df.loc[idx129[:5], '배출가스인증번호']

104    FMY-VTK-14-1
131    FMY-VTK-14-1
134    FMY-VTK-14-1
135    FMY-VTK-14-1
137    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [344]:
# 중복 추출된 샘플 제거
len(set(idx129) - set(idx126) - set(idx127) - set(idx128))

0

In [345]:
# 4번째 부분 2자리이면서 문자가 있는 경우 : 0(19191)
idx129_1 = []
p129_1 = re.compile('[a-zA-Z가-힣]+')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if (len(str(one).split('-')[1]) == 2) and p129_1.search(str(one).split('-')[1]):
            idx129_1.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 314502.47it/s]


19191

In [346]:
len(set(idx129_1) - set(idx126) - set(idx127) - set(idx128_1))

1

###### 한글이 포함된 경우

In [347]:
idx1210 = []
p1210 = re.compile('[가-힣]')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if p1210.search(one):
            idx1210.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 431662.54it/s]


17834

In [348]:
len(set(idx1210) - set(idx129) - set(idx126) - set(idx127) - set(idx128))

0

#### 배출가스인증번호 미부여 현황
- 필요한 컬럼 : 배출가스인증번호, 차량연식, 차량제작일자, 차명, 차종_제원정보, 차종분류, 차종유형, 자동차형식, 제작사명, 연료

In [349]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [350]:
gas_df.loc[gas_df['배출가스인증번호'].isnull() == True, ['배출가스인증번호', '차량연식', '차량제작일자', '차명', '차종_제원정보', '차종분류', '차종유형', '제작사명', '연료']].head()

,배출가스인증번호,차량연식,차량제작일자,차명,차종_제원정보,차종분류,차종유형,제작사명,연료
0,NaN,2008,20080924.0,메가트럭군물자수송,화물,특수용도형-특수용도형,대형,현대자동차(주),경유
1,NaN,2008,20080605.0,신정4㎥급진공노면총소차,화물,특수용도형-특수용도형,대형,신정개발특장차 주식회사,경유
2,NaN,2008,20080221.0,윈스톰2.0S디젤,승용,다목적,중형,한국지엠주식회사,경유
3,NaN,2008,20080315.0,마이티냉동차,화물,특수용도형-특수용도형,중형,현대자동차(주),경유
4,NaN,2007,20070227.0,리베로슈퍼렉커(LIBERO),특수,구난,중형,현대자동차(주),경유


In [351]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [352]:
gas_df.loc[gas_df['배출가스인증번호'].isnull() == True].shape

(123003, 45)

In [354]:
null_num_df = gas_df.loc[gas_df['배출가스인증번호'].isnull() == True]

In [356]:
null_num_df.shape

(123003, 45)

##### [출력] 배인번호 미부여 샘플

In [445]:
# [출력]4등급 차량 배출가스인증번호 미부여 샘플
null_num_df.to_csv('analysis/[G4][배출가스인증번호]미부여_샘플.csv', encoding='cp949')

In [363]:
null_num_df[['엔진형식', '차명', '차량연식','차대번호']].head()

,엔진형식,차명,차량연식,차대번호
0,D6GA,메가트럭군물자수송,2008,KMCLK18KP8C035195
1,D6GA,신정4㎥급진공노면총소차,2008,KMFLA18KP8C031274
2,Z20S,윈스톰2.0S디젤,2008,KLACA26RD8B226753
3,D4GA,마이티냉동차,2008,KMCGK17HP8C086937
4,D4CB,리베로슈퍼렉커(LIBERO),2007,KMCYSS7JP7U084697


In [364]:
null_num_df.groupby(['엔진형식', '차명'])['차대번호'].count().sort_values(ascending=False)

엔진형식          차명                   
B3            프라이드                     9885
F8C           티코(TICO)                 9435
A15MF         에스페로1.5MT                4941
C18LE         프린스1.8AT                 4591
              프린스1.8MT                 4277
                                       ... 
62M           PORSCHE 911 CARRERA 2       1
              PORSCHE 911카레라2             1
              포르쉐911카레라                   1
              포르쉐911카레라카브리올레              1
vg30-858076a  닛산세드릭                       1
Name: 차대번호, Length: 3103, dtype: int64

In [448]:
null_num_df.groupby(['엔진형식', '차명'])['차대번호'].count().sort_values(ascending=False).to_excel('analysis/[G4][배출가스인증번호]미부여샘플_통계.xlsx')

##### 배출가스인증번호 미부여 데이터 확인

In [365]:
# 배출가스인증번호 미부여 데이터 중
# - 동일 엔진형식 중 1천대 이상의 샘플만 배출가스인증번호 유무 확인

In [366]:
miss_gas_num_df = null_num_df.groupby(['엔진형식', '차명'])['차대번호'].count().sort_values(ascending=False).reset_index()
miss_gas_num_df

,엔진형식,차명,차대번호
0,B3,프라이드,9885
1,F8C,티코(TICO),9435
2,A15MF,에스페로1.5MT,4941
3,C18LE,프린스1.8AT,4591
4,C18LE,프린스1.8MT,4277
...,...,...,...
3098,62M,PORSCHE 911 CARRERA 2,1
3099,62M,PORSCHE 911카레라2,1
3100,62M,포르쉐911카레라,1
3101,62M,포르쉐911카레라카브리올레,1


In [367]:
temp = miss_gas_num_df[miss_gas_num_df['차대번호'] >= 1000]
temp

,엔진형식,차명,차대번호
0,B3,프라이드,9885
1,F8C,티코(TICO),9435
2,A15MF,에스페로1.5MT,4941
3,C18LE,프린스1.8AT,4591
4,C18LE,프린스1.8MT,4277
5,F8C,티코(TICO)오토,2710
6,G15SF,르망,2571
7,C20LE,수퍼살롱2.0AT,2299
8,A15DMS,누비라1.5,2122
9,G4AJ,프레스토,2074


In [368]:
# 제원정보에서 해당 엔진형식을 통해 배출가스인증번호 확인 여부 확인
count = 0
for one, two in zip(temp['엔진형식'], temp['차명']):
    if gas_df.loc[ (gas_df['엔진형식'] == one) & (gas_df['차명'] == two), ['엔진형식' ,'배출가스인증번호', '차명'] ].drop_duplicates(['배출가스인증번호', '차명'], keep='first').dropna(subset=['배출가스인증번호']).shape[0] != 0:
        print(one)
        print( gas_df.loc[ (gas_df['엔진형식'] == one) & (gas_df['차명'] == two), ['엔진형식' ,'배출가스인증번호', '차명'] ].drop_duplicates(['배출가스인증번호', '차명'], keep='first').dropna(subset=['배출가스인증번호']))
        print()
        count += 1 

B3
        엔진형식    배출가스인증번호    차명
1480586   B3  TY87-배K-03  프라이드

G15SF
          엔진형식    배출가스인증번호  차명
1349480  G15SF  TY87-배K-02  르망

G4CS
         엔진형식   배출가스인증번호      차명
1223970  G4CS  LMY-HD-08  그랜저2.4

B3
        엔진형식   배출가스인증번호   차명
1225014   B3  PMY-KM-20  아벨라

CD800
          엔진형식   배출가스인증번호   차명
1264214  CD800  SMY-KM-04  타우너

D5244T
          엔진형식     배출가스인증번호        차명
268522  D5244T  6MY-PA-14-4  볼보 S80D5



In [369]:
count

6

In [370]:
gas_df.loc[gas_df['엔진형식'] == 'B3', ['엔진형식' ,'배출가스인증번호', '차명']].drop_duplicates(['배출가스인증번호', '차명'])

,엔진형식,배출가스인증번호,차명
211236,B3,PMY-KM-20,KIA AVELLA
1170976,B3,NaN,프라이드베타
1171409,B3,NaN,아벨라
1173517,B3,NaN,프라이드
1183908,B3,NaN,프라이드웨곤
1189503,B3,NaN,프라이드오토
1200996,B3,NaN,프라이드프렌드
1201892,B3,PMY-KM-20,델타
1205133,B3,NaN,프라이드밴
1212358,B3,NaN,프라이드장애자차


In [371]:
gas_df.loc[gas_df['엔진형식'] == 'B3', ['엔진형식' ,'배출가스인증번호', '차명']].drop_duplicates(['배출가스인증번호', '차명']).dropna(subset=['배출가스인증번호'])

,엔진형식,배출가스인증번호,차명
211236,B3,PMY-KM-20,KIA AVELLA
1201892,B3,PMY-KM-20,델타
1225014,B3,PMY-KM-20,아벨라
1227811,B3,PMY-KM-20,아벨라운전교습차
1228534,B3,PMY-KM-20,아벨라델타운전교습차
1305234,B3,TY87-배K-03,프라이드베타오토
1373305,B3,RMY-KM-25,프라이드베타
1376211,B3,TY87-배K-03,프라이드베타
1480586,B3,TY87-배K-03,프라이드


In [372]:
# 제원정보에서 해당 엔진형식을 통해 배출가스인증번호 확인 여부 확인
t_temp = pd.DataFrame()
for one in temp['엔진형식']:
    temp = gas_df.loc[gas_df['엔진형식'] == one, ['엔진형식' ,'배출가스인증번호', '차명']].drop_duplicates(['배출가스인증번호', '차명']).dropna(subset=['배출가스인증번호'])
    if temp.shape[0] != 0:
        t_temp = pd.concat([t_temp, temp], ignore_index=True)

In [373]:
t_temp

,엔진형식,배출가스인증번호,차명
0,B3,PMY-KM-20,KIA AVELLA
1,B3,PMY-KM-20,델타
2,B3,PMY-KM-20,아벨라
3,B3,PMY-KM-20,아벨라운전교습차
4,B3,PMY-KM-20,아벨라델타운전교습차
5,B3,TY87-배K-03,프라이드베타오토
6,B3,RMY-KM-25,프라이드베타
7,B3,TY87-배K-03,프라이드베타
8,B3,TY87-배K-03,프라이드
9,G15SF,TY87-배K-02,르망


In [374]:
t_temp.drop_duplicates(['엔진형식', '배출가스인증번호', '차명']).shape

(36, 3)

In [375]:
t_temp = t_temp.drop_duplicates(['엔진형식', '배출가스인증번호', '차명'])
t_temp

,엔진형식,배출가스인증번호,차명
0,B3,PMY-KM-20,KIA AVELLA
1,B3,PMY-KM-20,델타
2,B3,PMY-KM-20,아벨라
3,B3,PMY-KM-20,아벨라운전교습차
4,B3,PMY-KM-20,아벨라델타운전교습차
5,B3,TY87-배K-03,프라이드베타오토
6,B3,RMY-KM-25,프라이드베타
7,B3,TY87-배K-03,프라이드베타
8,B3,TY87-배K-03,프라이드
9,G15SF,TY87-배K-02,르망


###### [출력] 배인번호 미부여 데이터 중 처리가능 샘플

In [460]:
# 미부여 데이터 처리 가능 샘플 추출
t_temp.to_excel('analysis/[G4][배출가스인증번호]미부여_처리용_샘플.xlsx', index=False)

##### 배인번호별, 제원관리번호별 통계

In [358]:
null_num_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [376]:
t_temp2 = t_temp.drop_duplicates(['엔진형식', '배출가스인증번호'], keep='first').reset_index(drop=True)
t_temp2

,엔진형식,배출가스인증번호,차명
0,B3,PMY-KM-20,KIA AVELLA
1,B3,TY87-배K-03,프라이드베타오토
2,B3,RMY-KM-25,프라이드베타
3,G15SF,TY87-배K-02,르망
4,G4AJ,KMY-HD-04,엑셀오토메틱
5,G4AJ,KMY-HD-02,엑셀
6,G4AJ,KMY-HD-01,엑셀
7,G4CS,TMY-HD-26,스타렉스9인승오토매틱
8,G4CS,LMY-HD-08,그랜저2.4
9,G4CS,KMY-HD-03,쏘나타 오토매틱


In [379]:
null_num_df.shape

(123003, 45)

In [380]:
null_num_df2 = null_num_df.merge(t_temp2, on=['배출가스인증번호', '엔진형식'], how='left')
null_num_df2.shape

(123003, 46)

In [381]:
null_num_df2.groupby(['배출가스인증번호', '제원관리번호']).count()

,,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,제원승인일자,구동형식,변속기종류,code1,시도,시구군,폐지여부,대기관리권역,대기관리권역YN,차명_y
배출가스인증번호,제원관리번호,,,,,,,,,,,,,,,,,,,,,


##### 제원관리번호 이용 미부여 배인 번호 부여

In [604]:
gas_df['제원관리번호'].isnull().sum()

0

In [605]:
len(gas_df['제원관리번호'].unique())

15312

In [609]:
temp = gas_df[['차대번호', '제원관리번호', '배출가스인증번호', '엔진형식', '차명']].drop_duplicates(['제원관리번호', '배출가스인증번호', '엔진형식', '차명'], keep='first').sort_values(['제원관리번호', '배출가스인증번호', '엔진형식', '차명'])
temp

,차대번호,제원관리번호,배출가스인증번호,엔진형식,차명
1171546,KL90E3ACGXSZ00001,00030110800000304,NaN,5S,도요다 해리어
1171777,KMHWF35V41A387514,00030112400001628,NaN,G6BV,쏘나타2.5GOLD
2039,KL90B2M9GWSZ00001,00031105000011116,NaN,12A,ROVER MINI
1171864,KL90A3AAGVSZ00002,00031105000011203,NaN,111921,벤츠C180
2840,KNAMC76439S281271,00031105000011314,7MY-KM-14-64,J3,카니발
...,...,...,...,...,...
150,YV2XT40K5JA834376,BRX20004600013317,FMY-VTK-14-1,D13,FM 카고
739,YV2V001D2HZ114313,BRX20004700003317,FMY-VTK-14-3,D8,FE 카고
736,YV2V001D3HZ114286,BRX20004800003317,FMY-VTK-14-3,D8,FE 윙바디
464,YV2XT40K9JA819380,BZB20003100003318,FMY-VTK-14-1,D13,삼부26톤볼보트럭


###### [출력] 동일 제원관리번호 차량 샘플

In [608]:
temp.to_excel('analysis/[G4][제원관리번호]동일차량확인.xlsx')

In [610]:
temp = temp.reset_index()
temp

,index,차대번호,제원관리번호,배출가스인증번호,엔진형식,차명
0,1171546,KL90E3ACGXSZ00001,00030110800000304,NaN,5S,도요다 해리어
1,1171777,KMHWF35V41A387514,00030112400001628,NaN,G6BV,쏘나타2.5GOLD
2,2039,KL90B2M9GWSZ00001,00031105000011116,NaN,12A,ROVER MINI
3,1171864,KL90A3AAGVSZ00002,00031105000011203,NaN,111921,벤츠C180
4,2840,KNAMC76439S281271,00031105000011314,7MY-KM-14-64,J3,카니발
...,...,...,...,...,...,...
15536,150,YV2XT40K5JA834376,BRX20004600013317,FMY-VTK-14-1,D13,FM 카고
15537,739,YV2V001D2HZ114313,BRX20004700003317,FMY-VTK-14-3,D8,FE 카고
15538,736,YV2V001D3HZ114286,BRX20004800003317,FMY-VTK-14-3,D8,FE 윙바디
15539,464,YV2XT40K9JA819380,BZB20003100003318,FMY-VTK-14-1,D13,삼부26톤볼보트럭


In [611]:
no_list = []
m_no = temp['제원관리번호'].to_list()
for one in tqdm(temp['제원관리번호']):
    n = m_no.count(one)
    no_list.append(n)

100%|██████████| 15541/15541 [00:04<00:00, 3395.22it/s]


In [618]:
m_no[:10]

['00030110800000304',
 '00030112400001628',
 '00031105000011116',
 '00031105000011203',
 '00031105000011314',
 '00031105000011412',
 '00031105000021116',
 '00031105000031116',
 '00031105000051211',
 '00031105000061214']

In [619]:
m_no.count('00031105000021116')

1

In [622]:
no_list.index(2)

1514

In [647]:
no_list.index(3)

3379

In [649]:
m_no[3379]

'10021000040002'

In [639]:
no_list.index(4)

14587

In [650]:
m_no[14587]

'B3C10016600003307'

In [637]:
no_list.index(5)

3629

In [625]:
temp['동일제원관리번호_개수'] = no_list

In [651]:
temp.groupby(['동일제원관리번호_개수'])['제원관리번호'].count()

동일제원관리번호_개수
1    15096
2      412
3       24
4        4
5        5
Name: 제원관리번호, dtype: int64

In [654]:
temp.loc[temp['동일제원관리번호_개수'] == 2, '제원관리번호'].head()

1514    00120002200031307
1515    00120002200031307
2484       10014700070001
2485       10014700070001
2918       10016000000000
Name: 제원관리번호, dtype: object

In [632]:
gas_df.loc[(gas_df['제원관리번호'] == '00120002200031307') & (gas_df['배출가스인증번호'].isnull() == True), '차대번호']

Series([], Name: 차대번호, dtype: object)

In [633]:
len(gas_df.loc[(gas_df['제원관리번호'] == '00120002200031307') & (gas_df['배출가스인증번호'].isnull() == True), '차대번호'])

0

In [656]:
count2 = 0
tes = temp.loc[temp['동일제원관리번호_개수'] == 2, '제원관리번호'].unique()
for one in tqdm(tes):
    count2 += len(gas_df.loc[(gas_df['제원관리번호'] == one) & (gas_df['배출가스인증번호'].isnull() == True), '차대번호'])

100%|██████████| 206/206 [00:34<00:00,  5.93it/s]


In [657]:
count2

3

In [658]:
count3 = 0
tes = temp.loc[temp['동일제원관리번호_개수'] == 3, '제원관리번호'].unique()
for one in tqdm(tes):
    count3 += len(gas_df.loc[(gas_df['제원관리번호'] == one) & (gas_df['배출가스인증번호'].isnull() == True), '차대번호'])

100%|██████████| 8/8 [00:01<00:00,  5.84it/s]


In [659]:
count3

0

In [660]:
count4 = 0
tes = temp.loc[temp['동일제원관리번호_개수'] == 4, '제원관리번호'].unique()
for one in tqdm(tes):
    count4 += len(gas_df.loc[(gas_df['제원관리번호'] == one) & (gas_df['배출가스인증번호'].isnull() == True), '차대번호'])

100%|██████████| 1/1 [00:00<00:00,  5.92it/s]


In [661]:
count4

1

In [662]:
count5 = 0
tes = temp.loc[temp['동일제원관리번호_개수'] == 5, '제원관리번호'].unique()
for one in tqdm(tes):
    count5 += len(gas_df.loc[(gas_df['제원관리번호'] == one) & (gas_df['배출가스인증번호'].isnull() == True), '차대번호'])

100%|██████████| 1/1 [00:00<00:00,  6.10it/s]


In [663]:
count5

0

###### 동일 제원관리번호로 기입 할 수 있는 샘플

In [664]:
# 동일 제원관리번호로 기입 할 수 있는 샘플 수 : 4
count2 + count3 + count4 + count5

4

### 배출가스인증번호별 통계

In [183]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [184]:
gas_df.shape

(1528557, 45)

In [185]:
len(gas_df['차대번호'].unique())

1528557

In [186]:
gas_df.loc[gas_df.duplicated('차대번호', keep=False) == True, ['차량관리번호', '차량등록번호', '차대번호', '배출가스인증번호']].sort_values('차대번호')

,차량관리번호,차량등록번호,차대번호,배출가스인증번호


In [187]:
gas_df.groupby('배출가스인증번호')['차대번호'].count()

배출가스인증번호
3MY-FK-08       1067
4MY-AD-20         38
4MY-DC-33          7
4MY-PA-06         13
5MY-AD-13-06     392
                ... 
XMY-HD-47        924
XMY-HD-48        364
XMY-HD-55       2532
XMY-KM-47       3217
YKM-KM-12         70
Name: 차대번호, Length: 300, dtype: int64

In [188]:
gas_df['배출가스인증번호'].isnull().sum()

123003

#### [출력] 배인번호별 통계

In [467]:
gas_df.groupby('배출가스인증번호')['차대번호'].count().to_excel('analysis/[G4][배출가스인증번호]번호별_통계.xlsx')

### 총중량으로 분류 현황

In [189]:
gas_df.shape

(1528557, 45)

In [190]:
gas_df['총중량'].unique()

array([10825., 11075.,  2215., ...,  1512.,  1197.,  1436.])

In [191]:
type(gas_df['총중량'][0])

numpy.float64

In [192]:
gas_df['총중량'].isnull().sum()

1

In [193]:
gas_df['총중량'] = gas_df['총중량'].fillna(-1)

In [194]:
gas_df['총중량'].isnull().sum()

0

In [195]:
weight_list = []
for i, one in tqdm(enumerate(gas_df['총중량'])):
    if one == -1:
        weight_list.append('미기입')
    elif one < 1500:
        weight_list.append('1.5ton 미만')
    elif one >= 1500 and one < 2000:
        weight_list.append('1.5ton 이상 2.0ton 미만')
    elif one >= 2000 and one < 2500:
        weight_list.append('2.0ton 이상 2.5ton 미만')
    elif one >= 2500 and one < 3000:
        weight_list.append('2.5ton 이상 3.0ton 미만')
    elif one >= 3000 and one < 3500:
        weight_list.append('3.0ton 이상 3.5ton 미만')
    elif one >= 3500 and one < 4000:
        weight_list.append('3.5ton 이상 4.0ton 미만')
    elif one >= 4000 and one < 4500:
        weight_list.append('4.0ton 이상 4.5ton 미만')
    elif one >= 4500 and one < 5000:
        weight_list.append('4.5ton 이상 5.0ton 미만')
    elif one >= 5000 and one < 5500:
        weight_list.append('5.0ton 이상 5.5ton 미만')
    elif one >= 5500 and one < 6000:
        weight_list.append('5.5ton 이상 6.0ton 미만')
    elif one >= 6000 and one < 8000:
        weight_list.append('6.0ton 이상 8.0ton 미만')
    elif one >= 8000 and one < 10000:
        weight_list.append('8.0ton 이상 10.0ton 미만')
    elif one >= 10000:
        weight_list.append('10.0ton 이상')

1528557it [00:01, 1040310.49it/s]


In [196]:
len(weight_list)

1528557

In [197]:
gas_df['총중량_분류'] = weight_list

In [198]:
total_weight_df = gas_df.groupby('총중량_분류')['차대번호'].count().reset_index()
total_weight_df

,총중량_분류,차대번호
0,1.5ton 미만,198424
1,1.5ton 이상 2.0ton 미만,330997
2,10.0ton 이상,30469
3,2.0ton 이상 2.5ton 미만,240259
4,2.5ton 이상 3.0ton 미만,436562
5,3.0ton 이상 3.5ton 미만,262435
6,3.5ton 이상 4.0ton 미만,135
7,4.0ton 이상 4.5ton 미만,192
8,4.5ton 이상 5.0ton 미만,23
9,5.0ton 이상 5.5ton 미만,891


#### [출력] [총중량] 분류

In [478]:
total_weight_df.to_excel('analysis/[G4][총중량]분류_통계.xlsx', encoding='utf-8')

#### 총중량 code 화

In [199]:
weight_code_list = []
for i, one in tqdm(enumerate(gas_df['총중량'])):
    if one < 250:
        weight_code_list.append('0025')
    elif one >= 250 and one < 500:
        weight_code_list.append('0050')
    elif one >= 500 and one < 750:
        weight_code_list.append('0075')
    elif one >= 750 and one < 1000:
        weight_code_list.append('0100')
    elif one >= 1000 and one < 1250:
        weight_code_list.append('0125')
    elif one >= 1250 and one < 1500:
        weight_code_list.append('0150')
    elif one >= 1500 and one < 1750:
        weight_code_list.append('0175')
    elif one >= 1750 and one < 2000:
        weight_code_list.append('0200')
    elif one >= 2000 and one < 2250:
        weight_code_list.append('0225')
    elif one >= 2250 and one < 2500:
        weight_code_list.append('0250')
    elif one >= 2500 and one < 2750:
        weight_code_list.append('0275')
    elif one >= 2750 and one < 3000:
        weight_code_list.append('0300')
    elif one >= 3000 and one < 3250:
        weight_code_list.append('0325')
    elif one >= 3250 and one < 3500:
        weight_code_list.append('0350')
    elif one >= 3500 and one < 3750:
        weight_code_list.append('0375')
    elif one >= 3750 and one < 4000:
        weight_code_list.append('0400')
    elif one >= 4000 and one < 4250:
        weight_code_list.append('0425')
    elif one >= 4250 and one < 4500:
        weight_code_list.append('0450')
    elif one >= 4500 and one < 4750:
        weight_code_list.append('0475')
    elif one >= 4750 and one < 5000:
        weight_code_list.append('0500')
    elif one >= 5000 and one < 5250:
        weight_code_list.append('0525')
    elif one >= 5250 and one < 5500:
        weight_code_list.append('0550')
    elif one >= 5500 and one < 5750:
        weight_code_list.append('0575')
    elif one >= 5750 and one < 6000:
        weight_code_list.append('0600')
    elif one >= 6000 and one < 6250:
        weight_code_list.append('0625')
    elif one >= 6250 and one < 6500:
        weight_code_list.append('0650')
    elif one >= 6500 and one < 6750:
        weight_code_list.append('0675')
    elif one >= 6750 and one < 7000:
        weight_code_list.append('0700')
    elif one >= 7000 and one < 7250:
        weight_code_list.append('0725')
    elif one >= 7250 and one < 7500:
        weight_code_list.append('0750')
    elif one >= 7500 and one < 7750:
        weight_code_list.append('0775')
    elif one >= 7750 and one < 8000:
        weight_code_list.append('0800')
    elif one >= 8000 and one < 8250:
        weight_code_list.append('0825')
    elif one >= 8250 and one < 8500:
        weight_code_list.append('0850')
    elif one >= 8500 and one < 8750:
        weight_code_list.append('0875')
    elif one >= 8750 and one < 9000:
        weight_code_list.append('0900')
    elif one >= 9000 and one < 9250:
        weight_code_list.append('0925')
    elif one >= 9250 and one < 9500:
        weight_code_list.append('0950')
    elif one >= 9500 and one < 9750:
        weight_code_list.append('0975')
    elif one >= 9750 and one < 10000:
        weight_code_list.append('1000')
    elif one >= 10000 and one < 10250:
        weight_code_list.append('1025')
    elif one >= 10250 and one < 10500:
        weight_code_list.append('1050')
    elif one >= 10500 and one < 10750:
        weight_code_list.append('1075')
    elif one >= 10750 and one < 11000:
        weight_code_list.append('1100')
    elif one >= 11000 and one < 11250:
        weight_code_list.append('1125')
    elif one >= 11250 and one < 11500:
        weight_code_list.append('1150')
    elif one >= 11500 and one < 11750:
        weight_code_list.append('1175')
    elif one >= 11750 and one < 12000:
        weight_code_list.append('1200')
    elif one >= 12000 and one < 12250:
        weight_code_list.append('1225')
    elif one >= 12250 and one < 12500:
        weight_code_list.append('1250')
    elif one >= 12500 and one < 12750:
        weight_code_list.append('1275')
    elif one >= 12750 and one < 13000:
        weight_code_list.append('1300')
    elif one >= 13000 and one < 13250:
        weight_code_list.append('1325')
    elif one >= 13250 and one < 13500:
        weight_code_list.append('1350')
    elif one >= 13500 and one < 13750:
        weight_code_list.append('1375')
    elif one >= 13750 and one < 14000:
        weight_code_list.append('1400')
    elif one >= 14000 and one < 14250:
        weight_code_list.append('1425')
    elif one >= 14250 and one < 14500:
        weight_code_list.append('1450')
    elif one >= 14500 and one < 14750:
        weight_code_list.append('1475')
    elif one >= 14750 and one < 15000:
        weight_code_list.append('1500')
    elif one >= 15000 and one < 15250:
        weight_code_list.append('1525')
    elif one >= 15250 and one < 15500:
        weight_code_list.append('1550')
    elif one >= 15500 and one < 15750:
        weight_code_list.append('1575')
    elif one >= 15750 and one < 16000:
        weight_code_list.append('1600')
    elif one >= 16000 and one < 16250:
        weight_code_list.append('1625')
    elif one >= 16250 and one < 16500:
        weight_code_list.append('1650')
    elif one >= 16500 and one < 16750:
        weight_code_list.append('1675')
    elif one >= 16750 and one < 17000:
        weight_code_list.append('1700')
    elif one >= 17000 and one < 17250:
        weight_code_list.append('1725')
    elif one >= 17250 and one < 17500:
        weight_code_list.append('1750')
    elif one >= 17500 and one < 17750:
        weight_code_list.append('1775')
    elif one >= 17750 and one < 18000:
        weight_code_list.append('1800')
    elif one >= 18000 and one < 18250:
        weight_code_list.append('1825')
    elif one >= 18250 and one < 18500:
        weight_code_list.append('1850')
    elif one >= 18500 and one < 18750:
        weight_code_list.append('1875')
    elif one >= 18750 and one < 19000:
        weight_code_list.append('1900')
    elif one >= 19000 and one < 19250:
        weight_code_list.append('1925')
    elif one >= 19250 and one < 19500:
        weight_code_list.append('1950')
    elif one >= 19500 and one < 19750:
        weight_code_list.append('1975')
    elif one >= 19750 and one < 20000:
        weight_code_list.append('2000')
    elif one >= 20000 and one < 20250:
        weight_code_list.append('2025')
    elif one >= 20250 and one < 20500:
        weight_code_list.append('2050')
    elif one >= 20500 and one < 20750:
        weight_code_list.append('2075')
    elif one >= 20750 and one < 21000:
        weight_code_list.append('2100')
    elif one >= 21000 and one < 21250:
        weight_code_list.append('2125')
    elif one >= 21250 and one < 21500:
        weight_code_list.append('2150')
    elif one >= 21500 and one < 21750:
        weight_code_list.append('2175')
    elif one >= 21750 and one < 22000:
        weight_code_list.append('2200')
    elif one >= 22000 and one < 22250:
        weight_code_list.append('2225')
    elif one >= 22250 and one < 22500:
        weight_code_list.append('2250')
    elif one >= 22500 and one < 22750:
        weight_code_list.append('2275')
    elif one >= 22750 and one < 23000:
        weight_code_list.append('2300')
    elif one >= 23000 and one < 23250:
        weight_code_list.append('2325')
    elif one >= 23250 and one < 23500:
        weight_code_list.append('2350')
    elif one >= 23500 and one < 23750:
        weight_code_list.append('2375')
    elif one >= 23750 and one < 24000:
        weight_code_list.append('2400')
    elif one >= 24000 and one < 24250:
        weight_code_list.append('2425')
    elif one >= 24250 and one < 24500:
        weight_code_list.append('2450')
    elif one >= 24500 and one < 24750:
        weight_code_list.append('2475')
    elif one >= 24750 and one < 25000:
        weight_code_list.append('2500')
    elif one >= 25000 and one < 25250:
        weight_code_list.append('2525')
    elif one >= 25250 and one < 25500:
        weight_code_list.append('2550')
    elif one >= 25500 and one < 25750:
        weight_code_list.append('2575')
    elif one >= 25750 and one < 26000:
        weight_code_list.append('2600')
    elif one >= 26000 and one < 26250:
        weight_code_list.append('2625')
    elif one >= 26250 and one < 26500:
        weight_code_list.append('2650')
    elif one >= 26500 and one < 26750:
        weight_code_list.append('2675')
    elif one >= 26750 and one < 27000:
        weight_code_list.append('2700')
    elif one >= 27000 and one < 27250:
        weight_code_list.append('2725')
    elif one >= 27250 and one < 27500:
        weight_code_list.append('2750')
    elif one >= 27500 and one < 27750:
        weight_code_list.append('2775')
    elif one >= 27750 and one < 28000:
        weight_code_list.append('2800')
    elif one >= 28000 and one < 28250:
        weight_code_list.append('2825')
    elif one >= 28250 and one < 28500:
        weight_code_list.append('2850')
    elif one >= 28500 and one < 28750:
        weight_code_list.append('2875')
    elif one >= 28750 and one < 29000:
        weight_code_list.append('2900')
    elif one >= 29000 and one < 29250:
        weight_code_list.append('2925')
    elif one >= 29250 and one < 29500:
        weight_code_list.append('2950')
    elif one >= 29500 and one < 29750:
        weight_code_list.append('2975')
    elif one >= 29750 and one < 30000:
        weight_code_list.append('3000')
    elif one >= 30000 and one < 30250:
        weight_code_list.append('3025')
    elif one >= 30250 and one < 30500:
        weight_code_list.append('3050')
    elif one >= 30500 and one < 30750:
        weight_code_list.append('3075')
    elif one >= 30750 and one < 31000:
        weight_code_list.append('3100')
    elif one >= 31000 and one < 31250:
        weight_code_list.append('3125')
    elif one >= 31250 and one < 31500:
        weight_code_list.append('3150')
    elif one >= 31500 and one < 31750:
        weight_code_list.append('3175')
    elif one >= 31750 and one < 32000:
        weight_code_list.append('3200')
    elif one >= 32000 and one < 32250:
        weight_code_list.append('3225')
    elif one >= 32250 and one < 32500:
        weight_code_list.append('3250')
    elif one >= 32500 and one < 32750:
        weight_code_list.append('3275')
    elif one >= 32750 and one < 33000:
        weight_code_list.append('3300')
    elif one >= 33000 and one < 33250:
        weight_code_list.append('3325')
    elif one >= 33250 and one < 33500:
        weight_code_list.append('3350')
    elif one >= 33500 and one < 33750:
        weight_code_list.append('3375')
    elif one >= 33750 and one < 34000:
        weight_code_list.append('3400')
    elif one >= 34000 and one < 34250:
        weight_code_list.append('3425')
    elif one >= 34250 and one < 34500:
        weight_code_list.append('3450')
    elif one >= 34500 and one < 34750:
        weight_code_list.append('3475')
    elif one >= 34750 and one < 35000:
        weight_code_list.append('3500')
    elif one >= 35000 and one < 35250:
        weight_code_list.append('3525')
    elif one >= 35250 and one < 35500:
        weight_code_list.append('3550')
    elif one >= 35500 and one < 35750:
        weight_code_list.append('3575')
    elif one >= 35750 and one < 36000:
        weight_code_list.append('3600')
    elif one >= 36000 and one < 36250:
        weight_code_list.append('3625')
    elif one >= 36250 and one < 36500:
        weight_code_list.append('3650')
    elif one >= 36500 and one < 36750:
        weight_code_list.append('3675')
    elif one >= 36750 and one < 37000:
        weight_code_list.append('3700')
    elif one >= 37000 and one < 37250:
        weight_code_list.append('3725')
    elif one >= 37250 and one < 37500:
        weight_code_list.append('3750')
    elif one >= 37500 and one < 37750:
        weight_code_list.append('3775')
    elif one >= 37750 and one < 38000:
        weight_code_list.append('3800')
    elif one >= 38000 and one < 38250:
        weight_code_list.append('3825')
    elif one >= 38250 and one < 38500:
        weight_code_list.append('3850')
    elif one >= 38500 and one < 38750:
        weight_code_list.append('3875')
    elif one >= 38750 and one < 39000:
        weight_code_list.append('3900')
    elif one >= 39000 and one < 39250:
        weight_code_list.append('3925')
    elif one >= 39250 and one < 39500:
        weight_code_list.append('3950')
    elif one >= 39500 and one < 39750:
        weight_code_list.append('3975')
    elif one >= 39750 and one < 40000:
        weight_code_list.append('4000')
    elif one >= 40000 and one < 40250:
        weight_code_list.append('4025')
    elif one >= 40250 and one < 40500:
        weight_code_list.append('4050')
    elif one >= 40500 and one < 40750:
        weight_code_list.append('4075')
    elif one >= 40750 and one < 41000:
        weight_code_list.append('4100')
    elif one >= 41000 and one < 41250:
        weight_code_list.append('4125')
    elif one >= 41250 and one < 41500:
        weight_code_list.append('4050')
    elif one >= 41500 and one < 41750:
        weight_code_list.append('4175')
    elif one >= 41750 and one < 42000:
        weight_code_list.append('4200')
    elif one >= 42000 and one < 42250:
        weight_code_list.append('4225')
    elif one >= 42250 and one < 42500:
        weight_code_list.append('4250')
    elif one >= 42500 and one < 42750:
        weight_code_list.append('4275')
    elif one >= 42750 and one < 43000:
        weight_code_list.append('4300')
    elif one >= 43000 and one < 43250:
        weight_code_list.append('4325')
    elif one >= 43250 and one < 43500:
        weight_code_list.append('4350')
    elif one >= 43500 and one < 43750:
        weight_code_list.append('4375')
    elif one >= 43750 and one < 44000:
        weight_code_list.append('4400')
    elif one >= 44000 and one < 44250:
        weight_code_list.append('4425')
    elif one >= 44250 and one < 44500:
        weight_code_list.append('4450')
    elif one >= 44500 and one < 44750:
        weight_code_list.append('4475')
    elif one >= 44750 and one < 45000:
        weight_code_list.append('4500')
    elif one >= 45000 and one < 45250:
        weight_code_list.append('4525')
    elif one >= 45250 and one < 45500:
        weight_code_list.append('4550')
    elif one >= 45500 and one < 45750:
        weight_code_list.append('4575')
    elif one >= 45750 and one < 46000:
        weight_code_list.append('4600')
    elif one >= 46000 and one < 46250:
        weight_code_list.append('4625')
    elif one >= 46250 and one < 46500:
        weight_code_list.append('4650')
    elif one >= 46500 and one < 46750:
        weight_code_list.append('4675')
    elif one >= 46750 and one < 47000:
        weight_code_list.append('4700')
    elif one >= 47000 and one < 47250:
        weight_code_list.append('4725')
    elif one >= 47250 and one < 47500:
        weight_code_list.append('4750')
    elif one >= 47500 and one < 47750:
        weight_code_list.append('4775')
    elif one >= 47750 and one < 48000:
        weight_code_list.append('4800')
    elif one >= 48000 and one < 48250:
        weight_code_list.append('4825')
    elif one >= 48250 and one < 48500:
        weight_code_list.append('4850')
    elif one >= 48500 and one < 48750:
        weight_code_list.append('4875')
    elif one >= 48750 and one < 49000:
        weight_code_list.append('4900')
    elif one >= 49000 and one < 49250:
        weight_code_list.append('4925')
    elif one >= 49250 and one < 49500:
        weight_code_list.append('4950')
    elif one >= 49500 and one < 49750:
        weight_code_list.append('4975')
    elif one >= 49750 and one < 50000:
        weight_code_list.append('5000')
    elif one >= 50000:
        weight_code_list.append('9999')

1528557it [00:02, 512430.18it/s]


In [200]:
len(weight_code_list)

1528557

In [201]:
gas_df.shape

(1528557, 46)

In [202]:
gas_df['총중량_코드'] = weight_code_list

In [203]:
n = 250
f'{n:0>4}'

'0250'

In [204]:
# 오류 : 9000개만 처리됨
# weight_code_list = []
# n1 = 250
# for i, one in tqdm(enumerate(df5['총중량'])):
#     for i in range(200):
#         n1 += 250 * i
#         n2 = n1 + 250
#         s = f'{n2:0>4}'
#         if one < 250:
#             weight_code_list.append('0025')
#         elif n1 <= one < n2:
#             weight_code_list.append(s)
#         elif one >= 50000:
#             weight_code_list.append('9999')
# len(weight_code_list)

### 배기량으로 분류 현황

In [205]:
gas_df['배기량'].value_counts()

2497.0    412725
1991.0    245077
2902.0    139727
2959.0     75052
1995.0     68704
           ...  
2687.0         1
5407.0         1
2605.0         1
1747.0         1
2286.0         1
Name: 배기량, Length: 627, dtype: int64

In [206]:
gas_df['배기량'].isnull().sum()

4

In [207]:
gas_df['배기량'] = gas_df['배기량'].fillna(-1)

In [208]:
gas_volume_list = []
for i, one in tqdm(enumerate(gas_df['배기량'])):
    if one == -1:
        gas_volume_list.append('미기입')
    elif one < 800:
        gas_volume_list.append('0.8L 미만')
    elif one >= 800 and one < 1000:
        gas_volume_list.append('0.8L 이상 1.0L 미만')
    elif one >= 1000 and one < 1600:
        gas_volume_list.append('1.0L 이상 1.6L 미만')
    elif one >= 1600 and one < 2000:
        gas_volume_list.append('1.6L 이상 2.0L 미만')
    elif one >= 2000 and one < 3000:
        gas_volume_list.append('2.0L 이상 3.0L 미만')
    elif one >= 3000 and one < 4000:
        gas_volume_list.append('3.0L 이상 4.0L 미만')
    elif one >= 4000 and one < 5000:
        gas_volume_list.append('4.0L 이상 5.0L 미만')
    elif one >= 5000 and one < 6000:
        gas_volume_list.append('5.0L 이상 6.0L 미만')
    elif one >= 6000 and one < 8000:
        gas_volume_list.append('6.0L 이상 8.0L 미만')
    elif one >= 8000 and one < 10000:
        gas_volume_list.append('8.0L 이상 10.0L 미만')
    elif one >= 10000:
        gas_volume_list.append('10.0L 이상')

1528557it [00:01, 953040.31it/s] 


In [209]:
len(gas_volume_list)

1528557

In [210]:
gas_df['배기량_분류'] = gas_volume_list

In [211]:
total_gas_volume_df = gas_df.groupby(['배기량_분류'])['차대번호'].count().reset_index()
total_gas_volume_df

,배기량_분류,차대번호
0,0.8L 미만,28673
1,0.8L 이상 1.0L 미만,171
2,1.0L 이상 1.6L 미만,165909
3,1.6L 이상 2.0L 미만,510876
4,10.0L 이상,16847
5,2.0L 이상 3.0L 미만,755436
6,3.0L 이상 4.0L 미만,31434
7,4.0L 이상 5.0L 미만,1340
8,5.0L 이상 6.0L 미만,15493
9,6.0L 이상 8.0L 미만,711


#### [출력] [배기량]분류

In [492]:
total_gas_volume_df.to_excel('analysis/[G4][배기량]분류_통계.xlsx', encoding='utf-8')

#### 배기량 code화

In [212]:
gas_volume_code_list = []
for i, one in tqdm(enumerate(gas_df['배기량'])):
    if one < 800:
        gas_volume_code_list.append('008')
    elif one >= 800 and one < 1000:
        gas_volume_code_list.append('010')
    elif one >= 1000 and one < 1600:
        gas_volume_code_list.append('016')
    elif one >= 1600 and one < 2000:
        gas_volume_code_list.append('020')
    elif one >= 2000 and one < 2500:
        gas_volume_code_list.append('025')
    elif one >= 2500 and one < 3000:
        gas_volume_code_list.append('030')
    elif one >= 3000 and one < 3500:
        gas_volume_code_list.append('035')
    elif one >= 3500 and one < 4000:
        gas_volume_code_list.append('040')
    elif one >= 4000 and one < 4500:
        gas_volume_code_list.append('045')
    elif one >= 4500 and one < 5000:
        gas_volume_code_list.append('050')
    elif one >= 5000 and one < 5500:
        gas_volume_code_list.append('055')
    elif one >= 5500 and one < 6000:
        gas_volume_code_list.append('060')
    elif one >= 6000 and one < 6500:
        gas_volume_code_list.append('065')
    elif one >= 6500 and one < 7000:
        gas_volume_code_list.append('070')
    elif one >= 7000 and one < 7500:
        gas_volume_code_list.append('075')
    elif one >= 7500 and one < 8000:
        gas_volume_code_list.append('080')
    elif one >= 8000 and one < 8500:
        gas_volume_code_list.append('085')
    elif one >= 8500 and one < 9000:
        gas_volume_code_list.append('090')
    elif one >= 9000 and one < 9500:
        gas_volume_code_list.append('095')
    elif one >= 9500 and one < 10000:
        gas_volume_code_list.append('100')
    elif one >= 10000 and one < 10500:
        gas_volume_code_list.append('105')
    elif one >= 10500 and one < 11000:
        gas_volume_code_list.append('110')
    elif one >= 11000 and one < 11500:
        gas_volume_code_list.append('115')
    elif one >= 11500 and one < 12000:
        gas_volume_code_list.append('120')
    elif one >= 12000 and one < 12500:
        gas_volume_code_list.append('125')
    elif one >= 12500 and one < 13000:
        gas_volume_code_list.append('130')
    elif one >= 13000 and one < 13500:
        gas_volume_code_list.append('135')
    elif one >= 13500 and one < 14000:
        gas_volume_code_list.append('140')
    elif one >= 14000 and one < 14500:
        gas_volume_code_list.append('145')
    elif one >= 14500 and one < 15000:
        gas_volume_code_list.append('150')
    elif one >= 15000 and one < 15500:
        gas_volume_code_list.append('155')
    elif one >= 15500 and one < 16000:
        gas_volume_code_list.append('160')
    elif one >= 16000 and one < 16500:
        gas_volume_code_list.append('165')
    elif one >= 16500 and one < 17000:
        gas_volume_code_list.append('170')
    elif one >= 17000 and one < 18000:
        gas_volume_code_list.append('180')
    elif one >= 18000 and one < 18500:
        gas_volume_code_list.append('185')
    elif one >= 18500 and one < 19000:
        gas_volume_code_list.append('190')
    elif one >= 19000 and one < 19500:
        gas_volume_code_list.append('195')
    elif one >= 19500 and one < 20000:
        gas_volume_code_list.append('200')
    elif one >= 20000 and one < 20500:
        gas_volume_code_list.append('205')
    elif one >= 20500 and one < 21000:
        gas_volume_code_list.append('210')
    elif one >= 21000 and one < 21500:
        gas_volume_code_list.append('215')
    elif one >= 21500 and one < 22000:
        gas_volume_code_list.append('220')
    elif one >= 22000 and one < 22500:
        gas_volume_code_list.append('225')
    elif one >= 22500 and one < 23000:
        gas_volume_code_list.append('230')
    elif one >= 23000 and one < 23500:
        gas_volume_code_list.append('235')
    elif one >= 23500 and one < 24000:
        gas_volume_code_list.append('240')
    elif one >= 24000 and one < 24500:
        gas_volume_code_list.append('245')
    elif one >= 24500 and one < 25000:
        gas_volume_code_list.append('250')
    elif one >= 25000 and one < 25500:
        gas_volume_code_list.append('255')
    elif one >= 25500 and one < 26000:
        gas_volume_code_list.append('260')
    elif one >= 26000 and one < 26500:
        gas_volume_code_list.append('265')
    elif one >= 26500 and one < 27000:
        gas_volume_code_list.append('270')
    elif one >= 27000 and one < 27500:
        gas_volume_code_list.append('275')
    elif one >= 27500 and one < 28000:
        gas_volume_code_list.append('280')
    elif one >= 28000 and one < 28500:
        gas_volume_code_list.append('285')
    elif one >= 28500 and one < 29000:
        gas_volume_code_list.append('290')
    elif one >= 29000 and one < 29500:
        gas_volume_code_list.append('295')
    elif one >= 29500 and one < 30000:
        gas_volume_code_list.append('300')

1528557it [00:01, 948612.65it/s] 


In [213]:
len(gas_volume_code_list)

1528557

In [214]:
gas_df['배기량_코드'] = gas_volume_code_list

In [215]:
gas_df['배출가스인증번호'] = gas_df['배출가스인증번호'].str.replace(' ', '')

### 배인 코드화

In [216]:
gas_df['배출가스인증번호'].unique()

array([nan, '7MY-HD-14-68', '7MY-HD-24-31', '7MY-HD-24-09',
       '7MY-HD-24-28', '7MY-KM-14-65', '6MY-SS-0M-24', '7MY-HD-13-58',
       '7MY-KM-14-63', '7MY-HD-14-47', '7MY-HD-14-46', '7MY-DB-24-42',
       '8MY-KM-13-19', '7MY-HD-14-57', '7MY-KM-14-64', '9MY-KM-14-41',
       '7MY-HD-24-30', 'FMY-VTK-14-1', '5MY-HD-24-59', '6MY-HD-23-55',
       '6MY-KM-14-61', '7MY-DW-13-07', '7MY-HD-24-29', '5MY-AD-14-18',
       'FMY-VTK-14-4', '6MY-HD-24-12', '7MY-SY-14-48', '6MY-HD-24-100',
       '7MY-HD-14-45', 'FMY-VTK-14-2', 'FMY-VTK-14-3', 'RMY-HD-20',
       'VMY-HD-30', 'WMY-HD-32', '5MY-AD-14-12', '9MY-HD-14-21',
       '5MY-AD-14-08', '7MY-SY-14-60', '7MY-SM-14-36', '6MY-HD-14-97',
       '7MY-HD-24-08', '6MY-KM-24-81', 'CMY-VTK-14-01', 'CMY-VTK-14-02',
       'KMY-HD-03', '6MY-HD-14-64', 'MMY-HD-11', '5MY-SY-23-58',
       '6MY-HD-14-63', 'SMY-KM-32', '6MY-KM-24-06', '7MY-KM-14-52',
       '8MY-HT-0M-24', '6MY-HD-24-44', '7MY-SY-14-34', '5MY-HB-24-02',
       'BMY-PPE-14-29', '6MY-DW-

In [217]:
temp = gas_df['배출가스인증번호'].value_counts().reset_index().sort_values('index')['index'].reset_index(drop=True)
temp

0         3MY-FK-08
1         4MY-AD-20
2         4MY-DC-33
3         4MY-PA-06
4      5MY-AD-13-06
           ...     
295       XMY-HD-47
296       XMY-HD-48
297       XMY-HD-55
298       XMY-KM-47
299       YKM-KM-12
Name: index, Length: 300, dtype: object

In [218]:
temp.to_frame()

,index
0,3MY-FK-08
1,4MY-AD-20
2,4MY-DC-33
3,4MY-PA-06
4,5MY-AD-13-06
...,...
295,XMY-HD-47
296,XMY-HD-48
297,XMY-HD-55
298,XMY-KM-47


In [219]:
gas_idx = []
for i in range(1, 301):
    s = f'EN{i:0>3}'
    gas_idx.append(s)
len(gas_idx)

300

In [220]:
gas_idx_df = pd.DataFrame({'배출가스인증번호_코드':gas_idx})
gas_idx_df

,배출가스인증번호_코드
0,EN001
1,EN002
2,EN003
3,EN004
4,EN005
...,...
295,EN296
296,EN297
297,EN298
298,EN299


In [221]:
temp2 = pd.concat([temp, gas_idx_df], axis=1, ignore_index=True)
temp2

,0,1
0,3MY-FK-08,EN001
1,4MY-AD-20,EN002
2,4MY-DC-33,EN003
3,4MY-PA-06,EN004
4,5MY-AD-13-06,EN005
...,...,...
295,XMY-HD-47,EN296
296,XMY-HD-48,EN297
297,XMY-HD-55,EN298
298,XMY-KM-47,EN299


In [222]:
temp2.columns = ['배출가스인증번호', '배출가스인증번호_코드']
temp2

,배출가스인증번호,배출가스인증번호_코드
0,3MY-FK-08,EN001
1,4MY-AD-20,EN002
2,4MY-DC-33,EN003
3,4MY-PA-06,EN004
4,5MY-AD-13-06,EN005
...,...,...
295,XMY-HD-47,EN296
296,XMY-HD-48,EN297
297,XMY-HD-55,EN298
298,XMY-KM-47,EN299


In [223]:
temp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   배출가스인증번호     300 non-null    object
 1   배출가스인증번호_코드  300 non-null    object
dtypes: object(2)
memory usage: 4.8+ KB


#### [출력] 배인번호 코드화 리스트

In [666]:
temp2[['배출가스인증번호_코드', '배출가스인증번호']].to_excel('analysis/[G4][배출가스인증번호]코드_리스트.xlsx')

In [224]:
gas_df.shape

(1528557, 49)

In [225]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드'],
      dtype='object')

In [226]:
gas_df = gas_df.merge(temp2, on='배출가스인증번호', how='left')
gas_df.shape

(1528557, 50)

In [227]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드'],
      dtype='object')

## 차대번호로 분류

### wmi 코드로 분류

In [228]:
wmi = pd.read_excel('data/20190730_WMI CODE.xlsx', sheet_name='WMI CODE')
wmi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   WMI            690 non-null    object 
 1   MF CODE        690 non-null    object 
 2   Identifier     690 non-null    object 
 3   Manufacturer   690 non-null    object 
 4   Country        688 non-null    object 
 5   Vehicle Types  249 non-null    object 
 6   Description    54 non-null     object 
 7   참고             0 non-null      float64
dtypes: float64(1), object(7)
memory usage: 43.2+ KB


In [229]:
df6 = gas_df.copy()

In [230]:
df6['차대번호'].head()

0    KMCLK18KP8C035195
1    KMFLA18KP8C031274
2    KLACA26RD8B226753
3    KMCGK17HP8C086937
4    KMCYSS7JP7U084697
Name: 차대번호, dtype: object

In [231]:
df6['wmi'] = df6['차대번호'].str[:3]

In [232]:
df6['wmi'].head()

0    KMC
1    KMF
2    KLA
3    KMC
4    KMC
Name: wmi, dtype: object

In [233]:
df6 = df6.merge(wmi, left_on='wmi', right_on='WMI', how='left')
df6[['차대번호', 'wmi', 'WMI']].head()

,차대번호,wmi,WMI
0,KMCLK18KP8C035195,KMC,NaN
1,KMFLA18KP8C031274,KMF,KMF
2,KLACA26RD8B226753,KLA,KLA
3,KMCGK17HP8C086937,KMC,NaN
4,KMCYSS7JP7U084697,KMC,NaN


In [234]:
df6.shape

(1528557, 59)

In [235]:
df6['WMI'].isnull().sum()

75232

In [236]:
df6['Manufacturer'].isnull().sum()

75232

In [237]:
wmi.columns

Index(['WMI', 'MF CODE', 'Identifier', 'Manufacturer', 'Country',
       'Vehicle Types', 'Description', '참고'],
      dtype='object')

In [238]:
df6[['WMI', 'MF CODE', 'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description', '참고']].head()

,WMI,MF CODE,Identifier,Manufacturer,Country,Vehicle Types,Description,참고
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KMF,HYUN,Hyundai,Hyundai,Korea,Commercail vehicle,NaN,NaN
2,KLA,GM D,GM Daewoo,Daewoo General Motors South Korea,Korea,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [239]:
df6['Identifier'].isnull().sum()

75232

In [240]:
df6['Manufacturer'] = df6['Manufacturer'].fillna(-1)
df6['Identifier'] = df6['Identifier'].fillna(-1)

In [241]:
manu_df = df6.groupby('Identifier')['차대번호'].count().reset_index()
manu_df

,Identifier,차대번호
0,-1,75232
1,Alfa Romeo,1
2,Audi,2107
3,Austin-Rover,83
4,BMW,2209
...,...,...
71,Toyota,140
72,Toyota,35
73,Volkswagen,5924
74,Volvo,6667


In [242]:
manu_df = manu_df.sort_values('차대번호', ascending=False).reset_index(drop=True)
manu_df

,Identifier,차대번호
0,Hyundai,887146
1,Kia,358326
2,GM Daewoo,116233
3,-1,75232
4,SsangYong,31200
...,...,...
71,Neoplan,1
72,Matra,1
73,Daimler,1
74,Kenworth,1


#### [출력] Identifier별 차대번호 통계

In [524]:
# 제작사(Manufacturer)별 차대번호 통계
manu_df.to_excel('analysis/[G4][차대번호]제작사별_통계.xlsx', encoding='utf-8')

## 차대번호

In [243]:
# - 오류
#     - 1\. null 값 : 0
#     - 2\. 정상(17자리, 영문 포함) : 1525955
#     - 3\. 17자리, 영문X : 1
#     - 4\. 17자리X, 영문 : 57
#     - 5\. 17자리X, 영문X : 2573

In [244]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1528557 entries, 0 to 1528556
Data columns (total 59 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   차량관리번호         1528557 non-null  object 
 1   차량등록번호         1528557 non-null  object 
 2   본거지법정동코드       1528557 non-null  int64  
 3   소유자구분          1528557 non-null  object 
 4   차종_등록정보        1528555 non-null  object 
 5   용도             1528557 non-null  object 
 6   차대번호           1528557 non-null  object 
 7   최초등록일자         1528557 non-null  int64  
 8   차량연식           1528557 non-null  int64  
 9   차량제작일자         1358915 non-null  float64
 10  검사유효일          1528545 non-null  float64
 11  제원관리번호         1528557 non-null  object 
 12  배출가스인증번호       1405554 non-null  object 
 13  배출가스등급         1528557 non-null  int64  
 14  저공해조치종류        195 non-null      object 
 15  저공해조치구조변경YN    1528557 non-null  object 
 16  취득일자           1435498 non-null  object 
 17  취득금액    

In [245]:
df6['차대번호'].isnull().sum()

0

In [246]:
len('KMFZCY7JBCU784032')

17

### 차대번호 정상 확인

In [247]:
# 17자리, 영문 -> 정상 : 1525955
count = 0
p61 = re.compile('[a-zA-Z]+')
idx61 = []
for i, one in tqdm(enumerate(df6.loc[:, '차대번호'].to_numpy())):
    if len(str(one)) == 17 and p61.search(str(one)):
        idx61.append(i)
        count += 1
count

1528557it [00:02, 659461.85it/s]


1525926

In [248]:
# 정상에서 "I, O, Q" 포함한 값 : 1018
temp61 = df6.loc[idx61, '차대번호'].to_numpy()
idx61_1 = []
for i, one in tqdm(enumerate(temp61)):
    if ('I' in one) or ('O' in one) or ('Q' in one):
        idx61_1.append(idx61[i])
len(idx61_1)

1525926it [00:00, 2102910.48it/s]


1018

In [249]:
df6.iloc[idx61_1, 6][:5]

1142      WPOZZZ96ZLS402952
117803    WPOZZZ96ZKS400603
330303    YV2ABSOC4AA695671
346135    YV2ABSOC1AA694168
346256    YV2ABSOC2AA693157
Name: 차대번호, dtype: object

In [250]:
# 17자리, 영문X  => 숫자만 : 1
count = 0
p62 = re.compile('[a-zA-Z]+')
idx62 = []
for i, one in tqdm(enumerate(df6.loc[:, '차대번호'].to_numpy())):
    n = 0
    for j in str(one):
        if p62.search(j):
            n += 1
    if len(str(one)) == 17 and n == 0:
        idx62.append(i)
        count += 1
count

1528557it [00:13, 114383.65it/s]


1

In [251]:
df6.loc[:, '차대번호'].reindex(idx62)

1471703    01901321371111111
Name: 차대번호, dtype: object

In [252]:
# 17자리 X, 영문 : 57
count = 0
p63 = re.compile('[a-zA-Z]+')
idx63 = []
for i, one in tqdm(enumerate(df6.loc[:, '차대번호'].to_numpy())):
    if len(str(one)) != 17 and p63.search(one):
        idx63.append(i)
        count += 1
count

1528557it [00:00, 1713221.91it/s]


57

In [253]:
df6.loc[:, '차대번호'].reindex(idx63)[:5]

352051    KMFWBX7ABU267970
362014    KMFZCZ7JAB640917
461606    KNCSJX4AAK452185
641800    KMFZC7JP9U519465
731532    KMFZC7JP9U481571
Name: 차대번호, dtype: object

In [254]:
# 17자리 X, 영문X : 2573
count = 0
p64 = re.compile('[a-zA-Z]+')
idx64 = []
for i, one in tqdm(enumerate(df6.loc[:, '차대번호'].to_numpy())):
    n = 0
    for j in one:
        if p64.search(j):
            n += 1
    if len(str(one)) != 17 and n == 0:
        idx64.append(i)
        count += 1
count

1528557it [00:13, 115942.31it/s]


2573

In [255]:
df6.loc[:, '차대번호'].reindex(idx64)[:5]

1144821    289711309283
1161719    119811229792
1170369    119811328945
1170707    119411329496
1170967    119311230058
Name: 차대번호, dtype: object

#### 차대번호 길이, IOQ포함 여부 통계

In [256]:
# 차대번호 길이 종류 확인
len_vin = []
for i, one in tqdm(enumerate(df6.loc[:, '차대번호'].to_numpy())):
    len_vin.append(len(str(one)))
set(len_vin)

1528557it [00:01, 1474395.69it/s]


{10, 11, 12, 13, 14, 15, 16, 17}

In [257]:
df6['VIN_length'] = [ len(one) for one in df6['차대번호'] ]

In [258]:
df6['VIN_length'].unique()

array([17, 16, 12, 15, 11, 13, 14, 10], dtype=int64)

In [259]:
# IOQ 포함 여부
IOQ = []
for one in df6['차대번호']:
    if ('I' in one) or ('O' in one) or ('Q' in one):
        IOQ.append('Y')
    else:
        IOQ.append('N')
df6['IOQ'] = IOQ

In [260]:
df6['IOQ'].unique()

array(['N', 'Y'], dtype=object)

In [261]:
# 알파벳 포함
included_alpha = []
count = 0
p65 = re.compile('[a-zA-Z]+')
for i, one in tqdm(enumerate(df6.loc[:, '차대번호'].to_numpy())):
    n = 0
    for j in one:
        if p65.search(j):
            n += 1
    if n == 0:
        included_alpha.append('N')
        count += 1
    else:
        included_alpha.append('Y')
count

1528557it [00:13, 117571.75it/s]


2574

In [262]:
df6['alpha'] = included_alpha

In [263]:
df6.groupby(['VIN_length', 'alpha', 'IOQ'])['차대번호'].count()

VIN_length  alpha  IOQ
10          N      N            1
            Y      N            3
11          N      N          209
            Y      N            7
12          N      N         2132
            Y      N            2
13          N      N          154
            Y      N            3
14          N      N           22
15          N      N           38
            Y      N            2
                   Y            1
16          N      N           17
            Y      N           39
17          N      N            1
            Y      N      1524908
                   Y         1018
Name: 차대번호, dtype: int64

In [264]:
df6.groupby(['VIN_length', 'IOQ'])['차대번호'].count()

VIN_length  IOQ
10          N            4
11          N          216
12          N         2134
13          N          157
14          N           22
15          N           40
            Y            1
16          N           56
17          N      1524909
            Y         1018
Name: 차대번호, dtype: int64

In [265]:
V_I_dict = df6.groupby(['VIN_length', 'IOQ'])['차대번호'].groups

In [266]:
V_I_dict.keys()

dict_keys([(10, 'N'), (11, 'N'), (12, 'N'), (13, 'N'), (14, 'N'), (15, 'N'), (15, 'Y'), (16, 'N'), (17, 'N'), (17, 'Y')])

In [267]:
g = df6.groupby(['VIN_length', 'IOQ'], as_index=False)['차대번호'].count()
g

,VIN_length,IOQ,차대번호
0,10,N,4
1,11,N,216
2,12,N,2134
3,13,N,157
4,14,N,22
5,15,N,40
6,15,Y,1
7,16,N,56
8,17,N,1524909
9,17,Y,1018


In [268]:
V_I_df = g.sort_values(['VIN_length', 'IOQ'], ascending=[True, True])
V_I_df

,VIN_length,IOQ,차대번호
0,10,N,4
1,11,N,216
2,12,N,2134
3,13,N,157
4,14,N,22
5,15,N,40
6,15,Y,1
7,16,N,56
8,17,N,1524909
9,17,Y,1018


##### 차대번호 길이, IOQ포함 여부 별 예시

In [269]:
for v in range(10, 18):
    for i in ['N', 'Y']:
        try:
            idx = V_I_dict[(v, i)]
            print(f'[VIN_length : {v}, "IOQ"포함 : {i}]')
            print('예시)')
            print(df6.loc[idx[:3], '차대번호'])
            print()
        except:
            pass

[VIN_length : 10, "IOQ"포함 : N]
예시)
1184741    FD3S112983
1191288    FD3S303919
1197803    EU13315046
Name: 차대번호, dtype: object

[VIN_length : 11, "IOQ"포함 : N]
예시)
1171917    H36A0602064
1179716    RPS13330414
1186002    DE2A0100210
Name: 차대번호, dtype: object

[VIN_length : 12, "IOQ"포함 : N]
예시)
1144821    289711309283
1161719    119811229792
1170369    119811328945
Name: 차대번호, dtype: object

[VIN_length : 13, "IOQ"포함 : N]
예시)
1176816    4197111236649
1179156    289711308190하
1180037    2893113157323
Name: 차대번호, dtype: object

[VIN_length : 14, "IOQ"포함 : N]
예시)
1182843    27911130156505
1183740    27931130152305
1349628    21891130315401
Name: 차대번호, dtype: object

[VIN_length : 15, "IOQ"포함 : N]
예시)
1171406    11-97-113-29992
1171678    11-97-113-28686
1171805    41-96-113-35598
Name: 차대번호, dtype: object

[VIN_length : 15, "IOQ"포함 : Y]
예시)
1528151    12851300018EORN
Name: 차대번호, dtype: object

[VIN_length : 16, "IOQ"포함 : N]
예시)
352051    KMFWBX7ABU267970
362014    KMFZCZ7JAB640917
461606   

In [270]:
df6.groupby(['VIN_length', '연료'])['차대번호'].count().unstack().T

VIN_length,10,11,12,13,14,15,16,17
연료,,,,,,,,
CNG(압축천연가스),NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
LPG(액화석유가스),NaN,NaN,15.0,NaN,NaN,NaN,2.0,26401.0
경유,NaN,NaN,NaN,NaN,NaN,NaN,10.0,1167595.0
기타연료,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
알코올,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
휘발유,4.0,216.0,2119.0,157.0,22.0,41.0,44.0,331928.0


##### [출력] vin 길이별, 연료별 차대번호 통계

In [561]:
# vin 길이별, 연료별 차대번호 개수 통계
df6.groupby(['VIN_length', '연료'])['차대번호'].count().unstack().T.to_excel('analysis/[G4][차대번호]길이_통계.xlsx')

##### [출력] vin IOQ 포함 여부별, 연료별 차대번호 통계

In [562]:
df6.groupby(['IOQ', '연료'])['차대번호'].count().unstack().T

IOQ,N,Y
연료,,
CNG(압축천연가스),1.0,NaN
LPG(액화석유가스),26414.0,4.0
경유,1167405.0,200.0
기타연료,1.0,NaN
알코올,1.0,NaN
휘발유,333716.0,815.0


In [563]:
# vin IOQ 포함별, 연료별 차대번호 개수 통계
df6.groupby(['IOQ', '연료'])['차대번호'].count().unstack().T.to_excel('analysis/[G4][차대번호]IOQ포함여부_통계.xlsx')

#### 차대번호 10번째 자리 문자로 연식 확인

In [564]:
df6['차량연식'].unique()

array([2008, 2007, 2006, 2012, 2011, 2009, 2001, 2010, 2019, 2020, 1992,
       1994, 1996, 2018, 1991, 1993, 1989, 1990, 1995, 1997, 2013, 1998,
       1999, 1988, 2017, 2005, 2016, 2002, 2004, 2003, 2015, 1987, 2014,
       1986, 2000, 1981, 1978, 1973, 1984, 1901, 1985, 1980, 1979, 1983,
       1968], dtype=int64)

In [565]:
df6.groupby(['차량연식', '연료'])['차대번호'].count().unstack()

연료,CNG(압축천연가스),LPG(액화석유가스),경유,기타연료,알코올,휘발유
차량연식,,,,,,
1901,NaN,NaN,NaN,NaN,NaN,1.0
1968,NaN,NaN,NaN,NaN,NaN,1.0
1973,NaN,NaN,NaN,NaN,NaN,1.0
1978,NaN,NaN,NaN,NaN,NaN,1.0
1979,NaN,NaN,NaN,NaN,NaN,3.0
1980,NaN,NaN,NaN,NaN,NaN,1.0
1981,NaN,NaN,NaN,NaN,NaN,1.0
1983,NaN,NaN,NaN,NaN,NaN,3.0
1984,NaN,NaN,NaN,NaN,NaN,3.0


##### [출력] 차량연식별, 연료별 차대번호 통계

In [566]:
# 차량연식별, 연료별 차대번호 개수 통계
df6.groupby(['차량연식', '연료'])['차대번호'].count().unstack().to_excel('analysis/[G4][차량연식]통계.xlsx')

In [567]:
df6['차대번호_10자리'] = df6['차대번호'].str[9]
df6['차대번호_10자리'].unique()

array(['8', '7', '6', 'C', 'B', '9', '1', 'A', 'K', 'L', 'N', 'R', 'T',
       'J', 'M', 'P', 'S', 'V', 'D', 'W', 'X', 'H', '5', 'G', '2', '4',
       '3', 'F', 'E', 'Y', 'U', '0', 'O', '-'], dtype=object)

In [568]:
vin_10_dict = {'A':1980, 'B':1981, 'C':1982, 'D':1983, 'E':1984, 'F':1985, 'G':1986, 'H':1987, 'J':1988, 'K':1989, 'L':1990, 'M':1991, 'N':1992, 'P':1993, 'R':1994, 'S':1995, 'T':1996, 'V':1997, 'W':1998, 'X':1999, 'Y':2000, '1':2001, '2':2002, '3':2003, '4':2004, '5':2005, '6':2006, '7':2007, '8':2008, '9':2009}

In [569]:
df6['차대번호_10자리_기준연식'] = df6['차대번호_10자리'].replace(vin_10_dict)

In [570]:
df6['차대번호_10자리_기준연식'].unique()

array([2008, 2007, 2006, 1982, 1981, 2009, 2001, 1980, 1989, 1990, 1992,
       1994, 1996, 1988, 1991, 1993, 1995, 1997, 1983, 1998, 1999, 1987,
       2005, 1986, 2002, 2004, 2003, 1985, 1984, 2000, 'U', '0', 'O', '-'],
      dtype=object)

In [571]:
df6.groupby(['차대번호_10자리_기준연식', '연료'])['차대번호'].count().unstack()

연료,CNG(압축천연가스),LPG(액화석유가스),경유,기타연료,알코올,휘발유
차대번호_10자리_기준연식,,,,,,
1980,NaN,NaN,252705.0,NaN,NaN,2.0
1981,NaN,NaN,237573.0,NaN,NaN,5.0
1982,NaN,NaN,94135.0,NaN,NaN,54.0
1983,NaN,NaN,29.0,NaN,NaN,70.0
1984,NaN,NaN,13.0,NaN,NaN,1.0
1985,NaN,NaN,463.0,NaN,NaN,6.0
1986,NaN,NaN,482.0,NaN,NaN,12.0
1987,NaN,NaN,732.0,NaN,NaN,1299.0
1988,NaN,91.0,483.0,NaN,1.0,11652.0


##### [출력] vin 10자리문자기준별, 연료별 차대번호 통계

In [572]:
# vin 10자리 문자기준별, 연료별 차대번호 통계
df6.groupby(['차대번호_10자리_기준연식', '연료'])['차대번호'].count().unstack().to_excel('analysis/[G4][차량연식]차대번호10번째자리문자기준_통계.xlsx')

In [575]:
df6.shape

(1528557, 64)

In [582]:
df6.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '차대번호_10자리', '차대번호_10자리_기준연식'],
      dtype='object')

# [출력] '총중량_분류', '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE', 'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description', '참고', 'VIN_length', 'IOQ', 'alpha', '차대번호_10자리', '차대번호_10자리_기준연식'

In [ ]:
# 약 42 s
df6.to_csv('data/[G4]220819_22.06월기준_4등급.csv')

## [로드]정기/정밀 검사 데이터 로드

In [3]:
# 약 4m
per_ins = pd.read_csv('data/[정기검사]20190701_20220630.csv', index_col=0)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_6508\4283608937.py:2: DtypeWarning: Columns (36,39,42,45,48,63) have mixed types. Specify dtype option on import or set low_memory=False.
  per_ins = pd.read_csv('data/[정기검사]20190701_20220630.csv', index_col=0)


In [4]:
per_ins.shape

(16535477, 63)

In [5]:
# 약 11m
per_d_ins = pd.read_csv('data/[정밀검사]20190701_20220630.csv', index_col=0)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_6508\1915627214.py:2: DtypeWarning: Columns (13,44,45,46,49,53,54,55,59,60,61,65,66,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  per_d_ins = pd.read_csv('data/[정밀검사]20190701_20220630.csv', index_col=0)


In [6]:
per_d_ins.shape

(22397343, 75)

In [7]:
per_ins.columns

Index(['검사소명', '정기검사일자', '검사종류', '검사접수번호', '통합접수번호', '차량번호', '차대번호', '차량형식번호',
       '차명', '차종', '차량용도', '차량연식', '차량연료', '주행거리', '차량길이', '차량너비', '차량높이',
       '엔진형식', '배기량', '엔진출력', '차량중량', '총중량', '정원', '비고', '최초등록일자', '제작일자',
       '검사전 유효기간 만료일', '관능검사판정', '관능부적합항목', '검사방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '이산화탄소값', '최대출력허용치', '최대출력측정값', '산소값', '공기온도', '공기압력',
       '검사판정', '재검사기간', '검사후 갱신유효기간', '관능검사원', '배출가스검사원', '책임검사원', '검사 판정시간',
       '자료구분코드', '파일명'],
      dtype='object')

In [675]:
per_ins['공기온도'].unique()

array([nan])

In [8]:
per_d_ins.columns

Index(['정밀검사소명', '정밀검사일자', '정밀검사종류', '정밀검사접수번호', '차량번호', '차대번호', '차량형식번호',
       '차명', '차종', '차량용도', '차량연식', '차량연료', '주행거리', '차량길이', '차량너비', '차량높이',
       '엔진형식', '원동기배기량', '엔진출력', '차량중량', '총중량', '정원', '비고', '최초등록일',
       '정밀검사유효기간', '관능검사판정', '관능검사부적합내용', '정밀검사측정방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '이산화탄소값', '최대출력허용치', '최대출력측정값', '산소값', '공기온도', '공기압력',
       '정밀검사판정', '재검사기간', '정밀검사유효기간2', '관능검사등록자', '정밀검사등록자', '정밀검사소대표명',
       '기타내용1', '기타내용2', '기타내용3', '기타내용4', '기타내용5', '원본대장테이블', '정밀검사시간정보',
       '통합접수번호', '데이터생성일자', '수신파일명', '제작등록일자', '데이터구분', '삭제구분', '삭제사유1',
       '삭제사유2', '삭제사유3', '삭제사유4'],
      dtype='object')

In [679]:
per_d_ins['산소값'].unique()

array([  0.  ,   3.7 ,    nan, ...,  19.17, 159.6 ,  30.9 ])

In [680]:
per_d_ins['산소값'].value_counts()

0.00     13318535
0.10       706340
0.20       275318
14.80      228366
14.70      222326
           ...   
8.89            1
19.73           1
10.01           1
31.40           1
30.90           1
Name: 산소값, Length: 2268, dtype: int64

In [678]:
per_d_ins['공기온도'].unique()

array([  0. ,   nan,  30.8, ..., 102. , 167. , 113. ])

In [677]:
per_d_ins['공기압력'].unique()

array([ 0.000e+00,        nan,  1.019e+02, ..., -2.000e-01,  9.998e+02,
        6.000e+00])

In [681]:
per_d_ins['공기압력'].max()

6552.0

In [682]:
per_d_ins['공기압력'].min()

-1499.7

In [9]:
per_ins['검사방법'].unique()

array(['무부하검사(급가속)', '무부하검사(TSI)', '무부하검사(정지가동)'], dtype=object)

In [683]:
per_ins['무부하매연측정치6'].max()

3.0

In [684]:
per_ins['무부하매연측정치6'].min()

0.0

In [685]:
per_ins['무부하매연측정치6'].value_counts()

1.00    1584200
1.01     968540
0.99     696920
1.02     356388
1.03     197479
         ...   
2.09          1
2.47          1
0.36          1
2.82          1
2.53          1
Name: 무부하매연측정치6, Length: 241, dtype: int64

In [690]:
per_ins.shape

(16535477, 63)

In [689]:
per_ins[(per_ins['무부하매연측정치6'] > 1.2) | (per_ins['무부하매연측정치6'] < 0.8)].shape

(5639, 63)

In [10]:
per_d_ins['정밀검사측정방법'].unique()

array(['부하검사(ASM-Idling)', '부하검사(KD-147)', '무부하검사(TSI)', '무부하검사(급가속)',
       '부하검사(LUG DOWN)', '무부하검사(정지가동)', '부하검사(ASM2525)'], dtype=object)

In [11]:
per_ins['검사 판정시간'].head()

0    20190702 14:51:40
1    20190702 14:52:45
2    20190702 14:49:28
3    20190702 15:58:57
4    20190702 14:52:04
Name: 검사 판정시간, dtype: object

In [13]:
per_d_ins['정밀검사시간정보'].head()

0    20190701 17:13:00
1    20190701 17:45:15
2                  NaN
3    20190701 17:17:41
4    20190701 17:23:59
Name: 정밀검사시간정보, dtype: object

In [14]:
per_ins['검사 판정시간'][0]

'20190702 14:51:40'

In [15]:
per_d_ins['정밀검사시간정보'][0]

'20190701 17:13:00'

In [16]:
per_ins['검사 판정시간'][0] > per_d_ins['정밀검사시간정보'][0]

True

In [17]:
per_ins['검사 판정시간'][0] < per_d_ins['정밀검사시간정보'][0]

False

### 정기 검사 데이터 확인

In [38]:
per_ins[['무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6']].isnull().sum()

무부하매연측정치1     1148260
무부하매연허용치1     1148260
무부하매연판정1      1148260
무부하매연측정치2    10820017
무부하매연허용치2    10820017
무부하매연판정2     10820017
무부하매연측정치3    10820017
무부하매연허용치3    10820052
무부하매연판정3     10820052
무부하매연측정치4    12029796
무부하매연허용치4    12029796
무부하매연판정4     12029796
무부하매연측정치5    12029796
무부하매연허용치5    12029796
무부하매연판정5     12029796
무부하매연측정치6    12029796
무부하매연허용치6    12029806
무부하매연판정6     12029806
dtype: int64

In [24]:
per_ins[per_ins['무부하매연측정치6'].isnull() == False].index

Int64Index([    1056,     1057,     1058,     1059,     1060,     1063,
                1064,     1067,     1068,     1070,
            ...
            16533202, 16533713, 16533714, 16533716, 16533722, 16533760,
            16533761, 16533766, 16533767, 16533769],
           dtype='int64', length=4505681)

In [26]:
# 6번 측정치 존재하면서 나머지 없는 경우 : 0
per_ins[(per_ins['무부하매연측정치6'].isnull() == False) & ((per_ins['무부하매연측정치1'].isnull() == True) | (per_ins['무부하매연측정치2'].isnull() == True) | (per_ins['무부하매연측정치3'].isnull() == True) | (per_ins['무부하매연측정치4'].isnull() == True) | (per_ins['무부하매연측정치5'].isnull() == True))].index

Int64Index([], dtype='int64')

In [27]:
# 5번 측정치 존재하면서 나머지 없는 경우 : 0
per_ins[(per_ins['무부하매연측정치5'].isnull() == False) & ((per_ins['무부하매연측정치1'].isnull() == True) | (per_ins['무부하매연측정치2'].isnull() == True) | (per_ins['무부하매연측정치3'].isnull() == True) | (per_ins['무부하매연측정치4'].isnull() == True))].index

Int64Index([], dtype='int64')

In [28]:
# 4번 측정치 존재하면서 나머지 없는 경우 : 0
per_ins[(per_ins['무부하매연측정치4'].isnull() == False) & ((per_ins['무부하매연측정치1'].isnull() == True) | (per_ins['무부하매연측정치2'].isnull() == True) | (per_ins['무부하매연측정치3'].isnull() == True))].index

Int64Index([], dtype='int64')

In [29]:
# 3번 측정치 존재하면서 나머지 없는 경우 : 0
per_ins[(per_ins['무부하매연측정치3'].isnull() == False) & ((per_ins['무부하매연측정치1'].isnull() == True) | (per_ins['무부하매연측정치2'].isnull() == True))].index

Int64Index([], dtype='int64')

In [30]:
# 2번 측정치 존재하면서 나머지 없는 경우 : 0
per_ins[(per_ins['무부하매연측정치3'].isnull() == False) & ((per_ins['무부하매연측정치1'].isnull() == True))].index

Int64Index([], dtype='int64')

In [31]:
per_ins.loc[1056, ['무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '검사 판정시간']].to_list()

[0.0,
 1.0,
 'Y',
 15.0,
 120.0,
 'Y',
 1.01,
 0.1,
 'Y',
 0.1,
 1.0,
 'Y',
 31.0,
 120.0,
 'Y',
 1.0,
 0.1,
 'Y',
 '20190701 13:44:39']

In [43]:
temp = per_ins.loc[0, ['무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '검사 판정시간']]
temp

무부하매연측정치1                  2.0
무부하매연허용치1                 25.0
무부하매연판정1                     Y
무부하매연측정치2                  NaN
무부하매연허용치2                  NaN
무부하매연판정2                   NaN
무부하매연측정치3                  NaN
무부하매연허용치3                  NaN
무부하매연판정3                   NaN
무부하매연측정치4                  NaN
무부하매연허용치4                  NaN
무부하매연판정4                   NaN
무부하매연측정치5                  NaN
무부하매연허용치5                  NaN
무부하매연판정5                   NaN
무부하매연측정치6                  NaN
무부하매연허용치6                  NaN
무부하매연판정6                   NaN
검사 판정시간      20190702 14:51:40
Name: 0, dtype: object

In [44]:
temp['무부하매연측정치1']

2.0

In [76]:
np.isnan(temp['무부하매연측정치2'])

True

### 정기검사 Grade 점수 계산 테스트

In [47]:
float('0.0')

0.0

In [53]:
float(np.nan)

nan

In [62]:
gas_mea_value_p = re.compile('\d[.]\d')
gas_mea_value_p.search('0.0')

<re.Match object; span=(0, 3), match='0.0'>

In [69]:
gas_mea_value_p.search('20190701 13:44:39')

In [64]:
gas_mea_value_p.search('H5262021070100191')

In [65]:
gas_mea_value_p.search('20210702')

In [92]:
temp['무부하매연측정치1']

2.0

In [81]:
np.isnan(temp['무부하매연측정치1'])

False

In [84]:
temp

무부하매연측정치1                  2.0
무부하매연허용치1                 25.0
무부하매연판정1                     Y
무부하매연측정치2                  NaN
무부하매연허용치2                  NaN
무부하매연판정2                   NaN
무부하매연측정치3                  NaN
무부하매연허용치3                  NaN
무부하매연판정3                   NaN
무부하매연측정치4                  NaN
무부하매연허용치4                  NaN
무부하매연판정4                   NaN
무부하매연측정치5                  NaN
무부하매연허용치5                  NaN
무부하매연판정5                   NaN
무부하매연측정치6                  NaN
무부하매연허용치6                  NaN
무부하매연판정6                   NaN
검사 판정시간      20190702 14:51:40
Name: 0, dtype: object

In [86]:
gas_mea_value_p.search(str(temp['무부하매연측정치1']))

<re.Match object; span=(0, 3), match='2.0'>

In [91]:
temp['무부하매연판정1']

'Y'

In [94]:
temp['무부하매연판정6']

nan

In [93]:
np.isnan(temp['무부하매연판정6'])

True

In [96]:
temp['무부하매연판정1'] not in ['Y', 'N']

False

In [97]:
# 1개의 배출가스 인증번호에 대한 처리 테스트
gas_mea_value_p = re.compile('\d[.]\d')
g_mea_list = []
p_f_list = []

# 변수 생성(측정치, 문자열)
sv1 = str(temp['무부하매연측정치1'])

sv3 = str(temp['무부하매연측정치3'])
sv4 = str(temp['무부하매연측정치4'])
sv5 = str(temp['무부하매연측정치5'])
sv6 = str(temp['무부하매연측정치6'])

# 변수 생성(측정치, 문자열)
v1 = temp['무부하매연측정치1']

v3 = temp['무부하매연측정치3']
v4 = temp['무부하매연측정치4']
v5 = temp['무부하매연측정치5']
v6 = temp['무부하매연측정치6']

# 변수 생성(판정)
f1 = temp['무부하매연판정1']
f3 = temp['무부하매연판정3']
f4 = temp['무부하매연판정4']
f5 = temp['무부하매연판정5']
f6 = temp['무부하매연판정6']

# 값 확인
if np.isnan(v6) or not gas_mea_value_p.search(sv6) or f6 not in ['Y', 'N']: # 6번째 값 부재
    print('6번째 값 부재')
    if np.isnan(v5) or not gas_mea_value_p.search(sv5) or f5 not in ['Y', 'N']: # 5번째 값 부재
        print('5번째 값 부재')
        if np.isnan(v4) or not gas_mea_value_p.search(sv4) or f4 not in ['Y', 'N']: # 4번째 값 부재
            print('4번째 값 부재')
            if np.isnan(v3) or not gas_mea_value_p.search(sv3) or f3 not in ['Y', 'N']: # 3번째 값 부재
                print('3번째 값 부재')
                if np.isnan(v1) or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                    print('1번째 값 부재')
                    g = np.nan
                    f = np.nan
                else: # 1번째 값 존재
                    g = v1
                    f = f1
            else: # 3번째 값 존재
                g = temp['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3'].mean()
                f = f3 
        else: # 4번째 값 존재
            g = temp['무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4'].mean()
            f = f4
    else: # 5번째 값 존재
        g = temp['무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5'].mean()
        f = f5
else: # 6번째 값 존재
    g = temp['무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6'].mean()
    f = f6
g_mea_list.append(g)
p_f_list.append(f)

6번째 값 부재
5번째 값 부재
4번째 값 부재
3번째 값 부재


In [98]:
g_mea_list, p_f_list

([2.0], ['Y'])

### 정밀 검사 데이터 확인

In [32]:
per_d_ins[per_d_ins['무부하매연측정치6'].isnull() == False].index

Int64Index([       0,        1,        3,        4,        6,        7,
                   9,       11,       12,       13,
            ...
            22397333, 22397334, 22397335, 22397336, 22397337, 22397338,
            22397339, 22397340, 22397341, 22397342],
           dtype='int64', length=17552908)

In [33]:
per_d_ins.loc[0, ['무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '정밀검사시간정보']].to_list()

[0.03,
 0.46,
 'Y',
 3.0,
 37.0,
 'Y',
 3.0,
 610.0,
 'Y',
 0.0,
 1.0,
 'Y',
 2.0,
 120.0,
 'Y',
 1.02,
 0.1,
 'Y',
 '20190701 17:13:00']

In [34]:
# 6번 측정치 존재하면서 나머지 없는 경우 : 0
per_d_ins[(per_d_ins['무부하매연측정치6'].isnull() == False) & ((per_d_ins['무부하매연측정치1'].isnull() == True) | (per_d_ins['무부하매연측정치2'].isnull() == True) | (per_d_ins['무부하매연측정치3'].isnull() == True) | (per_d_ins['무부하매연측정치4'].isnull() == True) | (per_d_ins['무부하매연측정치5'].isnull() == True))].index

Int64Index([13915437, 13915439, 13915442, 13915443, 13915455, 13915456,
            13915460, 13915461, 13915464, 13915465,
            ...
            22397306, 22397307, 22397309, 22397311, 22397316, 22397317,
            22397320, 22397324, 22397331, 22397335],
           dtype='int64', length=2314645)

In [35]:
per_d_ins.loc[13915437, ['무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '정밀검사시간정보']].to_list()

[0.0,
 15.0,
 'Y',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '20210701 15:50:27',
 'H5262021070100191',
 '20210702',
 nan]

In [36]:
per_d_ins.loc[13915439, ['무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '정밀검사시간정보']].to_list()

[2.0,
 20.0,
 'Y',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '20210701 16:41:21',
 'L0102021070100311',
 '20210702',
 nan]

In [99]:
# 5번 측정치 존재하면서 나머지 없는 경우 : 0
per_d_ins[(per_d_ins['무부하매연측정치5'].isnull() == False) & ((per_d_ins['무부하매연측정치1'].isnull() == True) | (per_d_ins['무부하매연측정치2'].isnull() == True) | (per_d_ins['무부하매연측정치3'].isnull() == True) | (per_d_ins['무부하매연측정치4'].isnull() == True))].index

Int64Index([], dtype='int64')

In [100]:
# 4번 측정치 존재하면서 나머지 없는 경우 : 0
per_d_ins[(per_d_ins['무부하매연측정치4'].isnull() == False) & ((per_d_ins['무부하매연측정치1'].isnull() == True) | (per_d_ins['무부하매연측정치2'].isnull() == True) | (per_d_ins['무부하매연측정치3'].isnull() == True))].index

Int64Index([], dtype='int64')

In [101]:
# 3번 측정치 존재하면서 나머지 없는 경우 : 0
per_d_ins[(per_d_ins['무부하매연측정치3'].isnull() == False) & ((per_d_ins['무부하매연측정치1'].isnull() == True) | (per_d_ins['무부하매연측정치2'].isnull() == True))].index

Int64Index([], dtype='int64')

In [102]:
# 2번 측정치 존재하면서 나머지 없는 경우 : 0
per_d_ins[(per_d_ins['무부하매연측정치3'].isnull() == False) & ((per_d_ins['무부하매연측정치1'].isnull() == True))].index

Int64Index([], dtype='int64')

### 제원정보와 정기/정밀 매칭 안되는 샘플

In [583]:
df4['차대번호'].isnull().sum()

0

In [584]:
len(df4['차대번호'].unique())

1528557

In [585]:
len(per_ins['차대번호'].unique())

10328354

In [586]:
len(per_d_ins['차대번호'].unique())

14203268

In [587]:
len(set(df4['차대번호']))

1528557

In [588]:
len(set(df4['차대번호']) - set(per_ins['차대번호']))

1047521

In [589]:
len(set(df4['차대번호']) - set(per_d_ins['차대번호']))

540567

In [590]:
len( set(df4['차대번호']) - set(per_ins['차대번호']) - set(per_d_ins['차대번호']) )

323516

## 제원정보, 정기 검사, 정밀 검사 병합(차대번호 기준)

In [273]:
df6.shape

(1528557, 62)

In [275]:
gas_col1 = ['차대번호', '검사방법', 
            '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', 
            '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', 
            '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', 
            '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
            '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5',
            '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', 
            '검사 판정시간',]
gas_col2 = ['차대번호', '정밀검사측정방법', 
            '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', 
            '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', 
            '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', 
            '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4', 
            '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', 
            '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6',
            '정밀검사시간정보']

In [276]:
# 약 50s
tdf1 = df6.merge(per_ins[gas_col1], on='차대번호', how='left')
tdf1.shape

(2232455, 82)

In [277]:
# 약 1m
tdf2 = df6.merge(per_d_ins[gas_col2], on='차대번호', how='left')
tdf2.shape

(2873363, 82)

In [278]:
# 약 2m 40s
tdf1.to_csv('data/[G4][통합]제원정보_정기_20190701_20220630.csv')
tdf2.to_csv('data/[G4][통합]제원정보_정밀_20190701_20220630.csv')

# [로드] 제원+정기, 제원+정밀
- [G4][통합]제원정보_정기_20190701_20220630.csv
- [G4][통합]제원정보_정밀_20190701_20220630.csv

In [8]:
# 약 1m
tdf1 = pd.read_csv('data/[G4][통합]제원정보_정기_20190701_20220630.csv', index_col=0)
tdf2 = pd.read_csv('data/[G4][통합]제원정보_정밀_20190701_20220630.csv', index_col=0)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_456\802190608.py:2: DtypeWarning: Columns (12,15,17,29,37,38,58,69,72,75,78,81) have mixed types. Specify dtype option on import or set low_memory=False.
  tdf1 = pd.read_csv('data/[G4][통합]제원정보_정기_20190701_20220630.csv', index_col=0)
C:\Users\kbjung\AppData\Local\Temp\ipykernel_456\802190608.py:3: DtypeWarning: Columns (12,15,17,29,37,38,58,81) have mixed types. Specify dtype option on import or set low_memory=False.
  tdf2 = pd.read_csv('data/[G4][통합]제원정보_정밀_20190701_20220630.csv', index_col=0)


In [9]:
tdf1.shape

(2232455, 82)

In [10]:
tdf2.shape

(2873363, 82)

In [11]:
tdf1.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '검사방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '검사 판정시간'],
      dtype='object')

In [12]:
tdf2.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', '총중량_분류',
       '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE',
       'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description',
       '참고', 'VIN_length', 'IOQ', 'alpha', '정밀검사측정방법', '무부하매연측정치1',
       '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2',
       '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4',
       '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6',
       '무부하매연허용치6', '무부하매연판정6', '정밀검사시간정보'],
      dtype='object')

In [13]:
tdf1['차종유형'].unique()

array(['대형', '중형', '소형', '경형'], dtype=object)

In [14]:
tdf1['차종_등록정보'].unique()

array(['화물', '승용', '특수', '승합', nan], dtype=object)

In [15]:
tdf1['연료'].unique()

array(['경유', '휘발유', '기타연료', 'LPG(액화석유가스)', 'CNG(압축천연가스)', '알코올'],
      dtype=object)

In [16]:
tdf1['차종분류'].unique()

array(['특수용도형-특수용도형', '다목적', '구난', '일반형-카고', '일반', '밴', '기타', '견인', '덤프',
       '특수작업형', '일반형-픽업', '특수용도형-피견인', '특수용도형-탱크로리', '특수', '승용겸화물'],
      dtype=object)

In [17]:
tdf2['차종유형'].unique()

array(['대형', '중형', '소형', '경형'], dtype=object)

In [18]:
tdf2['차종_등록정보'].unique()

array(['화물', '승용', '특수', '승합', nan], dtype=object)

In [19]:
tdf2['연료'].unique()

array(['경유', '휘발유', '기타연료', 'LPG(액화석유가스)', 'CNG(압축천연가스)', '알코올'],
      dtype=object)

### 제원_정기 파일 전처리

In [20]:
tdf1['검사방법'].unique()

array([nan, '무부하검사(급가속)', '무부하검사(TSI)', '무부하검사(정지가동)'], dtype=object)

In [21]:
# 제원_정기 파일 정렬
# 차대번호, 검사방법, 검사 판정시간
    # 최신순으로
# 차대번호 기준 중복 제거

In [22]:
gas_col1 = ['차대번호', '차량제작일자', '배출가스인증번호', '총중량', '검사방법', '차종유형', '차종_등록정보', '연료',
           '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1',
           '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2',
           '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', 
           '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4', 
           '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5',
           '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', 
           '검사 판정시간']
gas_per_ins = tdf1[gas_col1].copy()
gas_per_ins.shape

(2232455, 27)

In [23]:
gas_per_ins.head()

,차대번호,차량제작일자,배출가스인증번호,총중량,검사방법,차종유형,차종_등록정보,연료,무부하매연측정치1,무부하매연허용치1,...,무부하매연측정치4,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사 판정시간
0,KMCLK18KP8C035195,20080924.0,NaN,10825.0,NaN,대형,화물,경유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KMFLA18KP8C031274,20080605.0,NaN,11075.0,NaN,대형,화물,경유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KLACA26RD8B226753,20080221.0,NaN,2215.0,NaN,중형,승용,경유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KMCGK17HP8C086937,20080315.0,NaN,6150.0,NaN,중형,화물,경유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KMCYSS7JP7U084697,20070227.0,NaN,3155.0,NaN,중형,특수,경유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
gas_per_ins.sort_values(['차대번호', '검사방법', '검사 판정시간']).head()

,차대번호,차량제작일자,배출가스인증번호,총중량,검사방법,차종유형,차종_등록정보,연료,무부하매연측정치1,무부하매연허용치1,...,무부하매연측정치4,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사 판정시간
2198852,018013212A7,NaN,3MY-FK-08,1785.0,NaN,대형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2225371,01831300012,19880823.0,NaN,2145.0,NaN,중형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2215936,01841300699,19890328.0,NaN,1595.0,NaN,중형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2221443,01871300044,NaN,NaN,1540.0,NaN,대형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2218846,01881100314,19890125.0,NaN,2620.0,NaN,중형,승합,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
gas_per_ins = gas_per_ins.sort_values(['차대번호', '검사방법', '검사 판정시간']).drop_duplicates('차대번호', keep='first').reset_index(drop=True)
gas_per_ins.shape

(1528557, 27)

In [27]:
gas_per_ins.head()

,차대번호,차량제작일자,배출가스인증번호,총중량,검사방법,차종유형,차종_등록정보,연료,무부하매연측정치1,무부하매연허용치1,...,무부하매연측정치4,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사 판정시간
0,018013212A7,NaN,3MY-FK-08,1785.0,NaN,대형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01831300012,19880823.0,NaN,2145.0,NaN,중형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01841300699,19890328.0,NaN,1595.0,NaN,중형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01871300044,NaN,NaN,1540.0,NaN,대형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01881100314,19890125.0,NaN,2620.0,NaN,중형,승합,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
gas_per_ins['배출가스인증번호'].isnull().sum()

123003

In [29]:
gas_per_ins['검사방법'].isnull().sum()

1047521

In [30]:
gas_per_ins['무부하매연측정치1'].isnull().sum()

1047552

## 제원_정밀 파일 전처리

In [31]:
tdf2['정밀검사측정방법'].unique()

array([nan, '부하검사(LUG DOWN)', '부하검사(KD-147)', '부하검사(ASM-Idling)',
       '무부하검사(급가속)', '무부하검사(TSI)', '무부하검사(정지가동)'], dtype=object)

In [32]:
# 제원_정밀 파일 정렬
# 차대번호, 정밀검사측정방법, 정밀검사시간정보
    # 최신순으로
# 차대번호 기준 중복 제거

In [33]:
gas_col2 = ['차대번호', '차량제작일자','배출가스인증번호', '총중량', '정밀검사측정방법', '차종유형', '차종_등록정보', '연료',
           '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1',
           '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2',
           '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', 
           '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4', 
           '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5',
           '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', 
           '정밀검사시간정보']
gas_per_d_ins = tdf2[gas_col2].copy()
gas_per_d_ins.shape

(2873363, 27)

In [34]:
gas_per_d_ins.head()

,차대번호,차량제작일자,배출가스인증번호,총중량,정밀검사측정방법,차종유형,차종_등록정보,연료,무부하매연측정치1,무부하매연허용치1,...,무부하매연측정치4,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,정밀검사시간정보
0,KMCLK18KP8C035195,20080924.0,NaN,10825.0,NaN,대형,화물,경유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KMFLA18KP8C031274,20080605.0,NaN,11075.0,NaN,대형,화물,경유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KLACA26RD8B226753,20080221.0,NaN,2215.0,NaN,중형,승용,경유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KMCGK17HP8C086937,20080315.0,NaN,6150.0,NaN,중형,화물,경유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KMCYSS7JP7U084697,20070227.0,NaN,3155.0,NaN,중형,특수,경유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
gas_per_d_ins.sort_values(['차대번호', '정밀검사측정방법', '정밀검사시간정보']).head()

,차대번호,차량제작일자,배출가스인증번호,총중량,정밀검사측정방법,차종유형,차종_등록정보,연료,무부하매연측정치1,무부하매연허용치1,...,무부하매연측정치4,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,정밀검사시간정보
2839486,018013212A7,NaN,3MY-FK-08,1785.0,NaN,대형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2866198,01831300012,19880823.0,NaN,2145.0,NaN,중형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2856680,01841300699,19890328.0,NaN,1595.0,NaN,중형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2862220,01871300044,NaN,NaN,1540.0,NaN,대형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2859618,01881100314,19890125.0,NaN,2620.0,NaN,중형,승합,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
gas_per_d_ins = gas_per_d_ins.sort_values(['차대번호', '정밀검사측정방법', '정밀검사시간정보']).drop_duplicates('차대번호', keep='first').reset_index(drop=True)
gas_per_d_ins.shape

(1528557, 27)

In [37]:
gas_per_d_ins.head()

,차대번호,차량제작일자,배출가스인증번호,총중량,정밀검사측정방법,차종유형,차종_등록정보,연료,무부하매연측정치1,무부하매연허용치1,...,무부하매연측정치4,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,정밀검사시간정보
0,018013212A7,NaN,3MY-FK-08,1785.0,NaN,대형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01831300012,19880823.0,NaN,2145.0,NaN,중형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01841300699,19890328.0,NaN,1595.0,NaN,중형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01871300044,NaN,NaN,1540.0,NaN,대형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01881100314,19890125.0,NaN,2620.0,NaN,중형,승합,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
gas_per_d_ins['배출가스인증번호'].isnull().sum()

123003

In [39]:
gas_per_d_ins['정밀검사측정방법'].isnull().sum()

540567

In [40]:
gas_per_d_ins['무부하매연측정치1'].isnull().sum()

545132

## 두 파일 concat(제원+정기, 제원+정밀)

In [41]:
gas_per_ins['정기정밀'] = '정기'

In [42]:
gas_per_ins.columns

Index(['차대번호', '차량제작일자', '배출가스인증번호', '총중량', '검사방법', '차종유형', '차종_등록정보', '연료',
       '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2',
       '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4',
       '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5',
       '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '검사 판정시간', '정기정밀'],
      dtype='object')

In [43]:
gas_per_ins.columns = ['차대번호', '차량제작일자', '배출가스인증번호', '총중량', '검사방법', '차종유형', '차종등록정보', '연료',
                     '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1',
                     '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', 
                     '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', 
                     '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
                     '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', 
                     '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6',
                     '검사판정시간', '정기정밀']

In [44]:
gas_per_ins.columns

Index(['차대번호', '차량제작일자', '배출가스인증번호', '총중량', '검사방법', '차종유형', '차종등록정보', '연료',
       '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2',
       '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4',
       '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5',
       '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '검사판정시간', '정기정밀'],
      dtype='object')

In [45]:
gas_per_d_ins['정기정밀'] = '정밀'

In [46]:
gas_per_d_ins.columns

Index(['차대번호', '차량제작일자', '배출가스인증번호', '총중량', '정밀검사측정방법', '차종유형', '차종_등록정보',
       '연료', '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2',
       '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4',
       '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5',
       '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '정밀검사시간정보', '정기정밀'],
      dtype='object')

In [47]:
gas_per_d_ins.columns = ['차대번호', '차량제작일자', '배출가스인증번호', '총중량', '검사방법', '차종유형', '차종등록정보', '연료',
                     '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1',
                     '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', 
                     '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', 
                     '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4', 
                     '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', 
                     '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6',
                     '검사판정시간', '정기정밀']

In [48]:
gas_per_d_ins.columns

Index(['차대번호', '차량제작일자', '배출가스인증번호', '총중량', '검사방법', '차종유형', '차종등록정보', '연료',
       '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2',
       '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4',
       '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5',
       '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '검사판정시간', '정기정밀'],
      dtype='object')

In [49]:
gas_per_ins.shape, gas_per_d_ins.shape

((1528557, 28), (1528557, 28))

In [50]:
total = pd.concat([gas_per_ins, gas_per_d_ins], ignore_index=True)
total.shape

(3057114, 28)

In [51]:
total.shape[0] == gas_per_ins.shape[0] + gas_per_d_ins.shape[0]

True

### [출력] 통합 제원, 정기 concat 정밀

In [541]:
# 약 28s
total.to_csv('data/[G4][통합]제원정보_정기_concat_정밀_20190701_20220630.csv')

## [로드] 제원, 정기 concat 정밀

In [52]:
# # 약 
# total = pd.read_csv('data/[G4][통합]제원정보_정기_concat_정밀_20190701_20220630.csv', index_col=0)

## 검사판정시간으로 정렬

In [53]:
# 6개 index : 6, 1528566
total.sort_values(['차대번호', '검사방법', '검사판정시간']).head()

,차대번호,차량제작일자,배출가스인증번호,총중량,검사방법,차종유형,차종등록정보,연료,무부하매연측정치1,무부하매연허용치1,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
0,018013212A7,NaN,3MY-FK-08,1785.0,NaN,대형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,정기
1528557,018013212A7,NaN,3MY-FK-08,1785.0,NaN,대형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,정밀
1,01831300012,19880823.0,NaN,2145.0,NaN,중형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,정기
1528558,01831300012,19880823.0,NaN,2145.0,NaN,중형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,정밀
2,01841300699,19890328.0,NaN,1595.0,NaN,중형,승용,휘발유,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,정기


In [54]:
total.loc[1, '차량제작일자']

19880823.0

In [55]:
type(total.loc[1, '차량제작일자'])

numpy.float64

In [56]:
total.loc[13, '검사판정시간']

'20210706 15:59:05'

In [57]:
total.loc[13, '검사판정시간'] < '20221212'

True

In [58]:
total.loc[13, '총중량']

2765.0

In [59]:
type(total.loc[13, '총중량'])

numpy.float64

In [60]:
total.loc[0, '차량제작일자']

nan

In [61]:
total.loc[0, '차량제작일자']

nan

In [62]:
total.shape

(3057114, 28)

### 차대번호, 검사방법, 검사판정시간 정렬 차대번호 기준 중복 제거

In [63]:
d_total = total.sort_values(['차대번호', '검사방법', '검사판정시간']).drop_duplicates('차대번호', keep='first').reset_index(drop=True)
d_total.shape

(1528557, 28)

In [64]:
d_total['배출가스인증번호'].isnull().sum()

123003

In [65]:
uni = d_total['배출가스인증번호'].unique()
len(uni)

301

In [66]:
d_total['검사방법'].unique()

array([nan, '무부하검사(TSI)', '부하검사(ASM-Idling)', '부하검사(KD-147)',
       '무부하검사(정지가동)', '무부하검사(급가속)', '부하검사(LUG DOWN)'], dtype=object)

In [67]:
d_total['검사방법'].isnull().sum()

323516

In [68]:
d_total['연료'].unique()

array(['휘발유', 'LPG(액화석유가스)', '경유', '알코올', '기타연료', 'CNG(압축천연가스)'],
      dtype=object)

In [69]:
str(np.nan)

'nan'

In [70]:
str(np.nan) == 'nan'

True

In [72]:
d_total.shape

(1528557, 28)

### 측정치, 허용치 float 형태로 타입 변환

In [105]:
d_total['무부하매연측정치1'] = pd.to_numeric(d_total['무부하매연측정치1'], errors='coerce')
d_total['무부하매연측정치2'] = pd.to_numeric(d_total['무부하매연측정치2'], errors='coerce')
d_total['무부하매연측정치3'] = pd.to_numeric(d_total['무부하매연측정치3'], errors='coerce')
d_total['무부하매연측정치4'] = pd.to_numeric(d_total['무부하매연측정치4'], errors='coerce')
d_total['무부하매연측정치5'] = pd.to_numeric(d_total['무부하매연측정치5'], errors='coerce')
d_total['무부하매연측정치6'] = pd.to_numeric(d_total['무부하매연측정치6'], errors='coerce')

d_total[['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528557 entries, 0 to 1528556
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   무부하매연측정치1  1203560 non-null  float64
 1   무부하매연측정치2  347567 non-null   float64
 2   무부하매연측정치3  317220 non-null   float64
 3   무부하매연측정치4  315988 non-null   float64
 4   무부하매연측정치5  315988 non-null   float64
 5   무부하매연측정치6  296808 non-null   float64
dtypes: float64(6)
memory usage: 70.0 MB


## [EG] 경유(d_total2)

In [199]:
d_total2 = d_total[d_total['연료'] == '경유'].reset_index(drop=True).dropna(subset=['검사방법', '배출가스인증번호'])
d_total2.shape

(1132386, 28)

In [200]:
uni2 = d_total2['배출가스인증번호'].unique()
len(uni2)

204

In [201]:
d_total2['배출가스인증번호'].unique()[3]

'7MY-HD-14-46'

In [202]:
d_total2['배출가스인증번호'].value_counts()

7MY-HD-14-68     261128
8MY-KM-13-19      90562
6MY-KM-14-61      75495
9MY-KM-14-41      66558
7MY-HD-14-46      59127
                  ...  
9MY-PP-14-409         1
8MY-SN-0M-24          1
8MY-FM-0M-24          1
8MY-ET-0M-24          1
7MY-KL-0M-24          1
Name: 배출가스인증번호, Length: 204, dtype: int64

In [203]:
d_total2['검사방법'].unique()

array(['부하검사(KD-147)', '무부하검사(급가속)', '부하검사(LUG DOWN)', '무부하검사(TSI)',
       '무부하검사(정지가동)', '부하검사(ASM-Idling)'], dtype=object)

In [204]:
temp = d_total2[d_total2['배출가스인증번호'] == '7MY-HD-14-68'].reset_index(drop=True)
temp

,차대번호,차량제작일자,배출가스인증번호,총중량,검사방법,차종유형,차종등록정보,연료,무부하매연측정치1,무부하매연허용치1,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
0,KM99H4ACD8CZ00001,20080705.0,7MY-HD-14-68,2905.0,부하검사(KD-147),중형,승합,경유,0.0,15.0,...,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,20200304 11:08:28,정밀
1,KMF2CY7JABU693781,20101118.0,7MY-HD-14-68,3250.0,무부하검사(급가속),소형,화물,경유,19.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191216 11:09:20,정기
2,KMFACS7JP9U482553,20081215.0,7MY-HD-14-68,2955.0,부하검사(KD-147),소형,화물,경유,13.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200104 12:10:22,정밀
3,KMFACY7JABU652019,20100706.0,7MY-HD-14-68,3270.0,무부하검사(급가속),소형,화물,경유,5.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210723 10:02:36,정기
4,KMFACZ7JABU750953,20110429.0,7MY-HD-14-68,2950.0,무부하검사(급가속),소형,화물,경유,21.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190821 10:04:40,정기
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261123,KNFZCD7JP9U509805,20090415.0,7MY-HD-14-68,3280.0,무부하검사(급가속),소형,화물,경유,7.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200414 15:22:19,정기
261124,KNFZCD7JP9U523649,20090530.0,7MY-HD-14-68,3250.0,부하검사(KD-147),소형,화물,경유,9.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200511 14:56:01,정밀
261125,KNFZCZ7JAAU607457,20100130.0,7MY-HD-14-68,2925.0,무부하검사(급가속),소형,화물,경유,0.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220318 10:35:50,정기
261126,KNFZCZ7JBAU579855,20091110.0,7MY-HD-14-68,2960.0,무부하검사(급가속),소형,화물,경유,3.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191129 15:26:53,정기


In [205]:
temp.shape

(261128, 28)

In [206]:
temp.loc[0]

차대번호         KM99H4ACD8CZ00001
차량제작일자              20080705.0
배출가스인증번호          7MY-HD-14-68
총중량                     2905.0
검사방법              부하검사(KD-147)
차종유형                        중형
차종등록정보                      승합
연료                          경유
무부하매연측정치1                  0.0
무부하매연허용치1                 15.0
무부하매연판정1                     Y
무부하매연측정치2                  0.0
무부하매연허용치2                  0.0
무부하매연판정2                   NaN
무부하매연측정치3                  0.0
무부하매연허용치3                  0.0
무부하매연판정3                   NaN
무부하매연측정치4                  0.0
무부하매연허용치4                  0.0
무부하매연판정4                   NaN
무부하매연측정치5                  0.0
무부하매연허용치5                  0.0
무부하매연판정5                   NaN
무부하매연측정치6                  0.0
무부하매연허용치6                  0.0
무부하매연판정6                   NaN
검사판정시간       20200304 11:08:28
정기정밀                        정밀
Name: 0, dtype: object

In [207]:
temp.loc[0]['검사방법']

'부하검사(KD-147)'

In [208]:
temp['검사방법'][0]

'부하검사(KD-147)'

In [209]:
temp['무부하매연측정치1'][0]

0.0

In [210]:
pd.Series([1, 2])

0    1
1    2
dtype: int64

In [211]:
pd.Series([1, 2]).shape

(2,)

In [212]:
car_num = d_total2['차대번호'].unique()
len(car_num)

1132386

In [213]:
d_total2[(d_total2['무부하매연측정치1'].isnull() == True) & (d_total2['무부하매연측정치1'].isnull() == True) & (d_total2['무부하매연측정치2'].isnull() == True) & (d_total2['무부하매연측정치3'].isnull() == True) & (d_total2['무부하매연측정치4'].isnull() == True) & (d_total2['무부하매연측정치5'].isnull() == True) & (d_total2['무부하매연측정치6'].isnull() == True)].shape

(1390, 28)

In [214]:
d_total2[d_total2['무부하매연측정치1'].isnull() == True].shape

(1390, 28)

In [215]:
d_total2['배출가스인증번호'].isnull().sum()

0

### [백업]코드

In [216]:
# if tm == '부하검사(LUG DOWN)': # 1번째 측정치, 판정치 사용
#             if np.isnan(v1) or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
#                 # print('1번째 값 부재')
#                 g = np.nan
#                 f = np.nan
#                 std = 0 
#             else: # 1번째 값 존재
#                 g = v1
#                 f = f1
#                 # 매연 허용치
#                 if np.isnan(tw):
#                     std = 0
#                 elif tw <= 5500:
#                     if np.isnan(pt):
#                         std = 0
#                     elif pt <= 19951231:
#                         std = 70
#                     elif 19960101 <= pt <= 20001231:
#                         std = 60
#                     elif 20010101 <= pt <= 20071231:
#                         std = 50
#                     elif 20080101 <= pt:
#                         std = 20
#                 else:
#                     if np.isnan(pt):
#                         std = 0
#                     elif pt <= 19951231:
#                         std = 50
#                     elif 19960101 <= pt <= 20001231:
#                         std = 45
#                     elif 20010101 <= pt <= 20071231:
#                         std = 30
#                     elif 20080101 <= pt:
#                         std = 15

#         elif tm == '부하검사(KD-147)': # 1번째 측정치, 판정치 사용
#             if np.isnan(v1) or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
#                 # print('1번째 값 부재')
#                 g = np.nan
#                 f = np.nan
#                 std = 0
#             else: # 1번째 값 존재
#                 g = v1
#                 f = f1
#                 # 매연 허용치
#                 if np.isnan(pt):
#                     std = 0
#                 elif pt < 19921231:
#                     if str(td) == 'nan':
#                         std = 0
#                     elif td <= '20111231':
#                         std = 50
#                     else:
#                         std = 45
#                 elif 19930101 <= pt <= 19951231:
#                     if str(td) == 'nan':
#                         std = 0
#                     elif td <= '20111231':
#                         std = 45
#                     else:
#                         std = 40
#                 elif 19960101 <= pt <= 20001231:
#                     if str(td) == 'nan':
#                         std = 0
#                     elif td <= '20111231':
#                         std = 40
#                     else:
#                         std = 35
#                 elif 20010101 <= pt <= 20071231:
#                     if str(td) == 'nan':
#                         std = 0
#                     elif td <= '20111231':
#                         std = 30
#                     else:
#                         std = 25
#                 elif 20080101 <= pt <= 20160831:
#                     if str(td) == 'nan':
#                         std = 0
#                     elif td <= '20111231':
#                         std = 20
#                     else:
#                         std = 15
#                 elif 20160901 <= pt <= 20171231:
#                     if str(td) == 'nan':
#                         std = 0
#                     elif td <= '20111231':
#                         std = 0
#                     else:
#                         std = 8
#                 elif 20180101 <= pt:
#                     if str(td) == 'nan':
#                         std = 0
#                     elif td <= '20111231':
#                         std = 0
#                     else:
#                         std = 8

#         else: # 무부하 검사
#             if np.isnan(v6) or not gas_mea_value_p.search(sv6) or f6 not in ['Y', 'N']: # 6번째 값 부재
#                 # print('6번째 값 부재')
#                 if np.isnan(v5) or not gas_mea_value_p.search(sv5) or f5 not in ['Y', 'N']: # 5번째 값 부재
#                     # print('5번째 값 부재')
#                     if np.isnan(v4) or not gas_mea_value_p.search(sv4) or f4 not in ['Y', 'N']: # 4번째 값 부재
#                         # print('4번째 값 부재')
#                         if np.isnan(v3) or not gas_mea_value_p.search(sv3) or f3 not in ['Y', 'N']: # 3번째 값 부재
#                             # print('3번째 값 부재')
#                             if np.isnan(v1) or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
#                                 # print('1번째 값 부재')
#                                 g = np.nan
#                                 f = np.nan
#                                 std = 0
#                             else: # 1번째 값 존재
#                                 g = v1
#                                 f = f1
#                                 # 매연 허용치
#                                 if str(ci) == 'nan' or str(ci2) == 'nan':
#                                     std = 0
#                                 elif ci == '경형' or ci2 == '승용':
#                                     if np.isnan(pt):
#                                         std = 0
#                                     elif pt < 19951231:
#                                         std = 60
#                                     elif 19960101 <= pt <= 20001231:
#                                         std = 55
#                                     elif 20010101 <= pt <= 20031231:
#                                         std = 45
#                                     elif 20040101 <= pt <= 20071231:
#                                         std = 40
#                                     elif 20080101 <= pt <= 20160831:
#                                         std = 20
#                                     elif 20160901 <= pt:
#                                         std = 10
#                                 else:
#                                     if ci2 == '소형':
#                                         if np.isnan(pt):
#                                             std = 0
#                                         elif pt < 19951231:
#                                             std = 60
#                                         elif 19960101 <= pt <= 20001231:
#                                             std = 55
#                                         elif 20010101 <= pt <= 20031231:
#                                             std = 45
#                                         elif 20040101 <= pt <= 20071231:
#                                             std = 40
#                                         elif 20080101 <= pt <= 20160831:
#                                             std = 20
#                                         elif 20160901 <= pt:
#                                             std = 10
#                                     elif ci2 == '중형':
#                                         if np.isnan(pt):
#                                             std = 0
#                                         elif pt < 19921231:
#                                             std = 60
#                                         elif 19930101 <= pt <= 19951231:
#                                             std = 55
#                                         elif 19960101 <= pt <= 19971231:
#                                             std = 45
#                                         elif 19980101 <= pt <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
#                                             std = 45
#                                         elif 20010101 <= pt <= 20040930:
#                                             std = 45
#                                         elif 20041001 <= pt <= 20071231:
#                                             std = 40
#                                         elif 20080101 <= pt <= 20160831:
#                                             std = 20
#                                         elif 20160901 <= pt:
#                                             std = 10
#                                     elif ci2 == '대형':
#                                         if np.isnan(pt):
#                                             std = 0
#                                         elif pt < 19921231:
#                                             std = 60
#                                         elif 19930101 <= pt <= 19951231:
#                                             std = 55
#                                         elif 19960101 <= pt <= 19971231:
#                                             std = 45
#                                         elif 19980101 <= pt <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
#                                             std = 45
#                                         elif 20010101 <= pt <= 20040930:
#                                             std = 45
#                                         elif 20041001 <= pt <= 20071231:
#                                             std = 40
#                                         elif 20080101 <= pt:
#                                             std = 20
                                    
#                         else: # 3번째 값 존재
#                             g = temp['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3'].mean()
#                             f = f3
#                             # 매연 허용치
#                             if str(ci) == 'nan' or str(ci2) == 'nan':
#                                 std = 0
#                             elif ci == '경형' or ci2 == '승용':
#                                 if np.isnan(pt):
#                                     std = 0
#                                 elif pt < 19951231:
#                                     std = 60
#                                 elif 19960101 <= pt <= 20001231:
#                                     std = 55
#                                 elif 20010101 <= pt <= 20031231:
#                                     std = 45
#                                 elif 20040101 <= pt <= 20071231:
#                                     std = 40
#                                 elif 20080101 <= pt <= 20160831:
#                                     std = 20
#                                 elif 20160901 <= pt:
#                                     std = 10
#                             else:
#                                 if ci2 == '소형':
#                                     if np.isnan(pt):
#                                         std = 0
#                                     elif pt < 19951231:
#                                         std = 60
#                                     elif 19960101 <= pt <= 20001231:
#                                         std = 55
#                                     elif 20010101 <= pt <= 20031231:
#                                         std = 45
#                                     elif 20040101 <= pt <= 20071231:
#                                         std = 40
#                                     elif 20080101 <= pt <= 20160831:
#                                         std = 20
#                                     elif 20160901 <= pt:
#                                         std = 10
#                                 elif ci2 == '중형':
#                                     if np.isnan(pt):
#                                         std = 0
#                                     elif pt < 19921231:
#                                         std = 60
#                                     elif 19930101 <= pt <= 19951231:
#                                         std = 55
#                                     elif 19960101 <= pt <= 19971231:
#                                         std = 45
#                                     elif 19980101 <= pt <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
#                                         std = 45
#                                     elif 20010101 <= pt <= 20040930:
#                                         std = 45
#                                     elif 20041001 <= pt <= 20071231:
#                                         std = 40
#                                     elif 20080101 <= pt <= 20160831:
#                                         std = 20
#                                     elif 20160901 <= pt:
#                                         std = 10
#                                 elif ci2 == '대형':
#                                     if np.isnan(pt):
#                                         std = 0
#                                     elif pt < 19921231:
#                                         std = 60
#                                     elif 19930101 <= pt <= 19951231:
#                                         std = 55
#                                     elif 19960101 <= pt <= 19971231:
#                                         std = 45
#                                     elif 19980101 <= pt <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
#                                         std = 45
#                                     elif 20010101 <= pt <= 20040930:
#                                         std = 45
#                                     elif 20041001 <= pt <= 20071231:
#                                         std = 40
#                                     elif 20080101 <= pt:
#                                         std = 20
#                     else: # 4번째 값 존재
#                         g = temp['무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4'].mean()
#                         f = f4
#                         # 매연 허용치
#                         if str(ci) == 'nan' or str(ci2) == 'nan':
#                             std = 0
#                         elif ci == '경형' or ci2 == '승용':
#                             if np.isnan(pt):
#                                 std = 0
#                             elif pt < 19951231:
#                                 std = 60
#                             elif 19960101 <= pt <= 20001231:
#                                 std = 55
#                             elif 20010101 <= pt <= 20031231:
#                                 std = 45
#                             elif 20040101 <= pt <= 20071231:
#                                 std = 40
#                             elif 20080101 <= pt <= 20160831:
#                                 std = 20
#                             elif 20160901 <= pt:
#                                 std = 10
#                         else:
#                             if ci2 == '소형':
#                                 if np.isnan(pt):
#                                     std = 0
#                                 elif pt < 19951231:
#                                     std = 60
#                                 elif 19960101 <= pt <= 20001231:
#                                     std = 55
#                                 elif 20010101 <= pt <= 20031231:
#                                     std = 45
#                                 elif 20040101 <= pt <= 20071231:
#                                     std = 40
#                                 elif 20080101 <= pt <= 20160831:
#                                     std = 20
#                                 elif 20160901 <= pt:
#                                     std = 10
#                             elif ci2 == '중형':
#                                 if np.isnan(pt):
#                                     std = 0
#                                 elif pt < 19921231:
#                                     std = 60
#                                 elif 19930101 <= pt <= 19951231:
#                                     std = 55
#                                 elif 19960101 <= pt <= 19971231:
#                                     std = 45
#                                 elif 19980101 <= pt <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
#                                     std = 45
#                                 elif 20010101 <= pt <= 20040930:
#                                     std = 45
#                                 elif 20041001 <= pt <= 20071231:
#                                     std = 40
#                                 elif 20080101 <= pt <= 20160831:
#                                     std = 20
#                                 elif 20160901 <= pt:
#                                     std = 10
#                             elif ci2 == '대형':
#                                 if np.isnan(pt):
#                                     std = 0
#                                 elif pt < 19921231:
#                                     std = 60
#                                 elif 19930101 <= pt <= 19951231:
#                                     std = 55
#                                 elif 19960101 <= pt <= 19971231:
#                                     std = 45
#                                 elif 19980101 <= pt <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
#                                     std = 45
#                                 elif 20010101 <= pt <= 20040930:
#                                     std = 45
#                                 elif 20041001 <= pt <= 20071231:
#                                     std = 40
#                                 elif 20080101 <= pt:
#                                     std = 20
#                 else: # 5번째 값 존재
#                     g = temp['무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5'].mean()
#                     f = f5
#                     # 매연 허용치
#                     if str(ci) == 'nan' or str(ci2) == 'nan':
#                         std = 0
#                     elif ci == '경형' or ci2 == '승용':
#                         if np.isnan(pt):
#                             std = 0
#                         elif pt < 19951231:
#                             std = 60
#                         elif 19960101 <= pt <= 20001231:
#                             std = 55
#                         elif 20010101 <= pt <= 20031231:
#                             std = 45
#                         elif 20040101 <= pt <= 20071231:
#                             std = 40
#                         elif 20080101 <= pt <= 20160831:
#                             std = 20
#                         elif 20160901 <= pt:
#                             std = 10
#                     else:
#                         if ci2 == '소형':
#                             if np.isnan(pt):
#                                 std = 0
#                             elif pt < 19951231:
#                                 std = 60
#                             elif 19960101 <= pt <= 20001231:
#                                 std = 55
#                             elif 20010101 <= pt <= 20031231:
#                                 std = 45
#                             elif 20040101 <= pt <= 20071231:
#                                 std = 40
#                             elif 20080101 <= pt <= 20160831:
#                                 std = 20
#                             elif 20160901 <= pt:
#                                 std = 10
#                         elif ci2 == '중형':
#                             if np.isnan(pt):
#                                 std = 0
#                             elif pt < 19921231:
#                                 std = 60
#                             elif 19930101 <= pt <= 19951231:
#                                 std = 55
#                             elif 19960101 <= pt <= 19971231:
#                                 std = 45
#                             elif 19980101 <= pt <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
#                                 std = 45
#                             elif 20010101 <= pt <= 20040930:
#                                 std = 45
#                             elif 20041001 <= pt <= 20071231:
#                                 std = 40
#                             elif 20080101 <= pt <= 20160831:
#                                 std = 20
#                             elif 20160901 <= pt:
#                                 std = 10
#                         elif ci2 == '대형':
#                             if np.isnan(pt):
#                                 std = 0
#                             elif pt < 19921231:
#                                 std = 60
#                             elif 19930101 <= pt <= 19951231:
#                                 std = 55
#                             elif 19960101 <= pt <= 19971231:
#                                 std = 45
#                             elif 19980101 <= pt <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
#                                 std = 45
#                             elif 20010101 <= pt <= 20040930:
#                                 std = 45
#                             elif 20041001 <= pt <= 20071231:
#                                 std = 40
#                             elif 20080101 <= pt:
#                                 std = 20
#             else: # 6번째 값 존재
#                 g = temp['무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6'].mean()
#                 f = f6
#                 # 매연 허용치
#                 if str(ci) == 'nan' or str(ci2) == 'nan':
#                     std = 0
#                 elif ci == '경형' or ci2 == '승용':
#                     if np.isnan(pt):
#                         std = 0
#                     elif pt < 19951231:
#                         std = 60
#                     elif 19960101 <= pt <= 20001231:
#                         std = 55
#                     elif 20010101 <= pt <= 20031231:
#                         std = 45
#                     elif 20040101 <= pt <= 20071231:
#                         std = 40
#                     elif 20080101 <= pt <= 20160831:
#                         std = 20
#                     elif 20160901 <= pt:
#                         std = 10
#                 else:
#                     if ci2 == '소형':
#                         if np.isnan(pt):
#                             std = 0
#                         elif pt < 19951231:
#                             std = 60
#                         elif 19960101 <= pt <= 20001231:
#                             std = 55
#                         elif 20010101 <= pt <= 20031231:
#                             std = 45
#                         elif 20040101 <= pt <= 20071231:
#                             std = 40
#                         elif 20080101 <= pt <= 20160831:
#                             std = 20
#                         elif 20160901 <= pt:
#                             std = 10
#                     elif ci2 == '중형':
#                         if np.isnan(pt):
#                             std = 0
#                         elif pt < 19921231:
#                             std = 60
#                         elif 19930101 <= pt <= 19951231:
#                             std = 55
#                         elif 19960101 <= pt <= 19971231:
#                             std = 45
#                         elif 19980101 <= pt <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
#                             std = 45
#                         elif 20010101 <= pt <= 20040930:
#                             std = 45
#                         elif 20041001 <= pt <= 20071231:
#                             std = 40
#                         elif 20080101 <= pt <= 20160831:
#                             std = 20
#                         elif 20160901 <= pt:
#                             std = 10
#                     elif ci2 == '대형':
#                         if np.isnan(pt):
#                             std = 0
#                         elif pt < 19921231:
#                             std = 60
#                         elif 19930101 <= pt <= 19951231:
#                             std = 55
#                         elif 19960101 <= pt <= 19971231:
#                             std = 45
#                         elif 19980101 <= pt <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
#                             std = 45
#                         elif 20010101 <= pt <= 20040930:
#                             std = 45
#                         elif 20041001 <= pt <= 20071231:
#                             std = 40
#                         elif 20080101 <= pt:
#                             std = 20

#         g_mea_list.append(g)
#         p_f_list.append(f)

In [217]:
temp

,차대번호,차량제작일자,배출가스인증번호,총중량,검사방법,차종유형,차종등록정보,연료,무부하매연측정치1,무부하매연허용치1,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
0,KM99H4ACD8CZ00001,20080705.0,7MY-HD-14-68,2905.0,부하검사(KD-147),중형,승합,경유,0.0,15.0,...,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,20200304 11:08:28,정밀
1,KMF2CY7JABU693781,20101118.0,7MY-HD-14-68,3250.0,무부하검사(급가속),소형,화물,경유,19.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191216 11:09:20,정기
2,KMFACS7JP9U482553,20081215.0,7MY-HD-14-68,2955.0,부하검사(KD-147),소형,화물,경유,13.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200104 12:10:22,정밀
3,KMFACY7JABU652019,20100706.0,7MY-HD-14-68,3270.0,무부하검사(급가속),소형,화물,경유,5.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210723 10:02:36,정기
4,KMFACZ7JABU750953,20110429.0,7MY-HD-14-68,2950.0,무부하검사(급가속),소형,화물,경유,21.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190821 10:04:40,정기
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261123,KNFZCD7JP9U509805,20090415.0,7MY-HD-14-68,3280.0,무부하검사(급가속),소형,화물,경유,7.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200414 15:22:19,정기
261124,KNFZCD7JP9U523649,20090530.0,7MY-HD-14-68,3250.0,부하검사(KD-147),소형,화물,경유,9.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200511 14:56:01,정밀
261125,KNFZCZ7JAAU607457,20100130.0,7MY-HD-14-68,2925.0,무부하검사(급가속),소형,화물,경유,0.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220318 10:35:50,정기
261126,KNFZCZ7JBAU579855,20091110.0,7MY-HD-14-68,2960.0,무부하검사(급가속),소형,화물,경유,3.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191129 15:26:53,정기


In [218]:
temp['무부하매연측정치6']

0         0.0
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
261123    NaN
261124    NaN
261125    NaN
261126    NaN
261127    NaN
Name: 무부하매연측정치6, Length: 261128, dtype: float64

In [219]:
test = pd.DataFrame({1:[1,2,3], 2:[1, np.nan, 3], 3:[1, 2, 's']})
test

,1,2,3
0,1,1.0,1
1,2,NaN,2
2,3,3.0,s


In [220]:
test.loc[1]

1      2
2    NaN
3      2
Name: 1, dtype: object

In [221]:
test.loc[1].mean()

2.0

In [222]:
one = '7MY-KM-14-63'

In [223]:
btemp = d_total2[d_total2['배출가스인증번호'] == one].reset_index(drop=True)
# 1개의 배출가스 인증번호에 대한 처리 테스트
g_mea_list = [] # 샘플 1개에 대한 측정치 저장 리스트
p_f_list = [] # 샘플 1개에 대한 판정 저장 리스트
for i in range(btemp.shape[0]):
    temp = btemp.loc[i]

    test_method = temp['검사방법'] # 변수 생성(검사방법)

    product_date = temp['차량제작일자']# 변수 생성(차량제작일자)

    test_date = temp['검사판정시간'] # 변수 생성(검사판정시간)

    total_weight = temp['총중량'] # 변수 생성(총중량)

    ci = temp['차종등록정보'] # 승용, 승합, 화물, 특수
    ci2 = temp['차종유형'] # 경형, 소형, 중형, 대형

    # 변수 생성(측정치, 문자열)
    sv1 = str(temp['무부하매연측정치1'])
    sv2 = str(temp['무부하매연측정치2'])
    sv3 = str(temp['무부하매연측정치3'])
    sv4 = str(temp['무부하매연측정치4'])
    sv5 = str(temp['무부하매연측정치5'])
    sv6 = str(temp['무부하매연측정치6'])

    # 변수 생성(측정치, 문자열)
    v1 = temp['무부하매연측정치1']
    v2 = temp['무부하매연측정치2']
    v3 = temp['무부하매연측정치3']
    v4 = temp['무부하매연측정치4']
    v5 = temp['무부하매연측정치5']
    v6 = temp['무부하매연측정치6']

    # 변수 생성(판정)
    f1 = temp['무부하매연판정1']
    f2 = temp['무부하매연판정2']
    f3 = temp['무부하매연판정3']
    f4 = temp['무부하매연판정4']
    f5 = temp['무부하매연판정5']
    f6 = temp['무부하매연판정6']
    
    # 검사판정시간
    date = temp['검사판정시간']

    # 평균 매연, 불합격률 계산
    if test_method == '부하검사(LUG DOWN)': # 1번째 측정치, 판정치 사용
        if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
            # print('1번째 값 부재')
            g = np.nan
            f = np.nan
        else: # 1번째 값 존재
            g = v1
            f = f1

    elif test_method == '부하검사(KD-147)': # 1번째 측정치, 판정치 사용
        if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
            # print('1번째 값 부재')
            g = np.nan
            f = np.nan
        else: # 1번째 값 존재
            g = v1
            f = f1
    else: # 무부하 검사
        if sv6 == 'nan' or not gas_mea_value_p.search(sv6) or f6 not in ['Y', 'N']: # 6번째 값 부재
            # print('6번째 값 부재')
            if sv5 == 'nan' or not gas_mea_value_p.search(sv5) or f5 not in ['Y', 'N']: # 5번째 값 부재
                # print('5번째 값 부재')
                if sv4 == 'nan' or not gas_mea_value_p.search(sv4) or f4 not in ['Y', 'N']: # 4번째 값 부재
                    # print('4번째 값 부재')
                    if sv3 == 'nan' or not gas_mea_value_p.search(sv3) or f3 not in ['Y', 'N']: # 3번째 값 부재
                        # print('3번째 값 부재')
                        if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                            # print('1번째 값 부재')
                            g = np.nan
                            f = np.nan
                        else: # 1번째 값 존재
                            g = v1
                            f = f1                                
                    else: # 3번째 값 존재
                        g = temp[['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3']].mean()
                        f = f3
                else: # 4번째 값 존재
                    g = temp[['무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4']].mean()
                    f = f4
            else: # 5번째 값 존재
                g = temp[['무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5']].mean()
                f = f5
        else: # 6번째 값 존재
            g = temp[['무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6']].mean()
            f = f6

    g_mea_list.append(g)
    p_f_list.append(f)

# df 생성
one_sample = pd.DataFrame({'측정치':g_mea_list, '판정':p_f_list})
    
# Grade 계산
m = one_sample['측정치'].mean()
p_f_rate_df = one_sample['판정'].value_counts(normalize=True).reset_index()
if p_f_rate_df.shape[0] == 1:
    p_f_rate = 0
else:
    p_f_rate = p_f_rate_df.loc[p_f_rate_df['index'] == 'N', '판정'].values[0]

In [224]:
btemp.head()

,차대번호,차량제작일자,배출가스인증번호,총중량,검사방법,차종유형,차종등록정보,연료,무부하매연측정치1,무부하매연허용치1,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
0,1J4GAE4509L702033,20081127.0,7MY-KM-14-63,2245.0,부하검사(KD-147),대형,승용,경유,0.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201116 14:27:44,정밀
1,1J4GAE4509L732357,20090326.0,7MY-KM-14-63,2245.0,무부하검사(급가속),대형,승용,경유,0.0,25.0,...,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,20210428 10:30:49,정밀
2,1J4GAE4509L732360,20090114.0,7MY-KM-14-63,2245.0,부하검사(KD-147),대형,승용,경유,0.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210209 11:04:05,정밀
3,1J4GAE4509L739096,20090720.0,7MY-KM-14-63,2245.0,부하검사(KD-147),대형,승용,경유,0.0,15.0,...,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,20210624 13:43:48,정밀
4,1J4GAE4509L739101,20090128.0,7MY-KM-14-63,2245.0,부하검사(KD-147),대형,승용,경유,0.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190705 17:19:03,정밀


In [225]:
one_sample

,측정치,판정
0,0.0,Y
1,0.0,Y
2,0.0,Y
3,0.0,Y
4,0.0,Y
...,...,...
21679,11.0,Y
21680,4.0,Y
21681,0.0,Y
21682,12.0,Y


In [226]:
one_sample2 = one_sample[one_sample['판정'] == 'Y']
one_sample2

,측정치,판정
0,0.0,Y
1,0.0,Y
2,0.0,Y
3,0.0,Y
4,0.0,Y
...,...,...
21679,11.0,Y
21680,4.0,Y
21681,0.0,Y
21682,12.0,Y


In [227]:
m

11.314162436548223

In [228]:
p_f_rate_df

,index,판정
0,Y,0.897139
1,N,0.102861


In [229]:
p_f_rate_df.shape[0]

2

In [230]:
p_f_rate_df.loc[p_f_rate_df['index'] == 'N', '판정']

1    0.102861
Name: 판정, dtype: float64

In [231]:
p_f_rate_df.loc[p_f_rate_df['index'] == 'N', '판정'].values[0]

0.10286109829257037

In [232]:
p_f_rate

0.10286109829257037

In [233]:
one_sample2['판정'].value_counts(normalize=True)

Y    1.0
Name: 판정, dtype: float64

In [234]:
if one_sample2['판정'].value_counts(normalize=True).shape[0] == 1:
    p_f_rate = 0
else:
    p_f_rate = one_sample2['판정'].value_counts(normalize=True).loc[one_sample2['판정'].value_counts(normalize=True)['index'] == 'N', '판정'].values[0]

In [235]:
p_f_rate

0

### total 계산

In [410]:
uni2 = d_total2['배출가스인증번호'].unique()
len(uni2)

204

In [411]:
emno_list = [] # 배출가스인증번호 저장 리스트
car_num_list = []
total_point_list = [] # total 점수 저장 리스트
gas_mea_value_p = re.compile('\d[.]\d') # 값이 소수형인지 확인

for one in tqdm(uni2):
    btemp = d_total2[d_total2['배출가스인증번호'] == one].reset_index(drop=True)
    # 1개의 배출가스 인증번호에 대한 처리 테스트
    g_mea_list = [] # 샘플 1개에 대한 측정치 저장 리스트
    p_f_list = [] # 샘플 1개에 대한 판정 저장 리스트
    for i in range(btemp.shape[0]):
        temp = btemp.loc[i]

        test_method = temp['검사방법'] # 변수 생성(검사방법)

        product_date = temp['차량제작일자']# 변수 생성(차량제작일자)

        test_date = temp['검사판정시간'] # 변수 생성(검사판정시간)

        total_weight = temp['총중량'] # 변수 생성(총중량)

        ci = temp['차종등록정보'] # 승용, 승합, 화물, 특수
        ci2 = temp['차종유형'] # 경형, 소형, 중형, 대형

        # 변수 생성(측정치, 문자열)
        sv1 = str(temp['무부하매연측정치1'])
        sv2 = str(temp['무부하매연측정치2'])
        sv3 = str(temp['무부하매연측정치3'])
        sv4 = str(temp['무부하매연측정치4'])
        sv5 = str(temp['무부하매연측정치5'])
        sv6 = str(temp['무부하매연측정치6'])

        # 변수 생성(측정치, 문자열)
        v1 = temp['무부하매연측정치1']
        v2 = temp['무부하매연측정치2']
        v3 = temp['무부하매연측정치3']
        v4 = temp['무부하매연측정치4']
        v5 = temp['무부하매연측정치5']
        v6 = temp['무부하매연측정치6']

        # 변수 생성(판정)
        f1 = temp['무부하매연판정1']
        f2 = temp['무부하매연판정2']
        f3 = temp['무부하매연판정3']
        f4 = temp['무부하매연판정4']
        f5 = temp['무부하매연판정5']
        f6 = temp['무부하매연판정6']
        
        # 검사판정시간
        date = temp['검사판정시간']

        # 평균 매연, 불합격률 계산
        if test_method == '부하검사(LUG DOWN)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                g = np.nan
                f = np.nan
            else: # 1번째 값 존재
                g = v1
                f = f1

        elif test_method == '부하검사(KD-147)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                g = np.nan
                f = np.nan
            else: # 1번째 값 존재
                g = v1
                f = f1
        else: # 무부하 검사
            if sv6 == 'nan' or not gas_mea_value_p.search(sv6) or f6 not in ['Y', 'N']: # 6번째 값 부재
                # print('6번째 값 부재')
                if sv5 == 'nan' or not gas_mea_value_p.search(sv5) or f5 not in ['Y', 'N']: # 5번째 값 부재
                    # print('5번째 값 부재')
                    if sv4 == 'nan' or not gas_mea_value_p.search(sv4) or f4 not in ['Y', 'N']: # 4번째 값 부재
                        # print('4번째 값 부재')
                        if sv3 == 'nan' or not gas_mea_value_p.search(sv3) or f3 not in ['Y', 'N']: # 3번째 값 부재
                            # print('3번째 값 부재')
                            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                                # print('1번째 값 부재')
                                g = np.nan
                                f = np.nan
                            else: # 1번째 값 존재
                                g = v1
                                f = f1                                
                        else: # 3번째 값 존재
                            g = temp[['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3']].mean()
                            f = f3
                    else: # 4번째 값 존재
                        g = temp[['무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4']].mean()
                        f = f4
                else: # 5번째 값 존재
                    g = temp[['무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5']].mean()
                    f = f5
            else: # 6번째 값 존재
                g = temp[['무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6']].mean()
                f = f6

        g_mea_list.append(g)
        p_f_list.append(f)

    # df 생성
    one_sample = pd.DataFrame({'측정치':g_mea_list, '판정':p_f_list})
        
    # Grade 계산
    m = one_sample['측정치'].mean()
    p_f_rate_df = one_sample['판정'].value_counts(normalize=True).reset_index()
    if p_f_rate_df.shape[0] == 1:
        p_f_rate = 0
    else:
        p_f_rate = p_f_rate_df.loc[p_f_rate_df['index'] == 'N', '판정'].values[0]

    for i in range(btemp.shape[0]): # 배인번호 1의 df의 한줄씩 처리
        car_num = btemp.loc[i, '차대번호'] # 샘플의 차대번호
        temp = btemp.loc[i]

        test_method = temp['검사방법'] # 변수 생성(검사방법)

        product_date = temp['차량제작일자']# 변수 생성(차량제작일자)

        test_date = temp['검사판정시간'] # 변수 생성(검사판정시간)

        total_weight = temp['총중량'] # 변수 생성(총중량)

        ci = temp['차종등록정보'] # 승용, 승합, 화물, 특수
        ci2 = temp['차종유형'] # 경형, 소형, 중형, 대형

        # 변수 생성(측정치, 문자열)
        sv1 = str(temp['무부하매연측정치1'])

        sv3 = str(temp['무부하매연측정치3'])
        sv4 = str(temp['무부하매연측정치4'])
        sv5 = str(temp['무부하매연측정치5'])
        sv6 = str(temp['무부하매연측정치6'])

        # 변수 생성(판정)
        f1 = temp['무부하매연판정1']

        f3 = temp['무부하매연판정3']
        f4 = temp['무부하매연판정4']
        f5 = temp['무부하매연판정5']
        f6 = temp['무부하매연판정6']
        
        # 검사판정시간
        date = temp['검사판정시간']

        if test_method == '부하검사(LUG DOWN)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                std = 0 
            else: # 1번째 값 존재
                # 매연 허용치
                if np.isnan(total_weight):
                    std = 0
                elif total_weight <= 5500:
                    if np.isnan(product_date):
                        std = 0
                    elif product_date <= 19951231:
                        std = 70
                    elif 19960101 <= product_date <= 20001231:
                        std = 60
                    elif 20010101 <= product_date <= 20071231:
                        std = 50
                    elif 20080101 <= product_date:
                        std = 20
                else:
                    if np.isnan(product_date):
                        std = 0
                    elif product_date <= 19951231:
                        std = 50
                    elif 19960101 <= product_date <= 20001231:
                        std = 45
                    elif 20010101 <= product_date <= 20071231:
                        std = 30
                    elif 20080101 <= product_date:
                        std = 15

        elif test_method == '부하검사(KD-147)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                std = 0
            else: # 1번째 값 존재
                # 매연 허용치
                if np.isnan(product_date):
                    std = 0
                elif product_date < 19921231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 50
                    else:
                        std = 45
                elif 19930101 <= product_date <= 19951231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 45
                    else:
                        std = 40
                elif 19960101 <= product_date <= 20001231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 40
                    else:
                        std = 35
                elif 20010101 <= product_date <= 20071231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 30
                    else:
                        std = 25
                elif 20080101 <= product_date <= 20160831:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 20
                    else:
                        std = 15
                elif 20160901 <= product_date <= 20171231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 0
                    else:
                        std = 8
                elif 20180101 <= product_date:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 0
                    else:
                        std = 8

        else: # 무부하 검사
            if sv6 == 'nan' or not gas_mea_value_p.search(sv6) or f6 not in ['Y', 'N']: # 6번째 값 부재
                # print('6번째 값 부재')
                if sv5 == 'nan' or not gas_mea_value_p.search(sv5) or f5 not in ['Y', 'N']: # 5번째 값 부재
                    # print('5번째 값 부재')
                    if sv4 == 'nan' or not gas_mea_value_p.search(sv4) or f4 not in ['Y', 'N']: # 4번째 값 부재
                        # print('4번째 값 부재')
                        if sv3 == 'nan' or not gas_mea_value_p.search(sv3) or f3 not in ['Y', 'N']: # 3번째 값 부재
                            # print('3번째 값 부재')
                            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                                # print('1번째 값 부재')
                                std = 0
                            else: # 1번째 값 존재
                                # 매연 허용치
                                if str(ci) == 'nan' or str(ci2) == 'nan':
                                    std = 0
                                elif ci == '경형' or ci2 == '승용':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19951231:
                                        std = 60
                                    elif 19960101 <= product_date <= 20001231:
                                        std = 55
                                    elif 20010101 <= product_date <= 20031231:
                                        std = 45
                                    elif 20040101 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date <= 20160831:
                                        std = 20
                                    elif 20160901 <= product_date:
                                        std = 10
                                else:
                                    if ci2 == '소형':
                                        if np.isnan(product_date):
                                            std = 0
                                        elif product_date < 19951231:
                                            std = 60
                                        elif 19960101 <= product_date <= 20001231:
                                            std = 55
                                        elif 20010101 <= product_date <= 20031231:
                                            std = 45
                                        elif 20040101 <= product_date <= 20071231:
                                            std = 40
                                        elif 20080101 <= product_date <= 20160831:
                                            std = 20
                                        elif 20160901 <= product_date:
                                            std = 10
                                    elif ci2 == '중형':
                                        if np.isnan(product_date):
                                            std = 0
                                        elif product_date < 19921231:
                                            std = 60
                                        elif 19930101 <= product_date <= 19951231:
                                            std = 55
                                        elif 19960101 <= product_date <= 19971231:
                                            std = 45
                                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                            std = 45
                                        elif 20010101 <= product_date <= 20040930:
                                            std = 45
                                        elif 20041001 <= product_date <= 20071231:
                                            std = 40
                                        elif 20080101 <= product_date <= 20160831:
                                            std = 20
                                        elif 20160901 <= product_date:
                                            std = 10
                                    elif ci2 == '대형':
                                        if np.isnan(product_date):
                                            std = 0
                                        elif product_date < 19921231:
                                            std = 60
                                        elif 19930101 <= product_date <= 19951231:
                                            std = 55
                                        elif 19960101 <= product_date <= 19971231:
                                            std = 45
                                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                            std = 45
                                        elif 20010101 <= product_date <= 20040930:
                                            std = 45
                                        elif 20041001 <= product_date <= 20071231:
                                            std = 40
                                        elif 20080101 <= product_date:
                                            std = 20
                                    
                        else: # 3번째 값 존재
                            # 매연 허용치
                            if str(ci) == 'nan' or str(ci2) == 'nan':
                                std = 0
                            elif ci == '경형' or ci2 == '승용':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19951231:
                                    std = 60
                                elif 19960101 <= product_date <= 20001231:
                                    std = 55
                                elif 20010101 <= product_date <= 20031231:
                                    std = 45
                                elif 20040101 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date <= 20160831:
                                    std = 20
                                elif 20160901 <= product_date:
                                    std = 10
                            else:
                                if ci2 == '소형':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19951231:
                                        std = 60
                                    elif 19960101 <= product_date <= 20001231:
                                        std = 55
                                    elif 20010101 <= product_date <= 20031231:
                                        std = 45
                                    elif 20040101 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date <= 20160831:
                                        std = 20
                                    elif 20160901 <= product_date:
                                        std = 10
                                elif ci2 == '중형':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19921231:
                                        std = 60
                                    elif 19930101 <= product_date <= 19951231:
                                        std = 55
                                    elif 19960101 <= product_date <= 19971231:
                                        std = 45
                                    elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                        std = 45
                                    elif 20010101 <= product_date <= 20040930:
                                        std = 45
                                    elif 20041001 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date <= 20160831:
                                        std = 20
                                    elif 20160901 <= product_date:
                                        std = 10
                                elif ci2 == '대형':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19921231:
                                        std = 60
                                    elif 19930101 <= product_date <= 19951231:
                                        std = 55
                                    elif 19960101 <= product_date <= 19971231:
                                        std = 45
                                    elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                        std = 45
                                    elif 20010101 <= product_date <= 20040930:
                                        std = 45
                                    elif 20041001 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date:
                                        std = 20
                    else: # 4번째 값 존재
                        # 매연 허용치
                        if str(ci) == 'nan' or str(ci2) == 'nan':
                            std = 0
                        elif ci == '경형' or ci2 == '승용':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19951231:
                                std = 60
                            elif 19960101 <= product_date <= 20001231:
                                std = 55
                            elif 20010101 <= product_date <= 20031231:
                                std = 45
                            elif 20040101 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date <= 20160831:
                                std = 20
                            elif 20160901 <= product_date:
                                std = 10
                        else:
                            if ci2 == '소형':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19951231:
                                    std = 60
                                elif 19960101 <= product_date <= 20001231:
                                    std = 55
                                elif 20010101 <= product_date <= 20031231:
                                    std = 45
                                elif 20040101 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date <= 20160831:
                                    std = 20
                                elif 20160901 <= product_date:
                                    std = 10
                            elif ci2 == '중형':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19921231:
                                    std = 60
                                elif 19930101 <= product_date <= 19951231:
                                    std = 55
                                elif 19960101 <= product_date <= 19971231:
                                    std = 45
                                elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                    std = 45
                                elif 20010101 <= product_date <= 20040930:
                                    std = 45
                                elif 20041001 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date <= 20160831:
                                    std = 20
                                elif 20160901 <= product_date:
                                    std = 10
                            elif ci2 == '대형':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19921231:
                                    std = 60
                                elif 19930101 <= product_date <= 19951231:
                                    std = 55
                                elif 19960101 <= product_date <= 19971231:
                                    std = 45
                                elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                    std = 45
                                elif 20010101 <= product_date <= 20040930:
                                    std = 45
                                elif 20041001 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date:
                                    std = 20
                else: # 5번째 값 존재
                    # 매연 허용치
                    if str(ci) == 'nan' or str(ci2) == 'nan':
                        std = 0
                    elif ci == '경형' or ci2 == '승용':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19951231:
                            std = 60
                        elif 19960101 <= product_date <= 20001231:
                            std = 55
                        elif 20010101 <= product_date <= 20031231:
                            std = 45
                        elif 20040101 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date <= 20160831:
                            std = 20
                        elif 20160901 <= product_date:
                            std = 10
                    else:
                        if ci2 == '소형':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19951231:
                                std = 60
                            elif 19960101 <= product_date <= 20001231:
                                std = 55
                            elif 20010101 <= product_date <= 20031231:
                                std = 45
                            elif 20040101 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date <= 20160831:
                                std = 20
                            elif 20160901 <= product_date:
                                std = 10
                        elif ci2 == '중형':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19921231:
                                std = 60
                            elif 19930101 <= product_date <= 19951231:
                                std = 55
                            elif 19960101 <= product_date <= 19971231:
                                std = 45
                            elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                std = 45
                            elif 20010101 <= product_date <= 20040930:
                                std = 45
                            elif 20041001 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date <= 20160831:
                                std = 20
                            elif 20160901 <= product_date:
                                std = 10
                        elif ci2 == '대형':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19921231:
                                std = 60
                            elif 19930101 <= product_date <= 19951231:
                                std = 55
                            elif 19960101 <= product_date <= 19971231:
                                std = 45
                            elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                std = 45
                            elif 20010101 <= product_date <= 20040930:
                                std = 45
                            elif 20041001 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date:
                                std = 20
            else: # 6번째 값 존재
                # 매연 허용치
                if str(ci) == 'nan' or str(ci2) == 'nan':
                    std = 0
                elif ci == '경형' or ci2 == '승용':
                    if np.isnan(product_date):
                        std = 0
                    elif product_date < 19951231:
                        std = 60
                    elif 19960101 <= product_date <= 20001231:
                        std = 55
                    elif 20010101 <= product_date <= 20031231:
                        std = 45
                    elif 20040101 <= product_date <= 20071231:
                        std = 40
                    elif 20080101 <= product_date <= 20160831:
                        std = 20
                    elif 20160901 <= product_date:
                        std = 10
                else:
                    if ci2 == '소형':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19951231:
                            std = 60
                        elif 19960101 <= product_date <= 20001231:
                            std = 55
                        elif 20010101 <= product_date <= 20031231:
                            std = 45
                        elif 20040101 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date <= 20160831:
                            std = 20
                        elif 20160901 <= product_date:
                            std = 10
                    elif ci2 == '중형':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19921231:
                            std = 60
                        elif 19930101 <= product_date <= 19951231:
                            std = 55
                        elif 19960101 <= product_date <= 19971231:
                            std = 45
                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                            std = 45
                        elif 20010101 <= product_date <= 20040930:
                            std = 45
                        elif 20041001 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date <= 20160831:
                            std = 20
                        elif 20160901 <= product_date:
                            std = 10
                    elif ci2 == '대형':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19921231:
                            std = 60
                        elif 19930101 <= product_date <= 19951231:
                            std = 55
                        elif 19960101 <= product_date <= 19971231:
                            std = 45
                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                            std = 45
                        elif 20010101 <= product_date <= 20040930:
                            std = 45
                        elif 20041001 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date:
                            std = 20

        # Total 값 계산
        if std == 0:
            gas_point = 100 - m*100
        else:
            gas_point = 100 - m*100/std

        fail_point = 100 - p_f_rate * 100
        t = gas_point*0.9 + fail_point*0.1

        emno_list.append(one)
        car_num_list.append(car_num)
        total_point_list.append(t)

100%|██████████| 204/204 [07:49<00:00,  2.30s/it]


In [412]:
emno_total_df = pd.DataFrame({'차대번호':car_num_list, '배출가스인증번호':emno_list, 'total':total_point_list})

In [413]:
d_total2.shape

(1132386, 28)

In [414]:
d_total2['배출가스인증번호'].isnull().sum()

0

In [415]:
d_total2[d_total2['차대번호'] == '1J4GAE4509L702033']

,차대번호,차량제작일자,배출가스인증번호,총중량,검사방법,차종유형,차종등록정보,연료,무부하매연측정치1,무부하매연허용치1,...,무부하매연허용치4,무부하매연판정4,무부하매연측정치5,무부하매연허용치5,무부하매연판정5,무부하매연측정치6,무부하매연허용치6,무부하매연판정6,검사판정시간,정기정밀
1312,1J4GAE4509L702033,20081127.0,7MY-KM-14-63,2245.0,부하검사(KD-147),대형,승용,경유,0.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201116 14:27:44,정밀


In [416]:
# 검사방법이 있는 데이터에서 배출가스인증번호 없는 데이터 제외
# 1132386
emno_total_df

,차대번호,배출가스인증번호,total
0,1J4GAE4509L702033,7MY-KM-14-63,31.086414
1,1J4GAE4509L732357,7MY-KM-14-63,48.057658
2,1J4GAE4509L732360,7MY-KM-14-63,31.086414
3,1J4GAE4509L739096,7MY-KM-14-63,31.086414
4,1J4GAE4509L739101,7MY-KM-14-63,31.086414
...,...,...,...
1132381,YV2XZX0G1GA786606,FMY-VTK-14-2,96.142857
1132382,YV2XZX0G3GA787708,FMY-VTK-14-2,97.107143
1132383,YV2XZX0G4GA783070,FMY-VTK-14-2,97.107143
1132384,YV2XZX0G5GA787905,FMY-VTK-14-2,96.142857


In [417]:
emno_total_df.isnull().sum()

차대번호        0
배출가스인증번호    0
total       0
dtype: int64

In [418]:
d_total2_1 = d_total2.merge(emno_total_df, on=['차대번호', '배출가스인증번호'], how='left')
d_total2_1.shape

(1132386, 29)

In [419]:
d_total2_1.columns

Index(['차대번호', '차량제작일자', '배출가스인증번호', '총중량', '검사방법', '차종유형', '차종등록정보', '연료',
       '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2',
       '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4',
       '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5',
       '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '검사판정시간', '정기정밀', 'total'],
      dtype='object')

In [420]:
d_total2_1['total'].dtype

dtype('float64')

In [421]:
grade_list = []
for one in tqdm(d_total2_1['total']):
    if 100 >= one > 75:
        grade_list.append('A')
    elif 75 >= one > 50:
        grade_list.append('B')
    elif 50 >= one > 25:
        grade_list.append('C')
    elif 25 >= one >= 0:
        grade_list.append('D')
    else:
        grade_list.append('D')

100%|██████████| 1132386/1132386 [00:00<00:00, 1384868.44it/s]


In [422]:
d_total2_1['Grade'] = grade_list

In [423]:
d_total2_1['Grade'].value_counts(normalize=True).reset_index().sort_values(['index'])

,index,Grade
2,A,0.191689
1,B,0.297966
0,C,0.397056
3,D,0.113289


In [424]:
d_total2_1['Grade'].value_counts()

C    449621
B    337412
A    217066
D    128287
Name: Grade, dtype: int64

In [425]:
d_total2_1['Grade'].value_counts(normalize=True).reset_index().sort_values(['index']).to_excel('analysis/[G4]Grade(mod_rate).xlsx')

#### 4분위수로 등급 조정

In [404]:
d_total2_1['total'].quantile(q=0.25)

46.67689571560259

In [405]:
d_total2_1['total'].quantile(q=0.5)

57.31459852330411

In [406]:
d_total2_1['total'].quantile(q=0.75)

67.17643556496427

In [407]:
grade_list = []
for one in tqdm(d_total2_1['total']):
    if 100 >= one > 67:
        grade_list.append('A')
    elif 67 >= one > 57:
        grade_list.append('B')
    elif 57 >= one > 47:
        grade_list.append('C')
    elif 47 >= one >= 0:
        grade_list.append('D')
    else:
        grade_list.append('D')

100%|██████████| 1132386/1132386 [00:00<00:00, 1361727.62it/s]


In [408]:
d_total2_1['Grade'] = grade_list
d_total2_1['Grade'].value_counts(normalize=True).reset_index().sort_values(['index'])

,index,Grade
2,A,0.250327
1,B,0.252572
3,C,0.232611
0,D,0.264490


In [409]:
d_total2_1['Grade'].value_counts()

D    299505
B    286009
A    283467
C    263405
Name: Grade, dtype: int64

In [398]:
d_total2_1['Grade'].value_counts(normalize=True).reset_index().sort_values(['index']).to_excel('analysis/[G4]Grade(quantile).xlsx')

#### 3/4/2/1 조절

In [268]:
grade_list = []
for one in tqdm(d_total2_1['total']):
    if 100 >= one > 70:
        grade_list.append('A')
    elif 70 >= one > 30:
        grade_list.append('B')
    elif 30 >= one > 10:
        grade_list.append('C')
    elif 10 >= one >= 0:
        grade_list.append('D')
    else:
        grade_list.append('miss')

100%|██████████| 1132386/1132386 [00:00<00:00, 1486537.21it/s]


In [269]:
d_total2_1['Grade'] = grade_list
d_total2_1['Grade'].value_counts(normalize=True).reset_index().sort_values(['index'])

,index,Grade
1,A,0.246755
0,B,0.684610
2,C,0.055619
4,D,0.000091
3,miss,0.012925


In [270]:
d_total2_1['Grade'].value_counts(normalize=True).reset_index().sort_values(['index']).to_excel('analysis/[G4]Grade(3421).xlsx')

#### 3/3/3/1 조절

In [271]:
grade_list = []
for one in tqdm(d_total2_1['total']):
    if 100 >= one > 70:
        grade_list.append('A')
    elif 70 >= one > 40:
        grade_list.append('B')
    elif 40 >= one > 10:
        grade_list.append('C')
    elif 10 >= one >= 0:
        grade_list.append('D')
    else:
        grade_list.append('miss')

100%|██████████| 1132386/1132386 [00:00<00:00, 1471711.65it/s]


In [272]:
d_total2_1['Grade'] = grade_list
d_total2_1['Grade'].value_counts(normalize=True).reset_index().sort_values(['index'])

,index,Grade
1,A,0.246755
0,B,0.621327
2,C,0.118902
4,D,0.000091
3,miss,0.012925


In [273]:
d_total2_1['Grade'].value_counts(normalize=True).reset_index().sort_values(['index']).to_excel('analysis/[G4]Grade(3331).xlsx')

### 연료 종류 코드화

In [704]:
d_total2_1['배출가스등급'] = 4
d_total2_1['연료종류'] = 'D'

#### [로드] 저감장치 부착 상세내역

In [708]:
# 
dpf = pd.read_excel('data/01. 저감장치 부착 상세 내역_Total.xlsx')
dpf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993097 entries, 0 to 993096
Data columns (total 55 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   저감등록번호     993097 non-null  int64  
 1   저감등록상태     993096 non-null  object 
 2   시도         993097 non-null  object 
 3   시군구        992311 non-null  object 
 4   차량등록번호     993097 non-null  object 
 5   현재차량등록번호   76290 non-null   object 
 6   차대번호       993097 non-null  object 
 7   차종         993097 non-null  object 
 8   차량용도       993094 non-null  object 
 9   차량연식       993095 non-null  float64
 10  최초등록일자     957898 non-null  float64
 11  소유자명       990870 non-null  object 
 12  사용자전화번호    974774 non-null  object 
 13  부착업체명      983955 non-null  object 
 14  현재시군구      574316 non-null  object 
 15  부착일자       993095 non-null  object 
 16  저감장치구분     993097 non-null  object 
 17  저감장치종류     769194 non-null  object 
 18  인증모델번호     993094 non-null  object 
 19  장치일련번호     865034 non-n

In [ ]:
dpf.columns

## 연료에 따른 세분류
- '경유':'D', '휘발유':'G', 'LPG(액화석유가스)':'L', 'CNG(압축천연가스)':'C', '알코올':'A' 

In [ ]:
fuel_div_dict = {'경유':'D', '휘발유':'G', 'LPG(액화석유가스)':'L', 'CNG(압축천연가스)':'C', '알코올':'A'}

In [646]:
d_total2['연료'].unique()

array(['경유'], dtype=object)

## [EG] 휘발유/가스(d_total3)

In [649]:
d_total['연료'].unique()

array(['휘발유', 'LPG(액화석유가스)', '경유', '알코올', '기타연료', 'CNG(압축천연가스)'],
      dtype=object)

In [651]:
d_total3 = d_total[(d_total['연료'] == '휘발유') | (d_total['연료'] == '알코올') | (d_total['연료'] == 'LPG(액화석유가스)') | (d_total['연료'] == 'CNG(압축천연가스)')]
d_total3.shape

(360951, 28)

In [660]:
d_total3.columns

Index(['차대번호', '차량제작일자', '배출가스인증번호', '총중량', '검사방법', '차종유형', '차종등록정보', '연료',
       '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2',
       '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4',
       '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5',
       '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '검사판정시간', '정기정밀'],
      dtype='object')

In [ ]:
d_total3['검사방법'].unique()

### 휘발유

In [696]:
gasoline_df = d_total3.reset_index(drop=True).dropna(subset=['검사방법', '배출가스인증번호'])
gasoline_df.shape

(8, 28)

In [ ]:
gasoline_df['검사방법']

In [697]:
gasoline_df['정기정밀'].unique()

array(['정밀'], dtype=object)

In [701]:
gasoline_df.isnull().sum()

차대번호         0
차량제작일자       1
배출가스인증번호     8
총중량          0
검사방법         0
차종유형         0
차종등록정보       0
연료           0
무부하매연측정치1    0
무부하매연허용치1    0
무부하매연판정1     0
무부하매연측정치2    5
무부하매연허용치2    5
무부하매연판정2     8
무부하매연측정치3    5
무부하매연허용치3    5
무부하매연판정3     8
무부하매연측정치4    5
무부하매연허용치4    5
무부하매연판정4     8
무부하매연측정치5    5
무부하매연허용치5    5
무부하매연판정5     8
무부하매연측정치6    3
무부하매연허용치6    3
무부하매연판정6     6
검사판정시간       2
정기정밀         0
dtype: int64

### total 계산

In [699]:
uni_gasoline = gasoline_df['배출가스인증번호'].unique()
len(uni_gasoline)

1

In [ ]:
emno_list = [] # 배출가스인증번호 저장 리스트
car_num_list = []
total_point_list = [] # total 점수 저장 리스트
gas_mea_value_p = re.compile('\d[.]\d') # 값이 소수형인지 확인

for one in tqdm(uni_gasoline):
    btemp = gasoline_df[gasoline_df['배출가스인증번호'] == one].reset_index(drop=True)
    # 1개의 배출가스 인증번호에 대한 처리 테스트
    g_mea_list = [] # 샘플 1개에 대한 측정치 저장 리스트
    p_f_list = [] # 샘플 1개에 대한 판정 저장 리스트
    for i in range(btemp.shape[0]):
        temp = btemp.loc[i]

        test_method = temp['검사방법'] # 변수 생성(검사방법)

        product_date = temp['차량제작일자']# 변수 생성(차량제작일자)

        test_date = temp['검사판정시간'] # 변수 생성(검사판정시간)

        total_weight = temp['총중량'] # 변수 생성(총중량)

        ci = temp['차종등록정보'] # 승용, 승합, 화물, 특수
        ci2 = temp['차종유형'] # 경형, 소형, 중형, 대형

        # 변수 생성(측정치, 문자열)
        sv1 = str(temp['무부하매연측정치1'])
        sv2 = str(temp['무부하매연측정치2'])
        sv3 = str(temp['무부하매연측정치3'])
        sv4 = str(temp['무부하매연측정치4'])
        sv5 = str(temp['무부하매연측정치5'])
        sv6 = str(temp['무부하매연측정치6'])

        # 변수 생성(측정치, 문자열)
        v1 = temp['무부하매연측정치1']
        v2 = temp['무부하매연측정치2']
        v3 = temp['무부하매연측정치3']
        v4 = temp['무부하매연측정치4']
        v5 = temp['무부하매연측정치5']
        v6 = temp['무부하매연측정치6']

        # 변수 생성(판정)
        f1 = temp['무부하매연판정1']
        f2 = temp['무부하매연판정2']
        f3 = temp['무부하매연판정3']
        f4 = temp['무부하매연판정4']
        f5 = temp['무부하매연판정5']
        f6 = temp['무부하매연판정6']
        
        # 검사판정시간
        date = temp['검사판정시간']

        # 평균 매연, 불합격률 계산
        if test_method == '부하검사(LUG DOWN)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                g = np.nan
                f = np.nan
            else: # 1번째 값 존재
                g = v1
                f = f1

        elif test_method == '부하검사(KD-147)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                g = np.nan
                f = np.nan
            else: # 1번째 값 존재
                g = v1
                f = f1
        else: # 무부하 검사
            if sv6 == 'nan' or not gas_mea_value_p.search(sv6) or f6 not in ['Y', 'N']: # 6번째 값 부재
                # print('6번째 값 부재')
                if sv5 == 'nan' or not gas_mea_value_p.search(sv5) or f5 not in ['Y', 'N']: # 5번째 값 부재
                    # print('5번째 값 부재')
                    if sv4 == 'nan' or not gas_mea_value_p.search(sv4) or f4 not in ['Y', 'N']: # 4번째 값 부재
                        # print('4번째 값 부재')
                        if sv3 == 'nan' or not gas_mea_value_p.search(sv3) or f3 not in ['Y', 'N']: # 3번째 값 부재
                            # print('3번째 값 부재')
                            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                                # print('1번째 값 부재')
                                g = np.nan
                                f = np.nan
                            else: # 1번째 값 존재
                                g = v1
                                f = f1                                
                        else: # 3번째 값 존재
                            g = temp[['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3']].mean()
                            f = f3
                    else: # 4번째 값 존재
                        g = temp[['무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4']].mean()
                        f = f4
                else: # 5번째 값 존재
                    g = temp[['무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5']].mean()
                    f = f5
            else: # 6번째 값 존재
                g = temp[['무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6']].mean()
                f = f6

        g_mea_list.append(g)
        p_f_list.append(f)

    # df 생성
    one_sample = pd.DataFrame({'측정치':g_mea_list, '판정':p_f_list})
        
    # Grade 계산
    m = one_sample['측정치'].mean()
    p_f_rate_df = one_sample['판정'].value_counts(normalize=True).reset_index()
    if p_f_rate_df.shape[0] == 1:
        p_f_rate = 0
    else:
        p_f_rate = p_f_rate_df.loc[p_f_rate_df['index'] == 'N', '판정'].values[0]

    for i in range(btemp.shape[0]): # 배인번호 1의 df의 한줄씩 처리
        car_num = btemp.loc[i, '차대번호'] # 샘플의 차대번호
        temp = btemp.loc[i]

        test_method = temp['검사방법'] # 변수 생성(검사방법)

        product_date = temp['차량제작일자']# 변수 생성(차량제작일자)

        test_date = temp['검사판정시간'] # 변수 생성(검사판정시간)

        total_weight = temp['총중량'] # 변수 생성(총중량)

        ci = temp['차종등록정보'] # 승용, 승합, 화물, 특수
        ci2 = temp['차종유형'] # 경형, 소형, 중형, 대형

        # 변수 생성(측정치, 문자열)
        sv1 = str(temp['무부하매연측정치1'])

        sv3 = str(temp['무부하매연측정치3'])
        sv4 = str(temp['무부하매연측정치4'])
        sv5 = str(temp['무부하매연측정치5'])
        sv6 = str(temp['무부하매연측정치6'])

        # 변수 생성(판정)
        f1 = temp['무부하매연판정1']

        f3 = temp['무부하매연판정3']
        f4 = temp['무부하매연판정4']
        f5 = temp['무부하매연판정5']
        f6 = temp['무부하매연판정6']
        
        # 검사판정시간
        date = temp['검사판정시간']

        if test_method == '부하검사(LUG DOWN)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                std = 0 
            else: # 1번째 값 존재
                # 매연 허용치
                if np.isnan(total_weight):
                    std = 0
                elif total_weight <= 5500:
                    if np.isnan(product_date):
                        std = 0
                    elif product_date <= 19951231:
                        std = 70
                    elif 19960101 <= product_date <= 20001231:
                        std = 60
                    elif 20010101 <= product_date <= 20071231:
                        std = 50
                    elif 20080101 <= product_date:
                        std = 20
                else:
                    if np.isnan(product_date):
                        std = 0
                    elif product_date <= 19951231:
                        std = 50
                    elif 19960101 <= product_date <= 20001231:
                        std = 45
                    elif 20010101 <= product_date <= 20071231:
                        std = 30
                    elif 20080101 <= product_date:
                        std = 15

        elif test_method == '부하검사(KD-147)': # 1번째 측정치, 판정치 사용
            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                # print('1번째 값 부재')
                std = 0
            else: # 1번째 값 존재
                # 매연 허용치
                if np.isnan(product_date):
                    std = 0
                elif product_date < 19921231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 50
                    else:
                        std = 45
                elif 19930101 <= product_date <= 19951231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 45
                    else:
                        std = 40
                elif 19960101 <= product_date <= 20001231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 40
                    else:
                        std = 35
                elif 20010101 <= product_date <= 20071231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 30
                    else:
                        std = 25
                elif 20080101 <= product_date <= 20160831:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 20
                    else:
                        std = 15
                elif 20160901 <= product_date <= 20171231:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 0
                    else:
                        std = 8
                elif 20180101 <= product_date:
                    if str(test_date) == 'nan':
                        std = 0
                    elif test_date <= '20111231':
                        std = 0
                    else:
                        std = 8

        else: # 무부하 검사
            if sv6 == 'nan' or not gas_mea_value_p.search(sv6) or f6 not in ['Y', 'N']: # 6번째 값 부재
                # print('6번째 값 부재')
                if sv5 == 'nan' or not gas_mea_value_p.search(sv5) or f5 not in ['Y', 'N']: # 5번째 값 부재
                    # print('5번째 값 부재')
                    if sv4 == 'nan' or not gas_mea_value_p.search(sv4) or f4 not in ['Y', 'N']: # 4번째 값 부재
                        # print('4번째 값 부재')
                        if sv3 == 'nan' or not gas_mea_value_p.search(sv3) or f3 not in ['Y', 'N']: # 3번째 값 부재
                            # print('3번째 값 부재')
                            if sv1 == 'nan' or not gas_mea_value_p.search(sv1) or f1 not in ['Y', 'N']: # 1번째 값 부재
                                # print('1번째 값 부재')
                                std = 0
                            else: # 1번째 값 존재
                                # 매연 허용치
                                if str(ci) == 'nan' or str(ci2) == 'nan':
                                    std = 0
                                elif ci == '경형' or ci2 == '승용':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19951231:
                                        std = 60
                                    elif 19960101 <= product_date <= 20001231:
                                        std = 55
                                    elif 20010101 <= product_date <= 20031231:
                                        std = 45
                                    elif 20040101 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date <= 20160831:
                                        std = 20
                                    elif 20160901 <= product_date:
                                        std = 10
                                else:
                                    if ci2 == '소형':
                                        if np.isnan(product_date):
                                            std = 0
                                        elif product_date < 19951231:
                                            std = 60
                                        elif 19960101 <= product_date <= 20001231:
                                            std = 55
                                        elif 20010101 <= product_date <= 20031231:
                                            std = 45
                                        elif 20040101 <= product_date <= 20071231:
                                            std = 40
                                        elif 20080101 <= product_date <= 20160831:
                                            std = 20
                                        elif 20160901 <= product_date:
                                            std = 10
                                    elif ci2 == '중형':
                                        if np.isnan(product_date):
                                            std = 0
                                        elif product_date < 19921231:
                                            std = 60
                                        elif 19930101 <= product_date <= 19951231:
                                            std = 55
                                        elif 19960101 <= product_date <= 19971231:
                                            std = 45
                                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                            std = 45
                                        elif 20010101 <= product_date <= 20040930:
                                            std = 45
                                        elif 20041001 <= product_date <= 20071231:
                                            std = 40
                                        elif 20080101 <= product_date <= 20160831:
                                            std = 20
                                        elif 20160901 <= product_date:
                                            std = 10
                                    elif ci2 == '대형':
                                        if np.isnan(product_date):
                                            std = 0
                                        elif product_date < 19921231:
                                            std = 60
                                        elif 19930101 <= product_date <= 19951231:
                                            std = 55
                                        elif 19960101 <= product_date <= 19971231:
                                            std = 45
                                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                            std = 45
                                        elif 20010101 <= product_date <= 20040930:
                                            std = 45
                                        elif 20041001 <= product_date <= 20071231:
                                            std = 40
                                        elif 20080101 <= product_date:
                                            std = 20
                                    
                        else: # 3번째 값 존재
                            # 매연 허용치
                            if str(ci) == 'nan' or str(ci2) == 'nan':
                                std = 0
                            elif ci == '경형' or ci2 == '승용':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19951231:
                                    std = 60
                                elif 19960101 <= product_date <= 20001231:
                                    std = 55
                                elif 20010101 <= product_date <= 20031231:
                                    std = 45
                                elif 20040101 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date <= 20160831:
                                    std = 20
                                elif 20160901 <= product_date:
                                    std = 10
                            else:
                                if ci2 == '소형':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19951231:
                                        std = 60
                                    elif 19960101 <= product_date <= 20001231:
                                        std = 55
                                    elif 20010101 <= product_date <= 20031231:
                                        std = 45
                                    elif 20040101 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date <= 20160831:
                                        std = 20
                                    elif 20160901 <= product_date:
                                        std = 10
                                elif ci2 == '중형':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19921231:
                                        std = 60
                                    elif 19930101 <= product_date <= 19951231:
                                        std = 55
                                    elif 19960101 <= product_date <= 19971231:
                                        std = 45
                                    elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                        std = 45
                                    elif 20010101 <= product_date <= 20040930:
                                        std = 45
                                    elif 20041001 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date <= 20160831:
                                        std = 20
                                    elif 20160901 <= product_date:
                                        std = 10
                                elif ci2 == '대형':
                                    if np.isnan(product_date):
                                        std = 0
                                    elif product_date < 19921231:
                                        std = 60
                                    elif 19930101 <= product_date <= 19951231:
                                        std = 55
                                    elif 19960101 <= product_date <= 19971231:
                                        std = 45
                                    elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                        std = 45
                                    elif 20010101 <= product_date <= 20040930:
                                        std = 45
                                    elif 20041001 <= product_date <= 20071231:
                                        std = 40
                                    elif 20080101 <= product_date:
                                        std = 20
                    else: # 4번째 값 존재
                        # 매연 허용치
                        if str(ci) == 'nan' or str(ci2) == 'nan':
                            std = 0
                        elif ci == '경형' or ci2 == '승용':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19951231:
                                std = 60
                            elif 19960101 <= product_date <= 20001231:
                                std = 55
                            elif 20010101 <= product_date <= 20031231:
                                std = 45
                            elif 20040101 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date <= 20160831:
                                std = 20
                            elif 20160901 <= product_date:
                                std = 10
                        else:
                            if ci2 == '소형':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19951231:
                                    std = 60
                                elif 19960101 <= product_date <= 20001231:
                                    std = 55
                                elif 20010101 <= product_date <= 20031231:
                                    std = 45
                                elif 20040101 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date <= 20160831:
                                    std = 20
                                elif 20160901 <= product_date:
                                    std = 10
                            elif ci2 == '중형':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19921231:
                                    std = 60
                                elif 19930101 <= product_date <= 19951231:
                                    std = 55
                                elif 19960101 <= product_date <= 19971231:
                                    std = 45
                                elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                    std = 45
                                elif 20010101 <= product_date <= 20040930:
                                    std = 45
                                elif 20041001 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date <= 20160831:
                                    std = 20
                                elif 20160901 <= product_date:
                                    std = 10
                            elif ci2 == '대형':
                                if np.isnan(product_date):
                                    std = 0
                                elif product_date < 19921231:
                                    std = 60
                                elif 19930101 <= product_date <= 19951231:
                                    std = 55
                                elif 19960101 <= product_date <= 19971231:
                                    std = 45
                                elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                    std = 45
                                elif 20010101 <= product_date <= 20040930:
                                    std = 45
                                elif 20041001 <= product_date <= 20071231:
                                    std = 40
                                elif 20080101 <= product_date:
                                    std = 20
                else: # 5번째 값 존재
                    # 매연 허용치
                    if str(ci) == 'nan' or str(ci2) == 'nan':
                        std = 0
                    elif ci == '경형' or ci2 == '승용':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19951231:
                            std = 60
                        elif 19960101 <= product_date <= 20001231:
                            std = 55
                        elif 20010101 <= product_date <= 20031231:
                            std = 45
                        elif 20040101 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date <= 20160831:
                            std = 20
                        elif 20160901 <= product_date:
                            std = 10
                    else:
                        if ci2 == '소형':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19951231:
                                std = 60
                            elif 19960101 <= product_date <= 20001231:
                                std = 55
                            elif 20010101 <= product_date <= 20031231:
                                std = 45
                            elif 20040101 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date <= 20160831:
                                std = 20
                            elif 20160901 <= product_date:
                                std = 10
                        elif ci2 == '중형':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19921231:
                                std = 60
                            elif 19930101 <= product_date <= 19951231:
                                std = 55
                            elif 19960101 <= product_date <= 19971231:
                                std = 45
                            elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                std = 45
                            elif 20010101 <= product_date <= 20040930:
                                std = 45
                            elif 20041001 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date <= 20160831:
                                std = 20
                            elif 20160901 <= product_date:
                                std = 10
                        elif ci2 == '대형':
                            if np.isnan(product_date):
                                std = 0
                            elif product_date < 19921231:
                                std = 60
                            elif 19930101 <= product_date <= 19951231:
                                std = 55
                            elif 19960101 <= product_date <= 19971231:
                                std = 45
                            elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                                std = 45
                            elif 20010101 <= product_date <= 20040930:
                                std = 45
                            elif 20041001 <= product_date <= 20071231:
                                std = 40
                            elif 20080101 <= product_date:
                                std = 20
            else: # 6번째 값 존재
                # 매연 허용치
                if str(ci) == 'nan' or str(ci2) == 'nan':
                    std = 0
                elif ci == '경형' or ci2 == '승용':
                    if np.isnan(product_date):
                        std = 0
                    elif product_date < 19951231:
                        std = 60
                    elif 19960101 <= product_date <= 20001231:
                        std = 55
                    elif 20010101 <= product_date <= 20031231:
                        std = 45
                    elif 20040101 <= product_date <= 20071231:
                        std = 40
                    elif 20080101 <= product_date <= 20160831:
                        std = 20
                    elif 20160901 <= product_date:
                        std = 10
                else:
                    if ci2 == '소형':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19951231:
                            std = 60
                        elif 19960101 <= product_date <= 20001231:
                            std = 55
                        elif 20010101 <= product_date <= 20031231:
                            std = 45
                        elif 20040101 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date <= 20160831:
                            std = 20
                        elif 20160901 <= product_date:
                            std = 10
                    elif ci2 == '중형':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19921231:
                            std = 60
                        elif 19930101 <= product_date <= 19951231:
                            std = 55
                        elif 19960101 <= product_date <= 19971231:
                            std = 45
                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                            std = 45
                        elif 20010101 <= product_date <= 20040930:
                            std = 45
                        elif 20041001 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date <= 20160831:
                            std = 20
                        elif 20160901 <= product_date:
                            std = 10
                    elif ci2 == '대형':
                        if np.isnan(product_date):
                            std = 0
                        elif product_date < 19921231:
                            std = 60
                        elif 19930101 <= product_date <= 19951231:
                            std = 55
                        elif 19960101 <= product_date <= 19971231:
                            std = 45
                        elif 19980101 <= product_date <= 20001231: # 시내버스(40), 그 외(45) 조건 필요
                            std = 45
                        elif 20010101 <= product_date <= 20040930:
                            std = 45
                        elif 20041001 <= product_date <= 20071231:
                            std = 40
                        elif 20080101 <= product_date:
                            std = 20

        # Total 값 계산
        if std == 0:
            gas_point = 100 - m*100
        else:
            gas_point = 100 - m*100/std

        fail_point = 100 - p_f_rate * 100
        t = gas_point*0.7 + fail_point*0.3

        emno_list.append(one)
        car_num_list.append(car_num)
        total_point_list.append(t)

# 코드 마지막